# Εργαστηριακή Άσκηση 2. Μη επιβλεπόμενη μάθηση. 
## Σύστημα συστάσεων βασισμένο στο περιεχόμενο
## Σημασιολογική απεικόνιση δεδομένων με χρήση SOM 
Ημερομηνία εκφώνησης της άσκησης: 29 Νοεμβρίου 2022

**Θα βρείτε το παρόν σε μορφή jupyter notebook ως συνημμένο στο τέλος της εκφώνησης.**


<br>

***Ομάδα 42***
- Γαλανόπουλος Νικόλαος (03118024)
- Νικολόπουλος Χρήστος (03118188)

<br>

In [ ]:
# removed --upgrade. Needed in numpy for `import gensim.downloader` to work.
!pip install --upgrade numpy
!pip install pandas
!pip install nltk
!pip install scikit-learn
!pip install joblib

!pip install -U gensim
!pip install somoclu

!pip install tabulate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.1 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.1 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.22.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.1
    Uninstalling numpy-1.24.1:
      Successfully uninstalled numpy-1.24.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
import scipy
from nltk.corpus import stopwords, names
import string
import re
import time

In [ ]:
# For saving objects

# Make sure to add a shortcut to '/dumps' to your drive!
# https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab

  # from google.colab import drive
  # drive.mount('/content/drive')

  # SHELL_PATH = "\"/content/drive/My Drive/dumps\""
  # PATH = SHELL_PATH.replace('\"', '')

  # import joblib
  # import shutil

  # def save(obj, name):
  #   joblib.dump(obj, f'{PATH}/{name}.pkl') 

  # def load(name):
  #   return joblib.load(f'{PATH}/{name}.pkl')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Εισαγωγή του Dataset

Το σύνολο δεδομένων με το οποίο θα δουλέψουμε είναι βασισμένο στο [Carnegie Mellon Movie Summary Corpus](http://www.cs.cmu.edu/~ark/personas/). Πρόκειται για ένα dataset με 22.301 περιγραφές ταινιών. Η περιγραφή κάθε ταινίας αποτελείται από τον τίτλο της, μια ή περισσότερες ετικέτες που χαρακτηρίζουν το είδος της ταινίας και τέλος τη σύνοψη της υπόθεσής της. Αρχικά εισάγουμε το dataset (χρησιμοποιήστε αυτούσιο τον κώδικα, δεν χρειάζεστε το αρχείο csv) στο dataframe `df_data_1`: 

In [ ]:
import pandas as pd

dataset_url = "https://drive.google.com/uc?export=download&id=1zo13kUAf-MDMPZmBDxq1FxWtZY01lsxD"
df_data_1 = pd.read_csv(dataset_url, sep='\t',  header=None, quoting=3)

Κάθε ομάδα θα δουλέψει σε **ένα μοναδικό υποσύνολο 5.000 ταινιών** (διαφορετικό dataset για κάθε ομάδα) ως εξής:

1. Κάθε ομάδα του εργαστηρίου νευρωνικών έχει έναν αριθμό στο helios. Θα βάλετε τον αριθμό αυτό στη μεταβλητή team_seed_number στο επόμενο κελί κώδικα.

2. Το data frame `df_data_2` έχει γραμμές όσες και οι ομάδες και 5.000 στήλες. Σε κάθε ομάδα αντιστοιχεί η γραμμή του πίνακα με το `team_seed_number` της. Η γραμμή αυτή θα περιλαμβάνει 5.000 διαφορετικούς αριθμούς που αντιστοιχούν σε ταινίες του αρχικού dataset. 

3. Τρέξτε τον κώδικα. Θα προκύψουν τα μοναδικά για κάθε ομάδα  titles, categories, catbins, summaries και corpus με τα οποία θα δουλέψετε.

In [ ]:
import numpy as np

# Στο επόμενη γραμή βάλτε τον αριθμό της ομάδας στο εργαστήριο των νευρωνικών
team_seed_number = 42

movie_seeds_url = "https://drive.google.com/uc?export=download&id=1g6F4TCHrs2wgtdOk7D3gtONaeirNt_Vo"
df_data_2 = pd.read_csv(movie_seeds_url, header=None)

my_index = df_data_2.iloc[team_seed_number,:].values

titles = df_data_1.iloc[:, [2]].values[my_index]      # title(s) of movie (string)
categories = df_data_1.iloc[:, [3]].values[my_index]  # categories(s) of movie (string = comma-separated strings)
bins = df_data_1.iloc[:, [4]]
catbins = bins[4].str.split(',', expand=True).values.astype(float)[my_index]  # movie categories(s) in binary form (1 feature per category/"one-hot"bag)
summaries =  df_data_1.iloc[:, [5]].values[my_index]  # movie summary(s) (string), AS numpy array
corpus = summaries[:, 0].tolist()                     # movie summary(s) (string), AS vanilla LIST
corpus_df = pd.DataFrame(corpus)                      # movie summary(s) (string), AS DATAFRAME, 5000 x 1

- Ο πίνακας **titles** περιέχει τους τίτλους των ταινιών. Παράδειγμα: 'Sid and Nancy'.
- O πίνακας **categories** περιέχει τις κατηγορίες (είδη) της ταινίας υπό τη μορφή string. Παράδειγμα: <font color="red">'</font>"Tragedy",  "Indie",  "Punk rock",  "Addiction Drama",  "Cult",  "Musical",  "Drama",  "Biopic \[feature\]",  "Romantic drama",  "Romance Film",  "Biographical film"<font color="red">'</font>. Παρατηρούμε ότι είναι μια comma separated λίστα strings, με κάθε string να είναι μια κατηγορία.
- Ο πίνακας **catbins** περιλαμβάνει πάλι τις κατηγορίες των ταινιών αλλά σε δυαδική μορφή ([one hot encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)). Έχει διαστάσεις 5.000 x 322 (όσες οι διαφορετικές κατηγορίες). Αν η ταινία ανήκει στο συγκεκριμένο είδος η αντίστοιχη στήλη παίρνει την τιμή 1, αλλιώς παίρνει την τιμή 0.
- Ο πίνακας **summaries** και η λίστα **corpus** περιλαμβάνουν τις συνόψεις των ταινιών (η corpus είναι απλά ο summaries σε μορφή λίστας). Κάθε σύνοψη είναι ένα (συνήθως μεγάλο) string. Παράδειγμα: *'The film is based on the real story of a Soviet Internal Troops soldier who killed his entire unit  as a result of Dedovschina. The plot unfolds mostly on board of the prisoner transport rail car guarded by a unit of paramilitary conscripts.'*
- το dataframe **corpus_df** που είναι απλά το corpus σε μορφή dataframe. Τα summaries βρίσκονται στην κολόνα 0. Πιθανώς να σας βολεύει να κάνετε κάποιες προεπεξεργασίες με dataframes.


Θεωρούμε ως **ID** της κάθε ταινίας τον αριθμό γραμμής της ή το αντίστοιχο στοιχείο της λίστας. Παράδειγμα: για να τυπώσουμε τη σύνοψη της ταινίας με `ID=999` (την χιλιοστή) θα γράψουμε `print(corpus[999])`.

In [ ]:
def movie_info(ID):
  print(f"Title: {titles[ID]}")
  print(f"Genres: {categories[ID]}")
  print(f"Description:\n{corpus[ID]}")
  print()

In [ ]:
ID = 4996
movie_info(ID)

Title: ['Malajanha']
Genres: ['"Drama"']
Description:
Sati,the female protagonist of the movie,is forced into a marriage by her poor parents with a rich ugly old man living with a concubine. She finds the other way to overcome the mishap in her life when a young neighbor Nath develops love interest in her. Both finally leave the village and escape to Cuttack town. The conservative neighbors question their relationship. Sati, unable to bear the scandal concerning her and Natha, finally commits suicide by jumping into the river.



# Εφαρμογή 1. Υλοποίηση συστήματος συστάσεων ταινιών βασισμένο στο περιεχόμενο
<img src="http://clture.org/wp-content/uploads/2015/12/Netflix-Streaming-End-of-Year-Posts.jpg" width="70%">

Η πρώτη εφαρμογή που θα αναπτύξετε θα είναι ένα [σύστημα συστάσεων](https://en.wikipedia.org/wiki/Recommender_system) ταινιών βασισμένο στο περιεχόμενο (content based recommender system). Τα συστήματα συστάσεων στοχεύουν στο να προτείνουν αυτόματα στο χρήστη αντικείμενα από μια συλλογή τα οποία ιδανικά θέλουμε να βρει ενδιαφέροντα ο χρήστης. Η κατηγοριοποίηση των συστημάτων συστάσεων βασίζεται στο πώς γίνεται η επιλογή (filtering) των συστηνόμενων αντικειμένων. Οι δύο κύριες κατηγορίες είναι η <u>συνεργατική διήθηση</u> (collaborative filtering) όπου το σύστημα προτείνει στο χρήστη αντικείμενα που έχουν αξιολογηθεί θετικά από χρήστες που έχουν παρόμοιο με αυτόν ιστορικό αξιολογήσεων και η <u>διήθηση με βάση το περιεχόμενο</u> (content based filtering), όπου προτείνονται στο χρήστη αντικείμενα με παρόμοιο περιεχόμενο (με βάση κάποια χαρακτηριστικά) με αυτά που έχει προηγουμένως αξιολογήσει θετικά.

Το σύστημα συστάσεων που θα αναπτύξετε θα **βασίζεται στο περιεχόμενο** και συγκεκριμένα στις συνόψεις των ταινιών (**corpus**). 


## Προεπεξεργασία

Το πρώτο βήμα στην επεξεργασία μας είναι ο καθαρισμός των περιγραφών των ταινιών. 

Εκτυπώστε (αρκετές) διαφορετικές περιγραφές ταινιών για να δείτε πιθανά προβλήματα που θα πρέπει να αντιμετωπιστούν.

Τα (ελάχιστα) βήματα καθαρισμού που προτείνουμε είναι:
- μετατροπή όλων των χαρακτήρων σε <u>πεζά</u>,
- αφαίρεση των <u>stopwords</u>. Εδώ σημειώστε ότι για το δεδομένο task του συστήματος συστάσεων που είναι η πρόταση ταινιών ίσως θα είχαν ενδιαφέρον και λίστες stopwords πέραν αυτών της κοινής γλώσσας.
- αφαίρεση <u>σημείων στίξης</u> και ειδικών χαρακτρήρων (special characters). Αυτό δεν γίνεται μόνο με την punkt του NLTK. Θα μπορούσατε να βασιστείτε σε κανονικές εκφράσεις (regular expressions), και
- αφαίρεση <u>πολυ σύντομων συμβολοσειρών</u>.

Προσοχή: το corpus και τα τελικά tokens που θα το αποτελούν θα χρησιμοποιηθούν στη συνέχεια ως κλειδιά για να βρούμε εμφυτεύματα. Για το λόγο αυτό, πρέπει να είστε προσεκτικοί ως προς την εφαρμογή μεθόδων κανονικοποίησης (text normalization) όπως το stemming και το lemmatization.

In [ ]:
# Remove "special characters": Keep only A-Z, a-z, 0-9, space and punctuation. 
# Remove links starting from http.
corpus_edf = corpus_df.applymap(lambda s: re.sub(f'[^A-Za-z0-9\s{string.punctuation}]+', '', s)) # corpus_edf = edited corpus_df
corpus_edf = corpus_df.applymap(lambda s: re.sub(r'http\S+', '', s))

# Tokenize
nltk.download('punkt')
corpus_edf = corpus_edf.applymap(lambda s: nltk.word_tokenize(s))

# Remove stopwords & punctuation & common human names. Turn tokens into lowercase.
nltk.download('stopwords')
nltk.download('names')
common_names = names.words('male.txt') + names.words('female.txt')
unwanted_toks = common_names + stopwords.words('english') + list(string.punctuation)
unwanted_toks = set(token.lower() for token in unwanted_toks)
corpus_edf = corpus_edf.applymap(lambda tokens: [tok.lower() for tok in tokens if tok.lower() not in unwanted_toks])

print(corpus_edf)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


                                                      0
0     [movie, opens, caribbean, island, san, moreno,...
1     [count, vertigo, hires, merlyn, assassinate, p...
2     [raja, saab, aged, powerful, greedy, owner, co...
3     [1917, portugal, feeling, aftereffects, masoni...
4     [goundamani, sold, senthamarai, later, deserte...
...                                                 ...
4995  [dugay, runs, small, company, pool, cleans, sw...
4996  [sati, female, protagonist, movie, forced, mar...
4997  [banner, sisters, work, hostesses, new, york, ...
4998  [run-down, neighborhood, castle, wakes, exits,...
4999  [group, middle-aged, men, used, called, 'legen...

[5000 rows x 1 columns]


In [ ]:
# Removes words that consist completely of punctuation or digits.
# It also removes tokens with length less than 2.
def thorough_filter(words):
    filtered_words = []
    unwanted_chars = set(list(string.punctuation) + list(str(i) for i in range(10)))
    for word in words:
       if len(word) > 2:   
          pun = []
          for letter in word:
              pun.append(letter in unwanted_chars)
          if not all(pun):
              filtered_words.append(word)
    return filtered_words
        
corpus_edf = corpus_edf.applymap(lambda w: thorough_filter(w))

In [ ]:
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('rslp')

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

lemmatize = True

if lemmatize:
  wordnet_lemmatizer = WordNetLemmatizer()
  corpus_edf = corpus_edf.applymap(lambda words: [wordnet_lemmatizer.lemmatize(word) for word in words])
else:
  porter_stemmer = PorterStemmer()
  corpus_edf = corpus_edf.applymap(lambda words: [porter_stemmer.stem(word) for word in words])

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [ ]:
new_corpus = corpus_edf.applymap(lambda l: ' '.join(l)).values.flatten()
print(new_corpus)

['movie open caribbean island san moreno trio important looking men meeting island governor paying suitcase full money special operation three men joined via helicopter vladakov second best terrorist world charge operation entire event witnessed young boy named take picture governor men next morning find retired commander mchale making naval base san ysidro trading officer good service include selling home-brewed beer ice cream swimsuit calendar men san ysidro naval base exchange thing like medicine satellite photo help people san moreno satellite photo actually spying opposing baseball team stationed base old crew cigar-chomping lady man team lookout life treehouse team techie moonshine maker gruber slightly portly card-player cigar horder group musclehead unscrew beer bottle eye however base commanded newly arrived captain binghampton lieutenant carpenter capt binghampton belief men gone native confiscates product mchale sold want resurrect career mistakenly sunk luxury cruise liner 

## Μετατροπή σε TFIDF

Το πρώτο βήμα θα είναι λοιπόν να μετατρέψετε το corpus σε αναπαράσταση tf-idf:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create sparse tf_idf representation
vectorizer = TfidfVectorizer()
vectorizer.fit(new_corpus)
corpus_tf_idf_plain = vectorizer.transform(new_corpus)

Η συνάρτηση [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) όπως καλείται εδώ **δεν είναι βελτιστοποιημένη**. Οι επιλογές των μεθόδων και παραμέτρων της μπορεί να έχουν **δραματική επίδραση στην ποιότητα των συστάσεων** και είναι διαφορετικές για κάθε dataset. Επίσης, οι επιλογές αυτές έχουν πολύ μεγάλη επίδραση και στη **διαστατικότητα και όγκο των δεδομένων**. Η διαστατικότητα των δεδομένων με τη σειρά της θα έχει πολύ μεγάλη επίδραση στους **χρόνους εκπαίδευσης**, ιδιαίτερα στη δεύτερη εφαρμογή της άσκησης.

Προσοχή: ο TfidfVectorizer έχει κάποιες δυνατότητες προεπεξεργασίας παρόποιες με αυτές που αναφέραμε στην προηγούμενη ενότητα. Ό,τι προεπεξεργασία μπορείτε να κάνετε που χρειάζεται ως είσοδο μόνο το κάθε document ξεχωριστά, κάντε την στο πρώτο βήμα της προεπεξεργασίας. Αν χρειάζεται γνώση των συνολικών στατιστικών της συλλογής, κάντε την με τον TfidfVectorizer.

In [ ]:
print(corpus_tf_idf_plain.shape)

(5000, 40491)


In [ ]:
# Optimized Call
opt_vectorizer = TfidfVectorizer(max_df = 0.22, min_df = 3)
opt_vectorizer.fit(new_corpus)
corpus_tf_idf = opt_vectorizer.transform(new_corpus)

In [ ]:
print(corpus_tf_idf.shape)

(5000, 14996)


## Υλοποίηση του συστήματος συστάσεων

Το σύστημα συστάσεων που θα υλοποιήσετε θα είναι μια συνάρτηση `content_recommender` με τρία ορίσματα: `target_movie`, `max_recommendations` και `corpus_type`. Στην `target_movie` περνάμε το ID μιας ταινίας-στόχου για την οποία μας ενδιαφέρει να βρούμε παρόμοιες ως προς το περιεχόμενο (τη σύνοψη) ταινίες, `max_recommendations` στο πλήθος.
Υλοποιήστε τη συνάρτηση ως εξής: 
- για την ταινία-στόχο, θα υπολογίζετε την [ομοιότητα συνημιτόνου](https://en.wikipedia.org/wiki/Cosine_similarity) της με όλες τις ταινίες της συλλογής σας όπως αυτές αναπαριστώνται στο `corpus_type`.
- με βάση την ομοιότητα συνημιτόνου που υπολογίσατε, δημιουργήστε ταξινομημένο πίνακα από το μεγαλύτερο στο μικρότερο, με τα indices (`ID`) των ταινιών. Παράδειγμα: αν η ταινία με index 1 έχει ομοιότητα συνημιτόνου με 3 ταινίες \[0.2 1 0.6\] (έχει ομοιότητα 1 με τον εαύτό της) ο ταξινομημένος αυτός πίνακας indices θα είναι \[1 2 0\].
- Για την ταινία-στόχο εκτυπώστε: id, τίτλο, σύνοψη, κατηγορίες (categories)
- Για τις `max_recommendations` ταινίες (πλην της ίδιας της ταινίας-στόχου που έχει cosine similarity 1 με τον εαυτό της) με τη μεγαλύτερη ομοιότητα συνημιτόνου (σε φθίνουσα σειρά), τυπώστε σειρά σύστασης (1 πιο κοντινή, 2 η δεύτερη πιο κοντινή κλπ), ομοιότητα συνημιτόνου, id, τίτλο, σύνοψη, και κατηγορίες (categories)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Simple cosine recommender
def content_recommender(target_movie, max_recommendations, corpus_type):
  similarities = pd.DataFrame(cosine_similarity(corpus_type, corpus_type[target_movie]).flatten())
  return similarities.nlargest(max_recommendations + 1, 0).index.tolist()[1:]

## Βελτιστοποίηση του TfidfVectorizer

Αφού υλοποιήσετε τη συνάρτηση `content_recommender` χρησιμοποιήστε την για να βελτιστοποιήσετε την `TfidfVectorizer`. Συγκεκριμένα, αρχικά μπορείτε να δείτε τι επιστρέφει το σύστημα για τυχαίες ταινίες-στόχους και για ένα μικρό `max_recommendations` (2 ή 3). Αν σε κάποιες ταινίες το σύστημα μοιάζει να επιστρέφει σημασιολογικά κοντινές ταινίες σημειώστε το `ID` τους. Δοκιμάστε στη συνέχεια να βελτιστοποιήσετε την `TfidfVectorizer` για τα συγκεκριμένα `ID` ώστε να επιστρέφονται σημασιολογικά κοντινές ταινίες για μεγαλύτερο αριθμό `max_recommendations`. Παράλληλα, όσο βελτιστοποιείτε την `TfidfVectorizer`, θα πρέπει να λαμβάνετε καλές συστάσεις για μεγαλύτερο αριθμό τυχαίων ταινιών. 

Ταυτόχρονα, μια αντίρροπη κατά κάποιο τρόπο κατεύθυνση της βελτιστοποίησης είναι να χρησιμοποιείτε τις παραμέτρους του `TfidfVectorizer` έτσι ώστε να μειώνονται οι διαστάσεις του Vector Space Model μέχρι το σημείο που θα αρχίσει να εμφανίζονται επιπτώσεις στην ποιότητα των συστάσεων. 




In [ ]:
# Given an ID, the number of recommendations and the corpus that we want to use,
# "try_recommender" acts as a wrapper to the cosine recommender we've written above.
def try_recommender(ID, max_recommendations = 2, movie_corpus = corpus_tf_idf):
  print("Original Movie:\n")
  movie_info(ID)
  inds = content_recommender(ID, max_recommendations, movie_corpus)
  print("Recommended Movies:\n")
  for rank, ind in enumerate(inds): 
    print(f"{rank + 1}.")
    movie_info(ind)

<br>

In [ ]:
# id = 18, terrible recommendations.
# Conclusion based on experiments on id in {24, 177, 188}. Don't lower below max_df > 500.
# max_df >= 0.045 for 24. max_df > 0.15 for 177. > 0.1 for 188. A max_df > 0.1 is okay for 1305 as well.
# 2312 needs min_df == 0.035 for decent recommendations. A hard ID for recommendations.
# 1305 min_df < 5e-05.
# max_df < 0.42 removes a word. max_df >= 0.42 has no effect.
# Candidate: max_df = 0.22. In general max_df between 0.2, 0.22, 0.25.
# min_df > 0.003 for 177 & 24!

<br>

In [ ]:
test = np.random.randint(0, 5000, size = (10, ))

In [ ]:
for max_df in np.linspace(0.2, 0.42, num = int((0.42 - 0.2) / 2e-2)):
    for min_df in range(1, 5):
        test_vectorizer = TfidfVectorizer(max_df = max_df, min_df = min_df)
        test_vectorizer.fit(new_corpus)
        test_corpus = test_vectorizer.transform(new_corpus)

        print(f"Max DF: {max_df}, Min DF: {min_df}")
        for movie in test:
          try_recommender(movie, max_recommendations = 5, movie_corpus = test_corpus)

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
Description:
Nellie Owen is happily married to fisherman John, with whom she has a child, until a philanderer, Frank, convinces her that her husband is unfaithful. She runs away to the city with Frank, where she assists him extracting money from the rich in society, but after a while is consumed by guilt and wants to return home. After an argument with Frank, she arrives in time to help nurse her sick child through a serious illness, and is reunited with her forgiving husband.

Recommended Movies:

1.
Title: ['The Restless and the Damned']
Genres: ['"Drama"']
Description:
In Tahiti, an amibitious woman, Dominique, promotes the fortunes of her husband, George, by extracting money from George's family to finance his operations and seducing a prospective business partner. But when George decides to leave her for another woman, Claire, she tries to kill him.

2.
Title: ['The Lost & Found Family']
Genres: ['"Family Film",  "Family Drama

## Βαθιά μάθηση: δημιουργία corpora με χρήση word embeddings

Η προσέγγιση της κατασκευής μόνο μέσω tfidf του συστήματος συστάσεων έχει διάφορα μειονεκτήματα. Θα μας ενδιέφερε λοιπόν να δούμε αν μπορούμε να χρησιμοποιήσουμε για τις λέξεις **εμφυτεύματα (embeddings)**, δηλαδή τις πυκνές διανυσματικές αναπαραστάσεις για τις λέξεις που μας δίνει το μοντέλο **Word2Vec**

Ωστόσο, το dataset της κάθε ομάδας είναι πολύ μικρό για να εξάγουμε τα δικά μας word embeddings (και να είναι καλά). Για το λόγο αυτό θα χρησιμοποιήσουμε τη μεθοδολογία της Βαθιάς Μάθησης που είναι η **Μεταφορά Μάθησης (Transfer Learning).**.

Στη μεταφορά μάθησης ουσιαστικά μεταφέρουμε τη γνώση που έχει αποκτήσει ένα ήδη εκπαιδευμένο (και κατά κανόνα πολύ μεγάλο) σύστημα. Η μεταφορά γίνεται διαμέσου των τιμών των βαρών που έχει προσδιορίσει μετά το πέρας της εκπαίδευσης.

Στην περίπτωσή μας, δεν μας ενδιαφέρουν τόσο τα ίδια τα βάρη των μοντέλων από τα οποία θα κάνουμε μεταφορά μάθησης. Κάτι τέτοιο θα μας ενδιέφερε αν π.χ. θέλαμε να συνεχίσουμε την εκπαίδευση στα δικά μας κείμενα. Μας ενδιαφέρουν όμως τα ίδια τα εμφυτεύματα, δηλαδή τα embeddings (διανύσματα διαστάσεων $m$) που έχει μάθει το νευρωνικό για το λεξιλόγιο του (vocabulary). To vocabulary σε τέτοια μεγάλα νευρωνικά θα είναι πιθανότατα υπερσύνολο του δικού μας.

### Μεταφορά μάθησης εμφυτευμάτων



#### Εμφυτεύματα του Gensim-data
Το Gensim περιλαμβάνει αρκετά προεκπαιδευμένα μοντέλα εμφυτευμάτων Word2Vec. Με το επόμενο κελί παίρνουμε τη λίστα τους.

In [ ]:
### !pip install -U gensim
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


Τα μοντέλα αυτά βρίσκονται στο [αποθετήριο Gensim-data](https://github.com/RaRe-Technologies/gensim-data) όπου μπορείτε να βρείτε και την τεκμηρίωσή τους. Η φόρτωση των μοντέλων αυτών γίνεται με τη συνάρτηση `gensim.downloader.load`.

#### Άλλα εμφυτεύματα
Μπορείτε να βρείτε προεκπαιδευμένα εμφυτεύματα και από πηγές εκτός του Gensim. Για παράδειγμα:

- [Google News dataset](https://code.google.com/archive/p/word2vec/). Πρόκειται για προ-εκπαιδευμένα διανύσματα που έχουν εκπαιδευτεί σε μέρος του συνόλου δεδομένων Google News (περίπου 100 δισεκατομμύρια λέξεις). Το μοντέλο περιέχει διανύσματα 300 διαστάσεων για 3 εκατομμύρια λέξεις και φράσεις.
- [Amazon BlazingText](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html). Το BlazingText δεν είναι μόνο προεκπαιδευμένα εμφυτεύματα αλλα και βελτιστοποιημένες υλοποιήσεις των αλγορίθμων Word2vec για την επεξεργασία κειμένου. Προυπόθεση είναι να δουλέψει κανείς στο SageMaker.

Οι διαδικασίες φόρτωσης embeddings από εξωτερικά δεδομένα μπορεί να είναι ελαφρά διαφορετικές από αυτή του Gensim.



#### Παρατηρήσεις

*   Επαναλαμβάνουμε ότι στην εργασία αυτή δεν μας ενδιαφέρουν τα ίδια τα μοντέλα αλλά το να μπορούμε για μία λέξη του λεξιλογίου μας να μπορούμε να βρούμε το embedding (διάνυσμα) που της αντιστοιχεί στο εκάστοτε προεκπαιδευμένο μοντέλο. 

*   Επίσης, δεν θα χρησιμοποιήσουμε την `Phrases` για να βρούμε bigrams στο dataset μας όπως θα ήταν το ορθότερο, καθώς αυτό θα απαιτούσε την συνέχιση της εκπαίδευσης του μοντέλου σε νέο λεξιλόγιο με πολύ λίγα νέα δεδομένα.


 ### Δημιουργία corpora βασισμένων στα εμφυτεύματα

Για να μπορέσουμε να ενσωματώσουμε τη γνώση που υπάρχει στα προεκπαιδευμένα εμφυτεύματα στο δικό μας corpus θα προχωρήσουμε όπως περιγράφεται ακολούθως.

Για κάθε περιγραφή ταινίας $d$, η οποία αποτελείται από τις $N_d$ λέξεις $w_i$, το  $tfidf$ της κάθε λέξης $w_i$ δίνεται από τη σχέση:

$$ tfidf(w_i) = tf(w_i,d) \cdot idf(w_i)$$

Ταυτόχρονα, σε κάθε λέξη $w_i$ αντιστοιχεί ένα διάνυσμα $W2V(w_i)$ από το μοντέλο εμφυτευμάτων που έχουμε εισάγει. Τα διανύσματα εμφυτευμάτων $W2V$ θα έχουν διάσταση $m$, ανάλογα το μοντέλο. 

Για κάθε ταινία d, μπορούμε να δημιουργήσουμε μια διανυσματική αναπαράσταση $W2V(d)$ διαστάσεων $m$ χρησιμοποιώντας το $tfidf(w_i)$ ως συντελεστή βαρύτητας για κάθε εμφύτευμα $W2V(w_i)$:

$$ W2V(d) = \frac{tfidf(w_1)\cdot W2V(w_i) + tfidf(w_2)\cdot W2V(w_2) + \dotsc  + tfidf(w_{N_{d}})\cdot W2V(w_{N_{d}})}{tfidf(w_1)+tfidf(w_2)+ \dotsc + tfidf(w_{N_{d}})}$$


#### build_tfw2v

Υλοποιήστε μια συνάρτηση `build_tfw2v` με ορίσματα:
- `corpus` που θα είναι το προεπεξεργασμένο dataset σας,
- `vectors` που θα είναι το μοντέλο που θα σας δίνει τα διανύσματα των εμφυτεύσεων vectors, και 
- `embeddings_size` που θα είναι η διάσταση των εμφυτευμάτων $m$.

H συνάρτηση αυτή θα επιστρέφει ένα νέο corpus που θα είναι ένας πίνακας 5000 (όσες οι ταινίες σας) x $m$ (το η διάσταση των εμφυτευμάτων). Ανάλογα ποιο μοντέλο χρησιμποιείτε για transfer learning ο πίνακας αυτός θα είναι διαφορετικός.

Μπορείτε πλεόν να καλείτε την `content_recommender` με διαφορετικά corpora στο όρισμα `corpus_type`. Σημειώστε ότι στο TFidfVectorizer χρησιμοποιουμε τη σειριακή μορφή των numpy arrays και ίσως σας χρησιμεύσει η `sparse.csr_matrix()` από την Scipy.

In [ ]:
# Downloading a pre-trained word2vec embedder.
vectors = gensim.downloader.load('glove-wiki-gigaword-100')

[=================================================-] 99.8% 127.8/128.1MB downloaded


In [ ]:
# Print model info.
print(vectors)

KeyedVectors<vector_size=100, 400000 keys>


In [ ]:
def build_tfw2v(corpus, vectors, embeddings_size):
  global opt_vectorizer # "Optimized" tf-idf vectorizer.
  opt_vectorizer.fit(corpus)
  corpus_tf_idf = opt_vectorizer.transform(corpus)
  tokenizer = opt_vectorizer.build_tokenizer()
  new_corpus = np.zeros((len(corpus), embeddings_size))
  w2v_vocabulary = set(vectors.index_to_key) # Faster Searches. 
  # Stemming might produce more tokens not "available" in our embeddings. We choose to lemmatize.

  for i, description in enumerate(corpus):
    aux1, aux2 = 0, 0
    for token in tokenizer(description):
      if token in opt_vectorizer.vocabulary_: # Some tokens are excluded, see max_df & min_df.
        token_idx = opt_vectorizer.vocabulary_[token]
        tf = corpus_tf_idf[i, token_idx]
        aux2 += tf
        if token in w2v_vocabulary:
          aux1 += tf * vectors[token]   
    if aux2 > 0:
      new_corpus[i] = aux1 / aux2

  return scipy.sparse.csr_matrix(new_corpus)

In [ ]:
%%time 
the_best_corpus_ever = build_tfw2v(new_corpus, vectors, vectors.vector_size) # It's now or never, we're not gonna live forever...

CPU times: user 22.9 s, sys: 753 ms, total: 23.6 s
Wall time: 22.9 s


In [ ]:
print(the_best_corpus_ever.shape)

(5000, 100)


## Ανάλυση αποτελεσμάτων

### Σύστημα συστάσεων βασισμένο μόνο στο tfidf

- Σε markdown περιγράψτε τι προεπεξεργασία κάνετε στα κείμενα και γιατί.

- Περιγράψτε πως προχωρήσατε στις επιλογές σας για τη βελτιστοποίηση της `TfidfVectorizer`. 

- [Cherry-picking:](https://www.wikiwand.com/en/Cherry_picking) Δώσετε παραδείγματα (IDs) από τη συλλογή σας που επιστρέφουν καλά αποτελέσματα μέχρι `max_recommendations` (τουλάχιστον 5) και σχολιάστε.

- [Nit-picking:](https://en.wikipedia.org/wiki/Nitpicking) Δώστε παραδείγματα (IDs) από τη συλλογή σας που επιστρέφουν κακά αποτελέσματα και σχολιάστε.

- Ποια είναι συνολικά τα πλεονεκτήματα και μειονεκτήματα ενός recommender βασισμένου στο tfidf;

<br>

***Προεπεξεργασία***
1. Αφαιρούμε οτιδήποτε δεν είναι αλφαριθμητικός (αγγλικός) χαρακτήρας, κενό ή σημείο στίξης. (Υπήρχαν πχ. ξενόγλωσσοι χαρακτήρες στις περιγραφές, στους οποίους και δεν βασιζόμαστε για τις προτάσεις που θα κάνουμε στη συνέχεια.)
2. Αρκετές ταινίες περιέχουν links, κάτι το οποίο οδηγεί τον recommender στο να προτείνει άσχετες ταινίες απλώς επειδή περιείχαν links. Τα αφαιρούμε. (Τα αντιμετωπίζουμε ως ένα string το οποίο αρχίζει με το keyword "http" και στη συνέχεια περιέχει και άλλους χαρακτήρες, όχι όμως κενά)
3. Κάνουμε tokenization των φιλτραρισμένων περιγραφών, με χρήση του tokenizer της βιβλιοθήκης nltk. Έτσι, προσωρινά, κάθε ταινία έχει ως "περιγραφή" μία λίστα από tokens.
4. Από κάθε tokenized περιγραφή αφαιρούμε συνηθισμένες λέξεις της αγγλικής (βλ. stopwords της nltk), σημεία στίξης όπως και συνηθισμένα αντρικά και γυναικεία αγγλικά ονόματα. (Αυτό το κάναμε διότι όταν δοκιμάζαμε τον recommender που αναπτύξαμε, πολλές φορές πρότεινε "άσχετες" σημασιολογικά ταινίες, στις οποίες τύχαινε πχ. ο κεντρικός χαρακτήρας να είχε το ίδιο όνομα. Για ομοιομορφία, μετατρέπουμε όλα τα tokens σε πεζούς χαρακτήρες.
5. Το φιλτράρισμα που κάναμε παραπάνω, ίσως να μην αφαιρεί περιπτώσεις που έχουμε πολλά σημεία στίξης στη σειρά. (πχ. μπορεί το string "./#!" να μην έγινε tokenized στα επιμέρους σημεία στίξης) Γι'αυτό χρησιμοποιουμε τη συνάρτηση `thorough_filter`, της οποίας ο κώδικας υπάρχει στο notebook για εισαγωγή στο text mining, του μαθήματος. Τη τροποποιήσαμε ώστε να αφαιρεί tokens που αποτελούνται απόκλειστικά από ψηφία ή/και σημεία στίξης. (Δεν χρησιμοποιούμε σκέτους αριθμούς για τον recommender.) Επίσης, αφαιρούμε tokens με μήκος μικρότερο του 2 διότι υπάρχουν ελάχιστες λέξεις της αγγλικής με 2 γράμματα που μπορούν να δώσουν χρήσιμο σημασιολογικό περιεχόμενο και "διαχωρισιμότητα".
6. Έχουμε μία συλλογή από φιλτραρισμένα tokens. Θέλουμε να δημιουργήσουμε ένα vocabulary από tokens, τα οποία να περιγράφουν οποιαδήποτε ταινία του dataset μας. (Περιγράφουμε ουσιαστικά το bag-of-words.) Θέλουμε να ομαδοποιήσουμε σημασιολογικά ισοδύναμα tokens, πχ το 'been' με το 'be' το 'player' με το 'play' ενδεχομένως κοκ. Για να το κάνουμε αυτό χρησιμοποιούμε το lemmatization της nltk που αντικαθιστά σημασιολογικά ισοδύναμα tokens με έναν αντιπρόσωπο. Αυτό γίνεται με χρήση προϋπάρχοντος vocabulary, το οποίο αποτελεί μέρος της nltk. Επιλέξαμε lemmatization έναντι stemming, ώστε στη συνέχεια που θα χρησιμοποιήσουμε word embeddings, να έχουμε "υπαρκτά" tokens και να μην υπάρχει κίνδυνος να μην εμφανίζονται στα embeddings που θα χρησιμοποιήσουμε, κάτι που παρατηρήσαμε να συμβαίνει πιο συχνά με stemming.
7. Για κάθε ταινία, ενώνουμε τα tokens που προέκυψαν από τη προεπεξεργασία, με κενά. Αυτή είναι η νέα "περιγραφή" κάθε ταινίας που θα τροφοδοτήσουμε στον TfidfVectorizer.

***Βελτιστοποίηση του Vectorizer***

Η πρώτη ιδέα ήταν να επιλέξουμε κάποια ids (συγκεκριμένα τα `{1305, 2312, 24, 177, 188}`) και να επιχειρήσουμε να βελτιστοποίησουμε χωριστά τα `max_df` & `min_df`. Με αρκετό trial & error, καταλήξαμε σε κάποια όρια για τις παραμέτρους αυτές, που αφήσαμε σαν σχόλια παραπάνω. 

Δοκιμάσαμε, έχοντας βρει κάποια όρια για το `max_df` να το βελτιστοποιήσουμε χρησιμοποιώντας ένα υποσύνολο 10 τυχαίων ταινιών και κάνοντας "ευριστικό" binary search, με `max_recommendations = 5` και ανάγνωση των προκύπτωντων recommendations. Καταλήξαμε στη τιμή 0.22. Ενδεχομένως να μην είναι καθολικά βέλτιστη, αλλά έδινε καλές (τουλάχιστον ως προς το είδος) συστάσεις, για αρκετές από τις ταινίες που δοκιμάσαμε. 

Στη συνέχεια επιχειρήσαμε να βελτιστοποιήσουμε το `min_df`. Αρχικά δοκιμάσαμε να το βελτιστοποιήσουμε ως float. Έπειτα από αρκετές δοκιμές (σε στυλ GridSearch) διαπιστώσαμε πως τα αποτελέσματα ήταν αρκετά ασυνεπή. Αυτό διότι με το float `min_df`, δοκιμάζαμε αρκετά μεγάλο range από συχνότητες, ρίχνοντας δραματικά το μέγεθος του corpus μας. Το corpus μας κατά κύριο λόγο αποτελείται από πολλές σπάνιες λέξεις. Οπότε, ήδη συχνότητα `min_df = 1e-3` (ισοδύναμο με `min_df = 5`, αφού το dataset μας αποτελείται από 5000 περιγραφές), ρίχνει το μέγεθος του corpus από ~ 40 χιλιάδες σε ~ 10 χιλιάδες. Καταλήξαμε να κάνουμε GridSearch στο διάστημα `max_df = {0.2, 0.22, .., 0.4}` και `min_df = {1, .., 4}` για τυχαίες δεκάδες ταινιών.

Οι τιμές των παραμέτρων που θεωρούμε τελικώς βέλτιστες είναι οι `max_df = 0.22`, `min_df = 3`.

#### Αξιολόγηση του recommender

Δοκιμάσαμε 50+ ids. Ελέγξαμε τις 5 κορυφαίες προτάσεις που προτείνει ο recommender μας για κάθε id. Δεν έχουμε διατάξει τα ids απαραίτητα ως προς την ποιότητα των προτάσεων.

Για κάθε ID γράφουμε μία σύνοψη των "ειδών" από recommendations που λαμβάνουμε.

***Cherry-picking***

*Good Recommendations* 

- 4203 - Looney Tunes
- 1006 - Merrie Melodies
- 4109 - Sinister house stories
- 4903 - Money adventures in different genres
- 3195 - War and Marines in different genres
- 403 - Medical & nursery related dramas
- 904 - Politics Fiction (Irrelevant 5th recommendation)
- 1108 - Samurai movies & movies where a painter plays a central role
- 1503 - Djinns in different genres, mostly horror
- 1608 - Mostly Vietnam stories (Irrelevant 5th recommendation)
- 2609 - PG Friendly puppy movies
- 3703 - Drama revolving around children (some of the themes are child abuse & child abandonment)




*Okay Recommendations*

- 2508 - Mostly Cinderella & princess stories. Poor 4th and 5th recommendation. (Even though snow white is kind of related.) The third recommendation is a not quite relevant to Cinderella but contains a character with the said nickname.
- 3902 - Not a great top recommendation. The other ones are different views on family "dramas". Last recommendation is too short and almost irrelevant.
- 2308 - A Nightmare on Elm Street 4. The second recommendation is the direct sequel. Some decent horror recommendations all around but also kids movies like Doraemon because the description contains reference to dreams and nightmares (the Freddy Krueger special 😏 )
- 4404 - Recommendations about monster/creature/demon (quite broad) movies. Irrelevant 5th recommendation of a movie with a short description.
- 4608 - A confusive mix of recommendations about extraterrestial threats but also movies that contain the word earth.
- 1300 - Sleepwalking in different genres. Poor preprocessing and filtering, results to some bad recommendations of short description films. 
- 107 - PG Animal (mostly cat & mouse) Movies. Terrible top recommendation, a thriller. Besides that, not bad recommendations.

***Nit-picking***

- 3008 - The recommender manages to capture the essence of some of the movie's keywords, "students", "japanese" etc. but fails to produce meaningful recommendations. Tfidf does not succeed in finding semantic "equivalence" between descriptions.
- 107 - PG Animal (mostly cat & mouse) Movies. Terrible top recommendation, a thriller. Besides that, not bad recommendations.
- 500 - Mother Teresa biography results in some cheesy horror movie recommendations. 
- 3405 - Unfortunate naming of main character ('Hitler') results in recommendations that have nothing to do with the original movie, but Adolf Hitler.
- 3504 - A mess of movies about family. Some refer to conservative or religious families. Some to "families" metaphorically in the sense of a gang. We also get recommended a comedy and a thriller. In general we are recommended many movies with short descriptions that contain the word family. They are not all quite relevant to the original movie.
- 4703, 4803 - A mess of non cohesive recommendations.
- 2907 - A bad movie description results in non-sensical movie recommendations.

<br>

In [ ]:
# Check the TFIDF recommendations of any of the above IDs by changing the id value below.
try_recommender(ID = 403, max_recommendations = 5)

Original Movie:

Title: ['Dying Young']
Genres: ['"Romantic drama",  "Romance Film",  "Drama",  "Medical fiction"']
Description:
Hilary O'Neil  is a pretty, outgoing yet cautious young woman who has had little luck in work or love. After recently parting ways with her boyfriend when she caught him cheating, Hilary finds herself living with her eccentric mother . One day, Hilary answers an ad in a newspaper for a nurse only to find herself being escorted out before the interview starts. Victor Geddes  is a well-educated, rich, and shy 28 year-old. As the film progresses, Victor's health worsens progressively, due to leukemia. Despite his father's protests, Victor hires Hilary to be his live-in caretaker while he undergoes a traumatic course of chemotherapy. Hillary becomes insecure of her ability to care for Victor after her first exposure to the side effects of his chemotherapy treatment. She studies about leukemia and stocks healthier food in the kitchen. He is "finished" with his che

***Γιατί χρησιμοποιούμε tf-idf?***

Η απλούστερη ιδέα για τη Vector Space αναπαράσταση μίας συλλογής κειμένων, είναι η αναπαράσταση τους με ιστόγραμμα εμφάνισης των λέξεων. (Bag of words) Σαφώς ένα απλό ιστόγραμμα (σε συνδυασμό με cosine similarity) δεν είναι πολύ καλή ιδέα για διανύσματα τα οποία δεν έχουν κανονικοποιηθεί. Πολύ πιθανό μία περιγραφή που έχει πολλές λέξεις να βγει "όμοια" με κάποια άλλη που έχει λίγες λέξεις, διότι έτυχε κάποιες από τις "σημαντικές" λέξεις της δεύτερης να βρέθηκαν έστω και μία φορά στη πρώτη. Για να κανονικοποιήσουμε τη πληροφορία μίας λέξης εντός κειμένου, χρησιμοποιούμε τη σχετική της συχνότητα (tf: term frequency), άρα μας ενδιαφέρει άμεσα το πόσο μεγάλη (σε tokens) είναι μία περιγραφή, ώστε να αποδώσουμε καλύτερα την επίδραση ενός token, δηλαδή πόσο "μονοπωλεί" το νόημα της περιγραφής, σε σχέση πάντα με την έκταση της. 

Αυτό είναι μία πρώτη ιδέα, ωστόσο ενδεχομένως να ποινικοποιεί κάποιες σπάνιες λέξεις οι οποίες όμως συνδέεουν άμεσα περιγραφές που σχετίζονται. (Ένα παράδειγμα για διαίσθηση αυτού που λέμε, είναι πχ. η λέξη "bandit" [ληστής]. Δεν είναι ιδιαίτερα συνηθισμένη λέξη, αλλά παραπέμπει σε ταινίες δράσης, οπότε δεν θέλουμε να χάνεται στη διανυσματική αναπαράσταση που προσπαθούμε να κατασκευάσουμε.) Γι΄αυτό, πολλαπλασιάζουμε τη tf "επίδραση" κάθε λέξης με την idf (inverse document frequency). Το idf είναι μέγεθος (κλιμακωμένο λογαριθμικά) που υποδηλώνει πόσο σπάνια είναι μία λέξη στο συνολικό vocabulary που συντίθεται από **όλες** τις περιγραφές που μας ενδιαφέρουν.

Η tf-idf αναπαράσταση είναι αρκετά απλή και "οικονομική" ως προς τη κατασκευή της και κρύβει χρήσιμη πληροφορία για τη στατιστική των λέξεων του corpus με το οποίο ασχολούμαστε.

***Που υστερεί ο tf-idf?***

Ένα βασικό μειονέκτημα αυτής της τεχνικής (στην απλούστερη εκδοχή της, που δεν ασχολείται με n-grams, βλ. την υλοποίηση της sk-learn) είναι πως δεν αξιοποιούμε τη "χωρική" συσχέτιση των λέξεων εντός της περιγραφής ενός κειμένου. Πάνω σε αυτή την ιδέα (και γενικότερα σε αυτό που σε NLP εφαρμογές θα λέγαμε "short-time" analysis, αν και εδώ δεν ασχολούμαστε με την έννοια του χρόνου) χτίζουν τα W2V embeddings. Απομονώνουν κατάλληλες γειτονιές , με στόχο να μάθουν σημασιολογικές και συντακτικές συσχετίσεις μίας λέξεις στόχου, με τις υπόλοιπες που βρίσκονται στην γειτονιά της. Σαφώς, για να αποκτήσουμε καλά W2V embeddings, χρειάζεται πιο "περίπλοκο" infrastructure (νευρωνικά δίκτυα), όπως και πολλά δεδομένα εκπαίδευσης.

<br>

### Σύγκριση και σχολιασμός με recommenders βασισμένων στο Word2Vec

- Υλoποιήστε recommenders που βασίζονται σε μεταφορά μάθησης και εμφυτεύματα. Χρησιμοποιήστε παραδείγματα για να υποδείξετε δυνατά και αδύναμα σημεία τους.

- Μπορείτε να σχολιάσετε τα recommenders που βασίζονται στο Word2Vec σε σχέση με το απλό μοντέλο tfidf, εξετάζοντας τις συστάσεις για ίδια ID.

- Μπορείτε επίσης να εξετάσετε συγκριτικά τα Word2Vec recommenders μεταξύ τους και πάλι βασιζόμενοι σε παραδείγματα.

- Οι παρατηρήσεις σας θα βασίζονται στην ανάλυση των ποιοτικών χαρακτηριστικών που είναι η σειρά και το σύνολο των συστάσεων. Ωστόσο, μπορείτε να συμπεριλάβετε και ποσοτικά χαρακτηριστικά όπως τους χρονους loading και συγκρότησης του corpus αλλά και της διαστατικότητας $m$.

Χρησιμοποιήστε όποια μορφή reporting κρίνετε καταλληλότερη: κείμενο, πίνακες, διαγράμματα.


<br>

#### Αξιολόγηση του Recommender

Χρησιμοποιούμε τα embeddings `glove-wiki-gigaword-100`, τα οποία έχουν μικρό μέγεθος και κατεβαίνουν γρήγορα. Προέρχονται από τη wikipedia, οπότε ελπίζουμε στην ύπαρξη λέξεων "γενικής φύσεως" που να χρησιμοποιούνται στα διαφόρων ειδών descriptions που συναντούμε. Επίσης, χρησιμοποιούμε embeddings με μικρή διάσταση σχετικά, ελπίζοντας να γίνει καλύτερη η προβολή τους στο επίπεδο που θα δημιουργήσει το SOM στη συνέχεια.

Ενδεχομένως να ήταν καλύτερη η χρήση κάποιων άλλων embeddings που ίσως περιέχουν πιο "σύγχρονο λεξιλόγιο". Δεν συσχετίσαμε από ποια χρονική περίοδο προέρχονται οι περιγραφές των ταινιών και αντίστοιχα των embeddings.

***Τυχαίες Δοκιμές***

Επιλέξαμε 10 τυχαία ids που κάναμε hard-code παρακάτω.

In [ ]:
for id in [4714, 1328, 3442, 2404, 1069, 1793, 2449, 1362, 4904, 4631]: # Randomly selected ints.
  try_recommender(ID = id, max_recommendations = 5, movie_corpus = the_best_corpus_ever)
  print("================================\n")

Original Movie:

Title: ['The Last Hurrah']
Genres: ['"Drama"']
Description:
A big-city mayor, Frank Skeffington, runs a powerful political machine as he seeks a fourth term, but his age, health and unhappy adversaries all stand in his way.

Recommended Movies:

1.
Title: ['The American Ruling Class']
Genres: ['"Political cinema",  "Culture & Society",  "Social issues",  "Documentary"']
Description:
The film stars Lewis H. Lapham, who plays himself as editor of Harper's Magazine. Lapham opens the film with the question of whether or not America has a "ruling class," a circle of wealthy and powerful families that run the banks, businesses, and government, essentially controlling everything in America. To answer this question, Lapham devises a fictional scenario following the post-graduation paths of two young Yale graduates, themselves from opposite economic beginnings. The paths they take gradually clue them, and the audience, into the reality posed by the question. Caton Burwell plays

Σε κάποιες περιπτώσεις όπως για τη ταινία "Spider-Plant Man" υπάρχει "σύγχηση" ως προς τις προτάσεις. Η original ταινία είναι παρωδία του Spiderman. Οι προτάσεις αφορούν ταινίες με αράχνες, τέρατα, δράση και τον Superman. Η ταινία που "ξεγέλασε" τον recommender είναι η "Follow the Leader". Αφορά στρατιωτικό περιβάλλον στο οποίο ένας χαρακτήρας έχει το όνομα Spider O'Brien. Το όνομα Spider και το "περιβάλλον δράσης" ενδεχομένως μπέρδεψαν τον recommender. Δεν ήταν εύκολο να διακρίνει τη περίπτωση που υπάρχει χαρακτήρας με το όνομα Spider από τη συνήθη περίπτωση του "Spiderman".

Σε άλλες περιπτώσεις όπως η ταινία 'Bol Radha Bol', που αφορά ένα κοινωνικοικογενειακό ρομαντικό δράμα, ο recommender προτείνει επιτυχημένα αντίστοιχες ταινίες ενώ επιστρέφει κυρίως ταινίες Bollywood (& γενικότερα World Cinema) όπως είναι και η αρχική ταινία εξάλλου.

Για τη ταινία 'Shubhayathra', προτείνονται σε γενικές γραμμές δράματα που διαδραματίζονται στην Ινδία (και πολλές φορές συγκεκριμένα στο Mumbai, όπως και η original ταινία). Δεν ταυτίζονται εντελώς σε πλοκές, αλλά αυτό δεν είναι πάντα κακό στις προτάσεις ταινιών.

Στη ταινία 'Kuberan', η κατάσταση είναι περίπου ίδια με τη ταινία 'Shubhayathra'. Γενικά προτείνονται ταινίες όπου οι χαρακτήρες έχουν Ινδικά ονόματα. Οι πρώτες προτάσεις περιέχουν χαρακτήρα με το όνομα Pooja όπως και η original, κάτι που θα μας έκανε να πιστεύουμε πως μόνο αυτό το γεγονός οδήγησε στις προτάσεις αυτές. Οι υπόλοιπες προτάσεις δεν περιέχουν κάτι τέτοιο. Διαισθητικά φαίνεται να έγιναν γενικώς προτάσεις για Ινδικές ταινίες και ενδεχομένως να πριμοδοτήθηκαν αυτές με συνωνυμία στους χαρακτήρες τους.

Στη ταινία 'Air Cadet' η πρώτη πρόταση είναι αρκετά σχετική και η δεύτερη λιγότερο αλλά τουλάχιστον ικανοποιητική. Από τη τρίτη και μετά ο recommender ασχολείται περισσότερο με στρατιωτικό setting στο οποίο πραγματοποιούνται κάποιου είδους "πτήσεις" (είτε αεροπορικές είτε διαστημικές) χωρίς μεγάλη σημασιολογική επιτυχία. Το κοινό "στοιχείο" όλων αυτών των ταινιών δεν είναι αρκετό για να δικαιολογήσει το πόσο ασυσχέτιστες είναι οι περισσότερες προτάσεις.

Στη ταινία 'The Crow: Salvation', τόσο η original ταινία όσο και αρκετές από τις ταινίες που προτάθηκαν έχουν αρκετά μακροσκελής περιγραφές. Για άλλη μία φορά τα σενάρια δεν ταυτίζονται, αλλά είναι όλες ταινίες τρόμου ή θρίλερς με cult 70s-80s αισθητική, οι οποίες σε γενικές γραμμές είναι σε θέση να ικανοποιήσουν τον χρήστη του recommender.

Για τη ταινία 'Costa!' προτείνονται κυρίως ρομαντικές και δραματικές ταινίες που αφορούν χορό. Η 4η ταινία που προτείνεται μπορεί να μην σχετίζεται ξεκάθαρα με χορό αλλά αποτελείται από μικρή περιγραφή και εμφανίζεται τόσο στοιχείο χορού (dance) όσο και "ισπανικού" είδους background/setting. Αυτού του είδους setting χαρακτηρίζει την αρχική ταινία που διαδραματίζεται στην Ισπανία, όπως και τη πρώτη πρόταση που διαδραματίζεται στη Κούβα.

Η ταινία 'The Last Hurrah' αποτελείται από μικρή περιγραφή με βασικά keywords το mayor (που υπονοεεί μία θέση ισχύος, κάποιου είδους εξουσία), political, term, adversaries. Είναι αρκετά μικρή και γενικόλογη περιγραφή. Οι περισσότερες προτάσεις δεν σχετίζονται άμεσα με τη ταινία, καθώς όπως προείπαμε δεν είναι ιδιαίτερα "αναλυτική" η περιγραφή. Προτείνονται ταινίες που αφορούν θέσεις ισχύος (κυρίως σε οικονομικά πόστα, αλλά και μία πολιτική κωμωδία, βλ. δεύτερο suggestion) όπως και κοινωνικοπολιτικά ζητήματα. Δεν αποτελούν τις "καλύτερες" προτάσεις που έχουμε δει, αλλά σίγουρα έχουν εμφανή συσχέτιση με την αρχική.

Αδυνατούμε να ερμηνεύσουμε τις συστάσεις που επιστρέφει ο recommender μας για τη ταινία 'The Big Pond'. Αναγνωρίζουμε τουλάχιστον πως έχουν συνάφεια ως προς το είδος.

Στη ταινία 'A Town Called Panic' οι κεντρικοί χαρακτήρες είναι πλαστικά παιχνίδια εμπνευσμένα από western setting, ένα άλογο, ένας καουμπόης και ένας ινδιάνος. Αυτό σε συνδυασμό με τη σχετικά μικρή περιγραφή είναι αρκετό για να "ξεγελάσει" τον recommender μας. Οι δύο πρώτες προτάσεις είναι PG Friendly ταινίες που αφορούν είτε western setting (βλ. πρώτη) ή τις περιπέτειες τριών φίλων στο Arkansas. Οι υπόλοιπες ταινίες που προτείνονται έχουν αρκετά μακροσκελή περιγραφή αλλά όχι άμεση συσχέτιση με την original. Ειδικά το τελευταίο recommendation είναι ιδιαιτέρως αποτυχημένο καθώς από μία ταινία που μπορούν να δουν παιδιά, προτείνεται ταινία τρόμου.

***W2V recommendations on tf-idf tested IDs***

Παρακάτω έχουμε συνοψίσει τη ποιότητα των προτάσεων του W2V στα περισσότερα από τα IDs που εξετάσαμε προηγουμένως για τον tf-idf. 

- 4203 - Pretty much Looney Tunes recommendations once again, mostly reordered. Some recommendations are "new" but also revolve around Bugs Bunny adventures.
- 1006 - Some of the top recommendations are reordered. The embeddings recommender also recommends a movie that concerns Tweetie and not Porky, but this is also a decent recommendation.
- 107 - Only PG animal animations. Got rid of "Maneater" as a top 
recommendation.
- 4903 - More cohesive genre-wise recommendations on the "money & debt" movie premise.
- 904 - More relevant top recommendations (docudramas about communist parties). In general all recommendations revolve around politics more than the tfidf recommendations.
- 2609 - Mother centric movies (Motherless children, abusive mothers, single mothers etc.) Quite the shift from "puppy movies".
- 3405 - Still suffers from the naming of the main character.
- 4109 - Not quite the "sinister house" thematic we noticed in tfidf. Quite the lesser good recommendations.
- 403 - Some dramas revolving around life threatening conditions. 
- 1108 - Terrible top recommendation. The original movie is a horror movie about vengeance where the main character's a painter, the top recommendation's a documentary about the origins of creativity. Most of the other recommendations suffer of the same. The 3rd recommendation is the most relevant out of the top 5 ones.
- 1503 - From djinn recommendations we turn into paranormal horror recommendations. Not quite cohesive, but maybe people into the genre would explore such movies. The 2nd recommendation is quite irrelevant.
- 1608 - Lesser relevant recommendations besides the first three maybe.
- 2508 - Some good romance/fairy-tale like music. The 3rd recommendation is lacking, unless Cinderella fans are also Dragonball fans.
- 2308 - The sequel of the elm street is now the top recommendation, with no Doraemon to be found. More adult-oriented movies are recomended but also some not quite relevant comedies.
- 2907 - Non sensical description contains reference to the songs contained in the movie. This results in music and song oriented recommendations. These recommendations are more explainable than the ones we received with tfidf, but still not quite successful.

In [ ]:
# Check the W2V recommendations of any of the above IDs by changing the id value below.
try_recommender(ID = 2907, max_recommendations = 5, movie_corpus = the_best_corpus_ever)

Original Movie:

Title: ['Yummy Yummy']
Genres: ['"Short Film"']
Description:
The Wiggles introduce themselves and Paul the Cook. Paul has some hot potatoes, cold spaghetti, and mashed bananas. * Song 1&nbsp;– Hot Potato * Song 2&nbsp;– D.O.R.O.T.H.Y.  Anthony introduces the song about an old steam train called a pufferbelly, and asks everyone to follow along. * Song 3&nbsp;– Pufferbellies Captain Feathersword dances to the dance that Henry showed him and the Wiggles were woundering what he was doing. When Captain Feathersword explained about the dance they all tried it together and they think it's not much luck, so he tells the Wiggles to go and see Henry to do the dance. * Song 4&nbsp;– Henry's Dance Greg comments on how there are many places you can walk or run to. Dorothy is walking and eating some roses. Captain Feathersword is running to his pirate ship. Henry is walking to the ocean to have a swim. Wags is running to dig up some bones. Jeff is sleepwalking. Jeff leaves off scene

Δεν είναι εύκολη η άμεση σύγκριση των ταξινομητών W2V & tf-idf. Εξαρτάται πάντα από τα γούστα του θεατή. Σε κάποιες ταινίες έχει νόημα να δίνουμε μεγάλη προσοχή σε keywords. (βλ. Bugs Bunny) Αυτό μπορεί να αρκεί για να έχουμε καλές προτάσεις. Σε άλλες ταινίες έχει σημασία και το "context" (πχ. A Nightmare on the Elm Street & Doraemon δεν σχετίζονται ακόμη κι αν και οι δύο ταινίες αφορούν "όνειρα"). Εκεί έχει νόημα να χρησιμοποιήσουμε τα embeddings. Ωστόσο, έχει σημασία να επιλέξουμε "σωστά embeddings". Με τον τρόπο που συνδυάζουμε τα embeddings και τη tfidf αναπαράσταση, υπάρχει κίνδυνος tokens που να καθιστούν ταινίες όμοιες, να χάνονται στη W2V αναπαράσταση. Αυτό διότι ίσως δεν περιέχονται στο λεξιλόγιο των embeddings ή χειρότερα, έχουμε κάνει τέτοιο preprocessing (πχ. stemming) στο οποίο tokens να υπάρχουν μεταξύ ταινιών (πχ. movie -> movi), όμως να μην περιέχονται στα embeddings, άρα πλέον να μη προσφέρουν τη πληροφορία που παρείχαν προηγουμένως. 

***Συνοπτική σύγκριση διαφορετικών embeddings***

Παρακάτω, συγκρίνουμε για κάποια τυχαία IDs, προτάσεις που δίνουν recommenders που χρησιμοποιούν διαφορετικά embeddings. Συγκεκριμένα χρησιμοποιούμε τα εξής embeddings. Το `glove-wiki-gigaword-100`, το οποίο χρησιμοποιούμε συστηματικά σε όλη τη παρούσα εργασία, το `glove-wiki-gigaword-50`, ώστε να συγκρίνουμε την επίδραση του να χρησιμοποιούμε embeddings μικρότερης διάστασης που προέρχονται από το ίδιο dataset και τέλος το `glove-twitter-100`, ώστε να συγκρίνουμε το `glove-wiki-gigaword-100` με embeddings ιδίας διάστασης, από διαφορετικό dataset.

In [ ]:
smaller_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

[=================================================-] 99.4% 65.6/66.0MB downloaded


In [ ]:
%%time
smaller_corpus = build_tfw2v(new_corpus, smaller_vectors, smaller_vectors.vector_size)

CPU times: user 23.1 s, sys: 844 ms, total: 23.9 s
Wall time: 23.1 s


In [ ]:
twitter_vectors = gensim.downloader.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [ ]:
%%time
twitter_corpus = build_tfw2v(new_corpus, twitter_vectors, twitter_vectors.vector_size)

CPU times: user 23.9 s, sys: 864 ms, total: 24.8 s
Wall time: 24 s


<br>

In [ ]:
try_recommender(ID = 2508, max_recommendations = 5, movie_corpus = the_best_corpus_ever)

Original Movie:

Title: ['Cinderella']
Genres: ['"Romance Film",  "Fantasy",  "Family Film",  "Musical",  "Animation"']
Description:
Cinderella is the much-loved only child of a widowed aristocrat. After deciding that his beloved daughter needs a mother's care, Cinderella's father marries Lady Tremaine, a proud woman with two daughters from her first marriage, Drizella and Anastasia. Plain and socially awkward, these stepsisters are bitterly envious of the beautiful and charming Cinderella. After the death of Cinderella's father, Lady Tremaine and her daughters take over the estate, and begin to abuse and mistreat Cinderella out of jealousy, and even allow their cat, Lucifer, to torment her. Despite being forced into servitude in her own home, Cinderella becomes a kind woman and befriends the animals living in the barn and many of the mice and birds who live in and around the chateau. At the royal palace, the King is distressed that his son does not intend to marry. Determined to see g

In [ ]:
try_recommender(ID = 2508, max_recommendations = 5, movie_corpus = smaller_corpus)

Original Movie:

Title: ['Cinderella']
Genres: ['"Romance Film",  "Fantasy",  "Family Film",  "Musical",  "Animation"']
Description:
Cinderella is the much-loved only child of a widowed aristocrat. After deciding that his beloved daughter needs a mother's care, Cinderella's father marries Lady Tremaine, a proud woman with two daughters from her first marriage, Drizella and Anastasia. Plain and socially awkward, these stepsisters are bitterly envious of the beautiful and charming Cinderella. After the death of Cinderella's father, Lady Tremaine and her daughters take over the estate, and begin to abuse and mistreat Cinderella out of jealousy, and even allow their cat, Lucifer, to torment her. Despite being forced into servitude in her own home, Cinderella becomes a kind woman and befriends the animals living in the barn and many of the mice and birds who live in and around the chateau. At the royal palace, the King is distressed that his son does not intend to marry. Determined to see g

In [ ]:
try_recommender(ID = 2508, max_recommendations = 5, movie_corpus = twitter_corpus)

Original Movie:

Title: ['Cinderella']
Genres: ['"Romance Film",  "Fantasy",  "Family Film",  "Musical",  "Animation"']
Description:
Cinderella is the much-loved only child of a widowed aristocrat. After deciding that his beloved daughter needs a mother's care, Cinderella's father marries Lady Tremaine, a proud woman with two daughters from her first marriage, Drizella and Anastasia. Plain and socially awkward, these stepsisters are bitterly envious of the beautiful and charming Cinderella. After the death of Cinderella's father, Lady Tremaine and her daughters take over the estate, and begin to abuse and mistreat Cinderella out of jealousy, and even allow their cat, Lucifer, to torment her. Despite being forced into servitude in her own home, Cinderella becomes a kind woman and befriends the animals living in the barn and many of the mice and birds who live in and around the chateau. At the royal palace, the King is distressed that his son does not intend to marry. Determined to see g

<br>

In [ ]:
try_recommender(ID = 103, max_recommendations = 5, movie_corpus = the_best_corpus_ever)

Original Movie:

Title: ['The Food of the Gods']
Genres: ['"Natural horror films",  "Science Fiction",  "Horror",  "Film adaptation",  "Creature Film",  "Cult",  "Drama",  "Monster",  "B-movie"']
Description:
The film reduced the tale to an 'Ecology Strikes Back' scenario, common in science fiction movies at the time. The food mysteriously bubbles up from the ground on a remote island somewhere in British Columbia. The couple that discover it, Mr. and Mrs. Skinner  consider it a gift from God, and promptly begin feeding it to their chickens. Soon, rats, wasps, and worms consume the substance, and the island is crawling with giant vermin. One night, some of them kill Mr. Skinner. Morgan, a professional football player , and his buddies are camping on the island, and one of them is stung to death by giant wasps. After ferrying his friends back to the mainland, Morgan returns to investigate. Also thrown into the mix are Thomas and Rita, an expecting couple; Jack Bensington, the owner of a

In [ ]:
try_recommender(ID = 103, max_recommendations = 5, movie_corpus = smaller_corpus)

Original Movie:

Title: ['The Food of the Gods']
Genres: ['"Natural horror films",  "Science Fiction",  "Horror",  "Film adaptation",  "Creature Film",  "Cult",  "Drama",  "Monster",  "B-movie"']
Description:
The film reduced the tale to an 'Ecology Strikes Back' scenario, common in science fiction movies at the time. The food mysteriously bubbles up from the ground on a remote island somewhere in British Columbia. The couple that discover it, Mr. and Mrs. Skinner  consider it a gift from God, and promptly begin feeding it to their chickens. Soon, rats, wasps, and worms consume the substance, and the island is crawling with giant vermin. One night, some of them kill Mr. Skinner. Morgan, a professional football player , and his buddies are camping on the island, and one of them is stung to death by giant wasps. After ferrying his friends back to the mainland, Morgan returns to investigate. Also thrown into the mix are Thomas and Rita, an expecting couple; Jack Bensington, the owner of a

In [ ]:
try_recommender(ID = 103, max_recommendations = 5, movie_corpus = twitter_corpus)

Original Movie:

Title: ['The Food of the Gods']
Genres: ['"Natural horror films",  "Science Fiction",  "Horror",  "Film adaptation",  "Creature Film",  "Cult",  "Drama",  "Monster",  "B-movie"']
Description:
The film reduced the tale to an 'Ecology Strikes Back' scenario, common in science fiction movies at the time. The food mysteriously bubbles up from the ground on a remote island somewhere in British Columbia. The couple that discover it, Mr. and Mrs. Skinner  consider it a gift from God, and promptly begin feeding it to their chickens. Soon, rats, wasps, and worms consume the substance, and the island is crawling with giant vermin. One night, some of them kill Mr. Skinner. Morgan, a professional football player , and his buddies are camping on the island, and one of them is stung to death by giant wasps. After ferrying his friends back to the mainland, Morgan returns to investigate. Also thrown into the mix are Thomas and Rita, an expecting couple; Jack Bensington, the owner of a

<br>

In [ ]:
try_recommender(ID = 4967, max_recommendations = 5, movie_corpus = the_best_corpus_ever)

Original Movie:

Title: ['The Wonderful Country']
Genres: ['"Western"']
Description:
Mercenary Martin Brady is employed by the Castro brothers, Marcos , a General and Cipriano  a Governor. On a business trip to the United States to arrange the sale of a wagonload of rifles and ammunition he is delayed when he breaks his leg in an accident. Treated by Dr Stovall  he stays with two German Americans, Ben  and Ludwig  to recover. Brady's expertise is sought by the commanders of the local U.S. Army, Major Colton  and Texas Rangers Captain Rucker  who are planning a mission against hostile Apaches in Mexico. Captain Rucker knows that Brady revenged himself on the murderer of his father but no charges are standing. However his falling in love with Colton's wife Helen , and also his shooting a man who drew on him  sends him back to Mexico. Brady finds himself between the ambitious Castro brothers and assisting the U.S. Army's Buffalo Soldiers led by a wounded Major Colton and his sergeant .

R

In [ ]:
try_recommender(ID = 4967, max_recommendations = 5, movie_corpus = smaller_corpus)

Original Movie:

Title: ['The Wonderful Country']
Genres: ['"Western"']
Description:
Mercenary Martin Brady is employed by the Castro brothers, Marcos , a General and Cipriano  a Governor. On a business trip to the United States to arrange the sale of a wagonload of rifles and ammunition he is delayed when he breaks his leg in an accident. Treated by Dr Stovall  he stays with two German Americans, Ben  and Ludwig  to recover. Brady's expertise is sought by the commanders of the local U.S. Army, Major Colton  and Texas Rangers Captain Rucker  who are planning a mission against hostile Apaches in Mexico. Captain Rucker knows that Brady revenged himself on the murderer of his father but no charges are standing. However his falling in love with Colton's wife Helen , and also his shooting a man who drew on him  sends him back to Mexico. Brady finds himself between the ambitious Castro brothers and assisting the U.S. Army's Buffalo Soldiers led by a wounded Major Colton and his sergeant .

R

In [ ]:
try_recommender(ID = 4967, max_recommendations = 5, movie_corpus = twitter_corpus)

Original Movie:

Title: ['The Wonderful Country']
Genres: ['"Western"']
Description:
Mercenary Martin Brady is employed by the Castro brothers, Marcos , a General and Cipriano  a Governor. On a business trip to the United States to arrange the sale of a wagonload of rifles and ammunition he is delayed when he breaks his leg in an accident. Treated by Dr Stovall  he stays with two German Americans, Ben  and Ludwig  to recover. Brady's expertise is sought by the commanders of the local U.S. Army, Major Colton  and Texas Rangers Captain Rucker  who are planning a mission against hostile Apaches in Mexico. Captain Rucker knows that Brady revenged himself on the murderer of his father but no charges are standing. However his falling in love with Colton's wife Helen , and also his shooting a man who drew on him  sends him back to Mexico. Brady finds himself between the ambitious Castro brothers and assisting the U.S. Army's Buffalo Soldiers led by a wounded Major Colton and his sergeant .

R

<br>

In [ ]:
try_recommender(ID = 337, max_recommendations = 5, movie_corpus = the_best_corpus_ever)

Original Movie:

Title: ['The Two of Us']
Genres: ['"World cinema",  "Drama",  "Art film",  "Black-and-white",  "Comedy"']
Description:
Claude  is an 8-year-old Jewish boy living in France during the Nazi occupation. To reduce the chance that he would be sent to Auschwitz or a similar fate, his parents send him to live with a farm family, the elderly parents of Catholic friends of his parents.  The elderly couple honestly think that the boy has been sent to live with them because Paris is dangerous; it never crosses their mind that Claude is a Jew. Claude is given a new last name , is taught a few things about Catholic ritual, such as the Lord's Prayer, and most important, is told to never let anyone see his circumcised penis ; thus Claude's strange prudishness at bath time. Otherwise, he plays well the part of boy grateful to be safe in the countryside, building a warm relationship with Pepe  and Meme , his simple and likeable surrogate grandparents. They form a strong and mutually af

In [ ]:
try_recommender(ID = 337, max_recommendations = 5, movie_corpus = smaller_corpus)

Original Movie:

Title: ['The Two of Us']
Genres: ['"World cinema",  "Drama",  "Art film",  "Black-and-white",  "Comedy"']
Description:
Claude  is an 8-year-old Jewish boy living in France during the Nazi occupation. To reduce the chance that he would be sent to Auschwitz or a similar fate, his parents send him to live with a farm family, the elderly parents of Catholic friends of his parents.  The elderly couple honestly think that the boy has been sent to live with them because Paris is dangerous; it never crosses their mind that Claude is a Jew. Claude is given a new last name , is taught a few things about Catholic ritual, such as the Lord's Prayer, and most important, is told to never let anyone see his circumcised penis ; thus Claude's strange prudishness at bath time. Otherwise, he plays well the part of boy grateful to be safe in the countryside, building a warm relationship with Pepe  and Meme , his simple and likeable surrogate grandparents. They form a strong and mutually af

In [ ]:
try_recommender(ID = 337, max_recommendations = 5, movie_corpus = twitter_corpus)

Original Movie:

Title: ['The Two of Us']
Genres: ['"World cinema",  "Drama",  "Art film",  "Black-and-white",  "Comedy"']
Description:
Claude  is an 8-year-old Jewish boy living in France during the Nazi occupation. To reduce the chance that he would be sent to Auschwitz or a similar fate, his parents send him to live with a farm family, the elderly parents of Catholic friends of his parents.  The elderly couple honestly think that the boy has been sent to live with them because Paris is dangerous; it never crosses their mind that Claude is a Jew. Claude is given a new last name , is taught a few things about Catholic ritual, such as the Lord's Prayer, and most important, is told to never let anyone see his circumcised penis ; thus Claude's strange prudishness at bath time. Otherwise, he plays well the part of boy grateful to be safe in the countryside, building a warm relationship with Pepe  and Meme , his simple and likeable surrogate grandparents. They form a strong and mutually af

<br>

In [ ]:
try_recommender(ID = 462, max_recommendations = 5, movie_corpus = the_best_corpus_ever)

Original Movie:

Title: ['Three Strangers']
Genres: ['"Crime Fiction",  "Drama",  "Film noir",  "Black-and-white",  "Crime Drama"']
Description:
Crystal Shackelford  lures two strangers, solicitor Jerome K. Arbutny  and charming and erudite drunkard Johnny West  to her London flat on Chinese New Year in 1938 because of her belief that if three strangers make the same wish to an idol of Kwan Yin, Chinese goddess of fortune and destiny, the wish will be granted. Since money will make their dreams come true, the three go in on a sweepstakes ticket for the Grand National horse race together and agree that they will not sell the ticket if it is chosen, but will hold onto it until the race is run. Shackelford would use the money to try to win her estranged husband back, Arbutny to smooth the way for his selection to the prestigious Barrister's Club, and Johnny to buy a bar and live in it. The stories of the three strangers are revealed. Shackelford's husband David  moved to Canada and fell i

In [ ]:
try_recommender(ID = 462, max_recommendations = 5, movie_corpus = smaller_corpus)

Original Movie:

Title: ['Three Strangers']
Genres: ['"Crime Fiction",  "Drama",  "Film noir",  "Black-and-white",  "Crime Drama"']
Description:
Crystal Shackelford  lures two strangers, solicitor Jerome K. Arbutny  and charming and erudite drunkard Johnny West  to her London flat on Chinese New Year in 1938 because of her belief that if three strangers make the same wish to an idol of Kwan Yin, Chinese goddess of fortune and destiny, the wish will be granted. Since money will make their dreams come true, the three go in on a sweepstakes ticket for the Grand National horse race together and agree that they will not sell the ticket if it is chosen, but will hold onto it until the race is run. Shackelford would use the money to try to win her estranged husband back, Arbutny to smooth the way for his selection to the prestigious Barrister's Club, and Johnny to buy a bar and live in it. The stories of the three strangers are revealed. Shackelford's husband David  moved to Canada and fell i

In [ ]:
try_recommender(ID = 462, max_recommendations = 5, movie_corpus = twitter_corpus)

Original Movie:

Title: ['Three Strangers']
Genres: ['"Crime Fiction",  "Drama",  "Film noir",  "Black-and-white",  "Crime Drama"']
Description:
Crystal Shackelford  lures two strangers, solicitor Jerome K. Arbutny  and charming and erudite drunkard Johnny West  to her London flat on Chinese New Year in 1938 because of her belief that if three strangers make the same wish to an idol of Kwan Yin, Chinese goddess of fortune and destiny, the wish will be granted. Since money will make their dreams come true, the three go in on a sweepstakes ticket for the Grand National horse race together and agree that they will not sell the ticket if it is chosen, but will hold onto it until the race is run. Shackelford would use the money to try to win her estranged husband back, Arbutny to smooth the way for his selection to the prestigious Barrister's Club, and Johnny to buy a bar and live in it. The stories of the three strangers are revealed. Shackelford's husband David  moved to Canada and fell i

<br>

In [ ]:
try_recommender(ID = 1426, max_recommendations = 5, movie_corpus = the_best_corpus_ever)

Original Movie:

Title: ['Hands Up!']
Genres: ['"Comedy",  "War film"']
Description:
The film tells the story of Jack, a spy for the Confederacy during the Civil War, and his efforts to capture a Union shipment of gold. Obstacles along the way include a pair of sisters, hostile Indians, and a firing squad. The film features fictional incidents involving actual historical figures such as Abraham Lincoln, Brigham Young, and Sitting Bull.

Recommended Movies:

1.
Title: ['First Squad']
Genres: ['"Science Fiction",  "Japanese Movies",  "World cinema",  "History",  "Animation",  "Anime",  "Fantasy",  "War film",  "Drama"']
Description:
Set during the opening days of World War II on the Eastern Front . Its main cast are a group of Soviet teenagers with extraordinary abilities; the teenagers have been drafted to form a special unit to fight the invading German army. They are opposed by a Schutzstaffel  officer who is attempting to raise from the dead a supernatural army of crusaders from the 

In [ ]:
try_recommender(ID = 1426, max_recommendations = 5, movie_corpus = smaller_corpus)

Original Movie:

Title: ['Hands Up!']
Genres: ['"Comedy",  "War film"']
Description:
The film tells the story of Jack, a spy for the Confederacy during the Civil War, and his efforts to capture a Union shipment of gold. Obstacles along the way include a pair of sisters, hostile Indians, and a firing squad. The film features fictional incidents involving actual historical figures such as Abraham Lincoln, Brigham Young, and Sitting Bull.

Recommended Movies:

1.
Title: ['One Spy Too Many']
Genres: ['"Action",  "Adventure",  "Spy"']
Description:
The film opens with Alexander  stealing a chemical weapon from a military base. The weapon causes an enemy’s troops to lose the will to fight, thereby making conquest in battle far easier. This is part of Alexander’s dual goal: to conquer the world in the manner of Alexander the Great and to break each of the major moral codes in so doing . U.N.C.L.E. becomes involved after the theft of the weapon and agents Napoleon Solo  and Illya Kuryakin  are 

In [ ]:
try_recommender(ID = 1426, max_recommendations = 5, movie_corpus = twitter_corpus)

Original Movie:

Title: ['Hands Up!']
Genres: ['"Comedy",  "War film"']
Description:
The film tells the story of Jack, a spy for the Confederacy during the Civil War, and his efforts to capture a Union shipment of gold. Obstacles along the way include a pair of sisters, hostile Indians, and a firing squad. The film features fictional incidents involving actual historical figures such as Abraham Lincoln, Brigham Young, and Sitting Bull.

Recommended Movies:

1.
Title: ['Enigma']
Genres: ['"Thriller",  "Film adaptation",  "Mystery",  "Period piece",  "Drama",  "War film",  "Romance Film",  "Spy"']
Description:
The story, loosely based on actual events, takes place in March 1943, when World War II was at its height. The cryptanalysts at Bletchley Park, Buckinghamshire, have a problem: the Nazi U-boats have changed one of their code reference books used for Enigma machine ciphers, leading to a blackout in the flow of vital naval signals intelligence. The British cryptanalysts have cracked 

<br>

In [ ]:
try_recommender(ID = 1488, max_recommendations = 5, movie_corpus = the_best_corpus_ever)

Original Movie:

Title: ['The Invisible Man']
Genres: ['"Thriller",  "Science Fiction",  "Horror",  "Black-and-white",  "Action/Adventure",  "Action"']
Description:
A mysterious stranger, his face swathed in bandages and his eyes obscured by dark goggles, takes a room at an inn in the English village of Iping. The man demands that he be left alone. Later, the innkeeper, Mr. Hall  is sent by his wife ([[Una O'Connor  to evict him after he makes a huge mess in his room while doing research and is behind on his rent. Angered, he throws Mr. Hall down the stairs. Confronted by a policeman and some local villagers, he removes his bandages and goggles, revealing that he is invisible. Laughing maniacally, he takes off his clothes, making himself completely undetectable, and drives off his tormenters before fleeing into the countryside. The stranger is Dr. Jack Griffin , a chemist who has discovered the secret of invisibility while conducting a series of tests involving an obscure drug called "

In [ ]:
try_recommender(ID = 1488, max_recommendations = 5, movie_corpus = twitter_corpus)

Original Movie:

Title: ['The Invisible Man']
Genres: ['"Thriller",  "Science Fiction",  "Horror",  "Black-and-white",  "Action/Adventure",  "Action"']
Description:
A mysterious stranger, his face swathed in bandages and his eyes obscured by dark goggles, takes a room at an inn in the English village of Iping. The man demands that he be left alone. Later, the innkeeper, Mr. Hall  is sent by his wife ([[Una O'Connor  to evict him after he makes a huge mess in his room while doing research and is behind on his rent. Angered, he throws Mr. Hall down the stairs. Confronted by a policeman and some local villagers, he removes his bandages and goggles, revealing that he is invisible. Laughing maniacally, he takes off his clothes, making himself completely undetectable, and drives off his tormenters before fleeing into the countryside. The stranger is Dr. Jack Griffin , a chemist who has discovered the secret of invisibility while conducting a series of tests involving an obscure drug called "

In [ ]:
try_recommender(ID = 1491, max_recommendations = 5, movie_corpus = the_best_corpus_ever)

Original Movie:

Title: ['On the Inside']
Genres: ['"Thriller",  "Drama"']
Description:
Allen Meneric  is sent to a hospital for the criminally insane to be evaluated after he brutally murders his girlfriend’s alleged rapist. He very quickly becomes surrounded by evil psychopaths, people with seemingly mild personality disorders, sarcastic guards and indifferent doctors. In a courtyard Allen has a chance meeting with Ben . Ben's outgoing pleasant nature helps to initiate a friendship with Allen who is more inclined to keep to himself. Ben's state of mind, however, fluctuates and leads to endless droning, short-term memory loss, and occasional catatonic episodes as the film progresses. Ben introduces Allen to Carl Tarses  who strikes fear into everyone he meets. He has deeply rooted emotional problems from parental abuse in his childhood. Terrorizing people gives Carl a sinister form of satisfaction. Things erupt between Carl and Allen after Carl describes some of his prior heinous acts

In [ ]:
try_recommender(ID = 1491, max_recommendations = 5, movie_corpus = twitter_corpus)

Original Movie:

Title: ['On the Inside']
Genres: ['"Thriller",  "Drama"']
Description:
Allen Meneric  is sent to a hospital for the criminally insane to be evaluated after he brutally murders his girlfriend’s alleged rapist. He very quickly becomes surrounded by evil psychopaths, people with seemingly mild personality disorders, sarcastic guards and indifferent doctors. In a courtyard Allen has a chance meeting with Ben . Ben's outgoing pleasant nature helps to initiate a friendship with Allen who is more inclined to keep to himself. Ben's state of mind, however, fluctuates and leads to endless droning, short-term memory loss, and occasional catatonic episodes as the film progresses. Ben introduces Allen to Carl Tarses  who strikes fear into everyone he meets. He has deeply rooted emotional problems from parental abuse in his childhood. Terrorizing people gives Carl a sinister form of satisfaction. Things erupt between Carl and Allen after Carl describes some of his prior heinous acts

<br>

- 2508 - Slight edge του `wiki-100` έναντι του `twitter` διότι το δεύτερο προτείνει και ταινία τρόμου. Εξ'ίσου "κακές" είναι προτάσεις εκτός των 2 κορυφαίων. Σίγουρα καλύτερο το `wiki-100` έναντι του `wiki-50`.
- 103 - Genre-wise (και plot-wise κατά συνέπεια, στο συγκεκριμένο ID) τόσο το `wiki-50` όσο και το `twitter-100` αποδίδουν καλύτερα από το `wiki-100`. Το `wiki-100` προτείνει μία μόνο Natural Horror ταινία και μάλιστα 5η.
- 4967 - Το `twitter-100` επιλέγει περισσότερα σχετικά westerns σε αντίθεση με το `wiki-100`. Καλύτερο σε γενικές γραμμές το `wiki-100` από το `wiki-50` ως προς το ordering των προτάσεων.
- 337 - Φαίνεται όλα τα embeddings να αποτυγχάνουν το ίδιο. Το `wiki-100` εντοπίζει περισσότερες ταινίες που αφορούν τον 2ο ΠΠ χωρίς να σημαίνει πως είναι σχετικές ως προς θεματολογία με την αρχική και πολύ πιο επιτυχημένες προτάσεις από αυτές των άλλων embedings.
- 1426 - Αρκετά κακές προτάσεις all-around. Ίσως να προτιμούσαμε το `wiki-100` έναντι του `twitter-100`. Το πρώτο προτείνει γενικώς ταινίες για εφηβικό και ενήλικο κοινό. Το δεύτερο προτείνει και Pokemon που δεν φαίνεται να σχετίζεται ιδιαίτερα με την αρχική ταινία ή το target audience της.
- 462 - Το genre-ordering είναι καλύτερο στο `wiki-100`, το `twitter-100` προτείνει περισσότερες σχετικές ταινίες genre-wise αλλά σε "χαμηλότερες" θέσεις. 
- 1491 - Τα `twitter-100` & `wiki-100` προτείνουν δράματα κυρίως. Η κορυφαία πρόταση του `wiki-100` όσον αφορά τις "ψυχωτικές τάσεις" του κύριου πρωταγωνιστή μοιάζει καλύτερη από την αντίστοιχη του `wiki-100`.

Από τα λίγα IDs που είδαμε παραπάνω, παρατηρούμε πως σε γενικές γραμμές το `wiki` W2V 100 διαστάσεων αποδίδει σχεδόν πάντα καλύτερα από το αντίστοιχο των 50 . Για το `wiki` W2V & `twitter` W2V δεν μπορούμε να κρίνουμε καθολικά. Στα παραπάνω IDs υπάρχουν αρκετές περιπτώσεις που το `twitter` επικρατεί του `wiki`. Ωστόσο, σε άλλες πιο "δύσκολες" περιπτώσεις, φαίνεται το `wiki` να δίνει καλύτερες κορυφαίες προτάσεις ενώ το `twitter`, μεγαλύτερο πλήθος από καλές προτάσεις. 

Και τα δύο embeddings δουλεύουν εξ'ίσου καλά. Ίσως με βάση τα παραπάνω, να δίναμε slight-edge στο `twitter`, χωρίς ωστόσο να είναι μεγάλο, κρίνοντας από τις παραπάνω δοκιμές. Στη συνέχεια όμως, για τη παραγωγή του SOM, χρησιμοποιούμε το `wiki-100`, το οποίο όπως θα δούμε, δίνει πολύ ικανοποιητικά αποτελέσματα στον genre-wise διαχωρισμό των ταινιών.

***Ποιοτικά χαρακτηριστικά***

Η δημιουργία της W2V αναπαράστασης των περιγραφών δεν είναι ιδιαίτερα χρονοβόρα (συγκεκριμένα στη περίπτωση του transfer learning όπου τα word embeddings είναι ήδη έτοιμα). Η δημιουργία της W2V αναπαράστασης χρειάζεται γύρω στα 30 δεύτερα στα embeddings που χρησιμοποιήσαμε παραπάνω. (+ το χρόνο για το preprocessing & download των embeddings. Σαφώς δεν χρειάζεται να κάνουμε αυτή τη διαδικασία πάντα) Συνολικά δημιουργείται ένα coprus με αρκετά μικρότερο μέγεθος από ό,τι η tf-idf αναπαράσταση. (Από ~ 5000x15000 οδηγούμαστε σε dataset με shape 5000x200) Παρά τη μεγάλη συμπίεση, δεν χάνουμε σε "επίδοση" από τον tf-idf. Κάθε άλλο, υπάρχουν αρκετές περιπτώσεις, που αποδίδει καλύτερα η W2V αναπαράσταση, όπως είδαμε παραπάνω.

## Πρακτικό tip - persistence αντικειμένων με joblib.dump

Καθώς στην δεύτερη εργασία καλείστε να δημιουργήσετε διάφορα corpora των οποίων η δημιουργία παίρνει χρόνο, υπάρχει ένας εύκολος τρόπος να αποθηκεύουμε μεταβλητές σε dump files και να τις διαβάζουμε απευθείας.

H βιβλιοθήκη [joblib](https://pypi.python.org/pypi/joblib) της Python δίνει κάποιες εξαιρετικά χρήσιμες ιδιότητες στην ανάπτυξη κώδικα: pipelining, παραλληλισμό, caching και variable persistence. Τις τρεις πρώτες ιδιότητες τις είδαμε στην πρώτη άσκηση. Στην παρούσα άσκηση θα μας φανεί χρήσιμη η τέταρτη, το persistence των αντικειμένων. Συγκεκριμένα μπορούμε με:

```python
joblib.dump(my_object, 'my_object.pkl') 
```

να αποθηκεύσουμε οποιοδήποτε αντικείμενο-μεταβλητή (εδώ το `my_object`) απευθείας πάνω στο filesystem ως αρχείο, το οποίο στη συνέχεια μπορούμε να ανακαλέσουμε ως εξής:

```python
my_object = joblib.load('my_object.pkl')
```

Μπορούμε έτσι να ανακαλέσουμε μεταβλητές ακόμα και αφού κλείσουμε και ξανανοίξουμε το notebook, χωρίς να χρειαστεί να ακολουθήσουμε ξανά όλα τα βήματα ένα - ένα για την παραγωγή τους, κάτι ιδιαίτερα χρήσιμο αν αυτή η διαδικασία είναι χρονοβόρα.

Ας αποθηκεύσουμε το `corpus_tf_idf` και στη συνέχεια ας το ανακαλέσουμε.

In [ ]:
# Make sure to add a shortcut to '/dumps' to your drive!
# https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# SHELL_PATH = "\"/content/drive/My Drive/dumps\""
# PATH = SHELL_PATH.replace('\"', '')

In [ ]:
# Check if the drive is correcly mount.
# !ls $SHELL_PATH

sample_data


In [ ]:
# import joblib
# import shutil

# def save(obj, name):
#   joblib.dump(obj, f'{PATH}/{name}.pkl') 

# def load(name):
#   return joblib.load(f'{PATH}/{name}.pkl')

<br>

In [ ]:
# save(corpus_tf_idf, 'corpus_tf_idf')



Μπορείτε με ένα απλό `!ls` να δείτε ότι το αρχείο `corpus_tf_idf.pkl` υπάρχει στο filesystem σας (== persistence):

In [ ]:
# !ls $SHELL_PATH

 corpus_tf_idf.pkl  'test file.gdoc'   the_best_corpus_ever.pkl


και μπορούμε να τα διαβάσουμε με `joblib.load`

In [ ]:
# corpus_tf_idf = load('corpus_tf_idf')

<br>

In [ ]:
# save(the_best_corpus_ever, 'the_best_corpus_ever') 

# Εφαρμογή 2.  Τοπολογική και σημασιολογική απεικόνιση της ταινιών με χρήση SOM
<img src="https://i.imgur.com/Z4FdurD.jpg" width="60%">

## Δημιουργία dataset
Στη δεύτερη εφαρμογή θα βασιστούμε στις τοπολογικές ιδιότητες των Self Organizing Maps (SOM) για να φτιάξουμε ενά χάρτη (grid) δύο διαστάσεων όπου θα απεικονίζονται όλες οι ταινίες της συλλογής της ομάδας με τρόπο χωρικά συνεκτικό ως προς το περιεχόμενο και κυρίως το είδος τους (ο παραπάνω χάρτης είναι ενδεικτικός, δεν αντιστοιχεί στο dataset μας). 

Διαλέξτε για την αναπαράσταση των documents αυτήν που πιστεύετε απέδωσε καλύτερα στο πρώτα σκέλος της άσκησης. Έστω ότι αυτή είναι η `my_best_corpus`.

Η έτοιμη συνάρτηση `build_final_set` θα ενώσει την αναπαράσταση που θα της δώσετε ως όρισμα `mycorpus` με τις binarized κατηγορίες `catbins` των ταινιών ως επιπλέον κολόνες (χαρακτηριστικά). Συνεπώς, κάθε ταινία αναπαρίσταται στο Vector Space Model από τα χαρακτηριστικά της αναπαράστασης `mycorpus` και τις κατηγορίες της.

Τέλος, η συνάρτηση δέχεται ένα ορισμα για το πόσες ταινίες να επιστρέψει, με default τιμή όλες τις ταινίες (5000). Αυτό είναι χρήσιμο για να μπορείτε αν θέλετε να φτιάχνετε μικρότερα σύνολα δεδομένων ώστε να εκπαιδεύεται ταχύτερα το SOM. 

Θα τρέχουμε τη συνάρτηση με `final_set = build_final_set(my_best_corpus)`.

In [ ]:
# my_best_corpus = load('the_best_corpus_ever')
my_best_corpus = the_best_corpus_ever
my_best_corpus.shape

(5000, 100)

In [ ]:
def build_final_set(mycorpus, doc_limit = 5000, tf_idf_only=False):
    # convert sparse tf_idf to dense tf_idf representation
    dense_tf_idf = mycorpus.toarray()[0:doc_limit,:]
    if tf_idf_only:   # use only tf_idf
        final_set = dense_tf_idf
    else:             # append the binary categories features horizontaly to the (dense) tf_idf features
        final_set = np.hstack((dense_tf_idf, catbins[0:doc_limit,:]))   # np.hstack((np[1, 2], np[3, 4])) = np[1, 2, 3, 4]
    # η somoclu θέλει δεδομένα σε float32
    return np.array(final_set, dtype=np.float32)

Στο επόμενο κελί, τυπώνουμε τις διαστάσεις του τελικού dataset μας. **Χωρίς βελτιστοποίηση του TFIDF** θα έχουμε περίπου 50.000 χαρακτηριστικά και ο θα είναι ανέφικτο να προχωρήσουμε στην εκπαίδευση του SOM.

In [ ]:
final_set = build_final_set(my_best_corpus)

In [ ]:
final_set.shape

`final_set.shape = (5000, 422)`

## Εκπαίδευση χάρτη SOM

Θα δουλέψουμε με τη βιβλιοθήκη SOM ["Somoclu"](http://somoclu.readthedocs.io/en/stable/index.html). Εισάγουμε τις somoclu και matplotlib και λέμε στη matplotlib να τυπώνει εντός του notebook (κι όχι σε pop up window).

In [ ]:
# (removed --upgrade)
### !pip install somoclu

import somoclu

import matplotlib
%matplotlib inline
# We will plot inside the notebook and not in separate window

###!pip install tabulate
from tabulate import tabulate

Καταρχάς διαβάστε το [function reference](http://somoclu.readthedocs.io/en/stable/reference.html) του somoclu. Θα δoυλέψουμε με χάρτη τύπου <u>planar</u>, <u>παραλληλόγραμμου σχήματος</u> νευρώνων με <u>τυχαία αρχικοποίηση</u> (όλα αυτά είναι default). Μπορείτε να δοκιμάσετε διάφορα μεγέθη χάρτη ωστόσο όσο ο αριθμός των νευρώνων μεγαλώνει, μεγαλώνει και ο χρόνος εκπαίδευσης. Για το training δεν χρειάζεται να ξεπεράσετε τα 100 epochs. Σε γενικές γραμμές μπορούμε να βασιστούμε στις default παραμέτρους μέχρι να έχουμε τη δυνατότητα να οπτικοποιήσουμε και να αναλύσουμε ποιοτικά τα αποτελέσματα. Ξεκινήστε με ένα χάρτη 10 x 10, 100 epochs training και ένα υποσύνολο των ταινιών (π.χ. 2000). Χρησιμοποιήστε την `time` για να έχετε μια εικόνα των χρόνων εκπαίδευσης. 

In [ ]:
neurons_count_sqrt = 20   #20, 25, 30
som = somoclu.Somoclu(neurons_count_sqrt, neurons_count_sqrt, initialization='random')  #example: 10 x 10 = 100 neurons

start = time.time()
som.train(data=final_set, epochs=100)
print(f"Time to train SOM with {final_set.shape[0]} vectors and 100 epochs: {time.time() - start:.3f} seconds")

Time to train SOM with 5000 vectors and 100 epochs: 145.832 seconds


Για την εκπαίδευση του δικτύου SOM με 20 νευρώνες απαιτούνται 2.5 λεπτά περίπου.

`Time to train SOM with 5000 vectors and 100 epochs: 145.832 seconds`


Για 30 νευρώνες απαιτούνται 5 λεπτά:

`Time to train SOM with 5000 vectors and 100 epochs: 293.617 seconds`


Τα επόμενα κελιά θα εκτελεσθούν με τους 30 νευρώνες, καθώς είναι οι περισσότεροι νευρώνες και 5 λεπτά είναι αποδεκτός χρόνος αναμονής για την εκπαίδευση.

In [ ]:
# save(som, f'trained_som_{neurons_count_sqrt}')

In [ ]:
# som = load('BEST_som_30_clustered')


## Best matching units

Μετά από κάθε εκπαίδευση αποθηκεύστε σε μια μεταβλητή τα best matching units (bmus) για κάθε ταινία. Τα bmus μας δείχνουν σε ποιο νευρώνα ανήκει η κάθε ταινία. **Προσοχή: η σύμβαση των συντεταγμένων των νευρώνων στη Somoclu είναι (στήλη, γραμμή) δηλαδή το ανάποδο από την Python**. Με χρήση της [np.unique](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.unique.html) (μια πολύ χρήσιμη συνάρτηση στην άσκηση) αποθηκεύστε τα μοναδικά best matching units και τους δείκτες τους (indices) προς τις ταινίες. 

Σημειώστε ότι μπορεί να έχετε λιγότερα μοναδικά bmus από αριθμό νευρώνων γιατί μπορεί σε κάποιους νευρώνες να μην έχουν ανατεθεί ταινίες. Ως αριθμό νευρώνα θα θεωρήσουμε τον αριθμό γραμμής στον πίνακα μοναδικών bmus.


In [ ]:
#som.bmus.shape: doc_limit x 2D = movies x COL and ROW position of SOM neuron
bmus = som.bmus   # best(=nearest) neuron for each train movie (like using 100-means with non-moving centers)
bmus_u, bmus_u_firstindices = np.unique(som.bmus, return_index=True, axis=0)

print("bmus = ")
print(bmus)
print()
print(f"Bmus uniques shape = {bmus_u.shape}\n")
print("First bmu movie for each neuron = ")
print(bmus_u_firstindices)

#som.view_umatrix(bestmatches = True, colorbar = True)

**Τα best matching units**:
```
bmus = 
[[12 24]
 [25 22]
 [15  0]
 ...
 [29 21]
 [10 19]
 [15 19]]

Bmus uniques shape = (867, 2)

First bmu movie for each neuron = 
[ 164  648  710  220   57  904   16  162  253 1360 1203   63 1059   20
  141 2453  377  684 2569   62   37  330  740 1548  680  571   46  654
    6  358  304 1406  430 3510  216  317  972  442  782  168   98  224
 1543 1392 1016  148  390 1048 1176  105  219   47  371  383  794 3151
  149 2171  368  212  586   24  189   36   19 1000 1085   82  235    3
  823  861 1250  832 1102  532  177 2307  200  752   52  265  805  365
  354   81  155  894  180   41 1200 3164 2869 1533 1148  827 1135  182
  920  241  353 2912  749 2027 1404  516 1500  137  198  400 1417 2765
  937 3853   13  109  662 1385  195  777  229  118   28   31 1338   54
  343  315 1713 1754  643 1229  117  534  992  355  461 2443 1945 2995
  559  580 1312  299  172  938  103  875 1942 1238  159  750 2692  413
 1095  376 1175 2432 4369 1820  473 3485  857  802 1296  206 1783  531
  406 1486  492  966  255 1439  756 3546  314 1330  190  854  459  104
   89  217 1581 4670  870 1764  918  297 1125 1343  835 1205  298  675
 2900  706 3334  477 1990 1065  587 1077  114  489 1043  285  125  222
   79 1659 1335  622 3755 2918 4182 4486  100  373  924  251  775   27
  384 1560  205   76  214  642  395  160  576 1155 1716 1569 1362 1399
  101 1348  792 2183  798  996 2100  230  760  268 1277 4649  336   33
  307  411 1773 1761  930  423 1488  238  880  997 3944 1447  751   70
  154 1803  810   12  677   84  401 4840   71  796   85  566 2904  484
 3254  347 2229  521  110  621 1328  755 3229  977  615  720    9 1481
   45   78  801  509   40   23 3517  296  112 2655  917 2840  947 1380
 1143  550 2343  416   66 3055  460 1373  313   26  318  952  591  202
  402  113  249   14  578 2048 1388 2953  215   97 3458 3781 2126  569
 3574  115 3454  397  850 2016 1332  187  713   39 3744 1288 2280  769
 4897  231  292 2425  294 1146 1661 2828  475 3827  102 1630  599  254
 4484  167 3923  672  537  445 2083    0 1390 2462  175 2564  519 1026
  847 1838   75  845 1024 2324 4373 2419 1108   59 2674   83 2427 1739
 2153 1054 3482 2932 1840  234 3171  281  126  562  577  708  518 1790
 1056  420  146   42  157 1772   48   50   94   55  603  360 3326  652
 1740 1566  513 1589   10    5  540  145 2578  150 1398  363 1257  522
 1578   67    2  352  121 1204 4174 3400  579  608 2418  213  123  140
  588  712  728 1309  106  525 1874  436  269 1567   34 2874 1206 1261
 1061   90  279 1895  332  147  812  124 2284  262 1133 2409 3448  312
  366  142    4 3819 1504 1598  199  803  911   88   86   60  122 3497
  325 1463 2311 2049  545 1256  971 1074  612  284 1185  874  490  165
   61  592   72 2978 1892  616  393 4252  399 2671 2408  589  362  394
  306  718 1055 1443 1781  119 1880 1974  514  226  841  334 1722  151
   65 1966  563  690  295 1190  933   29 4387  266  278 1162  722 2416
  218  194  415  575 2927  130 3007 3012 1196 3679  734 1280  178  768
  431  135  986 1758  239  272  799  658 1071  271 2834 1005  631 4120
  533  440  501  204  942 1850 4412   22  153 1649 1949  761 4797 2235
 1117  133   11 2335  541 1106  581  303 1923  779  733  638 1717 2577
 1745 2216  111  341  619  374  179 1050  670  547   43 1924  396  264
  476  482 1268  957 2546 1161  283   91  620   44  250 1031 1767  300
 3402  181 1473 1301  554  132  335  660 1637 3690 2471  472 1356   25
  539 1386 1222    7 2647  961 2024 1830  520  480  386 1835 2813 1609
  370 1317  853   49 2585 2204  408 1303 2780   32 1520   17 1047  538
  822 3985 1094  333  345 2091 2412 1410 1092  232  183 2768   38 1297
  170  344  176 1073 1170  138  169  375 1341 1314  207 1618  273  463
  453  474  955  703   87  301  233 1152 1675  770   30   77  171  367
 1917  465  560  502  499  228  328 2191  107  946  210  526  173  323
  666  364 4995 1749 1081  717   96   73  821 1060  542 1889  327  309
 1233  410    1 2137  161 4448  773  339 2371  747  244  809  683  602
  653 1856 1920 1353  466  481 1315  495 1565 2152  903  429  188  627
  558  120  388  730  681  555  887  163  286  721 2937  270    8  403
 2222  131  209 2125  134  378  263   15  186  487   35  143   93 1535
   18  651 1232 2337 1705 2979  166  582  650  848 1099  331 1715  908
  192  772 1156 2014  725  528   53 2388 1992  208  369  421  438  256
  739  647  116  548  695  536  248   64  225 1542 1013  136 1536  293
 1174   68  448  221  454  381  859 2364  667  511  422   58  505  561
  158  329 1977  682   95   56 1044   21  348  806 1239 1249  302]
  ```


## Ομαδοποίηση (clustering)

Τυπικά, η ομαδοποίηση σε ένα χάρτη SOM προκύπτει από το unified distance matrix (<u>U-matrix</u>): για κάθε κόμβο υπολογίζεται η **μέση απόστασή** του από τους γειτονικούς κόμβους. Εάν χρησιμοποιηθεί <font color="blue">μπλε</font> χρώμα στις περιοχές του χάρτη όπου η τιμή αυτή είναι χαμηλή (μικρή απόσταση) και <font color="red">κόκκινο</font> εκεί που η τιμή είναι υψηλή (μεγάλη απόσταση), τότε μπορούμε να πούμε ότι **οι μπλε περιοχές αποτελούν clusters** και οι κόκκινες αποτελούν σύνορα μεταξύ clusters.

To somoclu δίνει την επιπρόσθετη δυνατότητα να κάνουμε **ομαδοποίηση των νευρώνων** χρησιμοποιώντας οποιονδήποτε αλγόριθμο ομαδοποίησης του scikit-learn. Στην άσκηση θα χρησιμοποιήσουμε τον k-Means. Για τον αρχικό σας χάρτη δοκιμάστε ένα k=20 ή 25. Οι δύο προσεγγίσεις ομαδοποίησης είναι διαφορετικές, οπότε περιμένουμε τα αποτελέσματα να είναι κοντά αλλά όχι τα ίδια.


In [ ]:
from sklearn.cluster import KMeans

def somc(clusters): #som object is MODIFIED
  somcl = som
  somcl.cluster(algorithm = KMeans(n_clusters=clusters))
  somcl.view_umatrix(bestmatches = True, colorbar = True)
  return somcl
# color of neuron indicates the cluster

In [ ]:
som_cl_20 = somc(20)   # 20-means

Το SOM των 30 νευρώνων με 20-means clustering:

![20-means](https://i.postimg.cc/ydnncn0T/sssssssss.png)

In [ ]:
som_cl_25 = somc(25)   # 25-means

Με 25-means clustering:

![25means](https://i.postimg.cc/mkpRr0Vf/25means.png)


## Αποθήκευση του SOM

Επειδή η αρχικοποίηση του SOM γίνεται τυχαία και το clustering είναι και αυτό στοχαστική διαδικασία, οι θέσεις και οι ετικέτες των νευρώνων και των clusters θα είναι διαφορετικές κάθε φορά που τρέχετε τον χάρτη, ακόμα και με τις ίδιες παραμέτρους. Για να αποθηκεύσετε ένα συγκεκριμένο **som** και clustering χρησιμοποιήστε και πάλι την `joblib`. Μετά την ανάκληση ενός SOM θυμηθείτε να ακολουθήσετε τη διαδικασία για τα **bmus**.


In [ ]:
# save(som, f'GOODENOUGH_som_tor_20_clustered')


## Οπτικοποίηση U-matrix, clustering και μέγεθος clusters

Για την εκτύπωση του U-matrix χρησιμοποιήστε τη `view_umatrix` με ορίσματα `bestmatches=True` και `figsize=(15, 15)` ή `figsize=(20, 20)`. Τα διαφορετικά χρώματα που εμφανίζονται στους κόμβους αντιπροσωπεύουν τα διαφορετικά clusters που προκύπτουν από τον k-Means. Μπορείτε να εμφανίσετε τη λεζάντα του U-matrix με το όρισμα `colorbar`. Μην τυπώνετε τις ετικέτες (labels) των δειγμάτων, είναι πολύ μεγάλος ο αριθμός τους.

Για μια δεύτερη πιο ξεκάθαρη οπτικοποίηση του clustering τυπώστε απευθείας τη μεταβλητή `clusters`.

Τέλος, χρησιμοποιώντας πάλι την `np.unique` (με διαφορετικό όρισμα) και την `np.argsort` (υπάρχουν και άλλοι τρόποι υλοποίησης) εκτυπώστε τις ετικέτες των clusters (αριθμοί από 0 έως k-1) και τον αριθμό των νευρώνων σε κάθε cluster, με φθίνουσα ή αύξουσα σειρά ως προς τον αριθμό των νευρώνων. Ουσιαστικά είναι ένα εργαλείο για να βρίσκετε εύκολα τα μεγάλα και μικρά clusters. 

Ακολουθεί ένα μη βελτιστοποιημένο παράδειγμα για τις τρεις προηγούμενες εξόδους:

<img src="https://image.ibb.co/i0tsfR/umatrix_s.jpg" width="35%">
<img src="https://image.ibb.co/nLgHEm/clusters.png" width="35%">



In [ ]:
# som = load('my_best_som')
som.view_umatrix(colorbar=True, bestmatches=True, figsize=(15, 15))   #distance of neighboring neurons

print("Clusters:")
print(som.clusters)

print()
ids, counts = np.unique(som.clusters, return_counts=True)
count_order = np.argsort(counts)
print(tabulate([ ["Cluster"] + ids[count_order].tolist(), ["SOM neurons count"] + counts[count_order].tolist()]))

![](https://i.postimg.cc/hvZzqZdh/big.png)

Clusters:
```
[[12 12 12 12 12 12 15 15 15 15 15 15 15  4  4  4  4  4  4  4  4  4  3  8  8  8  8  8  8  8]
 [12 12 12 12 12 12  3 15 15 15 15 15 15  4  4  4  4  4  4  4  4  4  8  8  8  8  8  8  8  8]
 [12 12 12 12 12 12 10 10 15 15 15 15 15  4  4  4  4  4  4  4  4  4  8  8  8  8  8  8  8  8]
 [12 12 12 12 12  3  3  3  3 15 15 15 16 16  4  4  4  4  4  4  4 16  8  8  8  8  8  8  3  3]
 [12 12 12 12  3  3  3  3  3 16 16 16 16 16 16 16 16  4  4  4 16 16  8  8  8  6  8  3  3  3]
 [12 12  3  3  3  3  3  3  3  3 16 16 16 16 16 16 16 16  4  4 16  6  6  6  6  6  6  3  3  3]
 [12  3  3  3  3  3  3  3  3  3 16 16 16 16 16 16 16 16 16 16  6  6  6  6  6  6  6 10 10  3]
 [ 3  3  3  3  3  3  3  3  3  3  3 16 16 16 16 16 16 16 16 16  6  6  6  6  6  6  6 10 10 10]
 [ 3  3  3  3  3  3  3  3  3  3  3  3 16 16 16 16 16 16 16 16  6  6  6  6  6  6  6 10 10 10]
 [ 3  3  3  3  3  3  3  3  3  3  3  3 16 18 18 18 18 18 18 18 18  6  6  6  6  6  6 10 10 10]
 [ 3  3  3  3  3  3  3  3  3  3  3  3 18 18 18 18 18 18 18 18 18 18  6  6  6  9 10 10 10 10]
 [ 3  3  3  3  3  3  3  3  3  3  3  3 18 18 18 18 18 18 18 18 18 18  6  6  9  9 10 10 10 10]
 [ 3  3  3  3  3  3  3  3  3  3  3  3 18 18 18 18 18 18 18 18 18  9  9  9  9  9 10 10 10 10]
 [ 3  3  3  3  3  3  3  3  3  5  3  3 12 18 18 18 18 18 18 18 18  9  9  9  9  9  9 10 10 10]
 [ 3  3  3  3  3  3  5  5  5  5  5  3  3 18 18 18 18 18 18 18 18  9  9  9  9  9  9  9  9  9]
 [ 3  3  3  3 17  5  5  5  5  5  5  5  5 18 18 18 18 18 18 18 18  9  9  9  9  9  9  9  9  9]
 [ 3  3 17 17 17 17  5  5  5  5  5  5  5 18 18 18 18 18 18 18  1  3  9  9  9  9  9  9  9  9]
 [ 3 19 17 17 17 17 17  5  5  5  5  5  5 18 18 18 18 18  1  1  1  1  3  9  9  9  9  9  9  9]
 [19 19 17 17 17 17 17  5  5  5  5  5  5  2  2  2  1  1  1  1  1  1  1 11  9  9  9  9  9  9]
 [19 19 17 17 17 17 17 17 17  5  5  5  2  2  2  2  1  1  1  1  1  1 11 11  9  9  9  9  9  9]
 [19 19 19 17 17 17 17 17 17 14  5  5  2  2  2  2  2  1  1  1  1  1 11 11 11 11  9  9  9  9]
 [19 19 19 19 17 17 17 17 14 14 14  5  2  2  2  2  2  1  1  1  1  1 11 11 11 11 11  9  9  9]
 [19 19 19 19 19 19 17 14 14 14 14  5  2  2  2  2  2  1  1  1  1 11 11 11 11 11 11 11 11 11]
 [13 19 19 19 19 19 19 14 14 14 14 14 14  2  2  2  2  1  1  1  1 11 11 11 11  7  7 11 11 11]
 [13 13 13 19 19 19 19 14 14 14 14 14  2  2  2  2  2  2  0  0  0 18 18 11  7  7  7 11 11 11]
 [13 13 13 13 19 19 19 14 14 14 14 14  2  2  2  2  2  0  0  0  0 18 18  7  7  7  7  7 11 11]
 [13 13 13 13 13 19 19 14 14 14 14 14 14  2  2  2  0  0  0  0  0  0 18  7  7  7  7  7  7  7]
 [13 13 13 13 13 13 19 14 14 14 14 14 14  2  2  2  0  0  0  0  0  0  0  7  7  7  7  7  7  7]
 [13 13 13 13 13 13 19 14 14 14 14 14 14  2  2  2  2  0  0  0  0  0  0  7  7  7  7  7  7  7]
 [13 13 13 13 13 13 13 19 14 14 14 14  0  2  2  2  2  0  0  0  0  0  0  0  7  7  7  7  7  7]]
```

```
-----------------  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  ---
Cluster            15  10  12  13  17   8   0  11  19   1   7   6   4   5  14  16   2   9  18    3
SOM neurons count  21  28  31  32  33  33  34  34  36  36  37  39  39  40  46  50  51  66  78  136
-----------------  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  ---
```

In [ ]:
print("Τα IDs των clusters ταξινομημένα, σε μορφή λίστας:")
ids[count_order].tolist()

Τα IDs των clusters ταξινομημένα, σε μορφή λίστας:


[15, 10, 12, 13, 17, 8, 0, 11, 19, 1, 7, 6, 4, 5, 14, 16, 2, 9, 18, 3]


## Σημασιολογική ερμηνεία των clusters

Προκειμένου να μελετήσουμε τις τοπολογικές ιδιότητες του SOM και το αν έχουν ενσωματώσει σημασιολογική πληροφορία για τις ταινίες διαμέσου της διανυσματικής αναπαράστασης του tf-idf, των εμφυτευμάτων και των κατηγοριών, χρειαζόμαστε ένα κριτήριο ποιοτικής επισκόπησης των clusters. 

Θα υλοποιήσουμε το εξής κριτήριο: Λαμβάνουμε όρισμα έναν αριθμό (ετικέτα) cluster. Για το cluster αυτό βρίσκουμε όλους τους νευρώνες που του έχουν ανατεθεί από τον k-Means. Για όλους τους νευρώνες αυτούς βρίσκουμε όλες τις ταινίες που τους έχουν ανατεθεί (για τις οποίες αποτελούν bmus). Για όλες αυτές τις ταινίες τυπώνουμε ταξινομημένη τη συνολική στατιστική όλων των ειδών (κατηγοριών) και τις συχνότητές τους. Αν το cluster διαθέτει καλή συνοχή και εξειδίκευση, θα πρέπει κάποιες κατηγορίες να έχουν σαφώς μεγαλύτερη συχνότητα από τις υπόλοιπες. Θα μπορούμε τότε να αναθέσουμε αυτήν/ές την/τις κατηγορία/ες ως ετικέτες κινηματογραφικού είδους στο cluster.

Μπορείτε να υλοποιήσετε τη συνάρτηση αυτή όπως θέλετε. Μια πιθανή διαδικασία θα μπορούσε να είναι η ακόλουθη:

1. Ορίζουμε <u>συνάρτηση `print_categories_stats`</u> που δέχεται ως είσοδο λίστα με ids ταινιών. Δημιουργούμε μια κενή λίστα συνολικών κατηγοριών. Στη συνέχεια, για κάθε ταινία επεξεργαζόμαστε το string `categories` ως εξής: δημιουργούμε μια λίστα διαχωρίζοντας το string κατάλληλα με την `split` και αφαιρούμε τα whitespaces μεταξύ ετικετών με την `strip`. Προσθέτουμε τη λίστα αυτή στη συνολική λίστα κατηγοριών με την `extend`. Τέλος χρησιμοποιούμε πάλι την `np.unique` για να μετρήσουμε συχνότητα μοναδικών ετικετών κατηγοριών και ταξινομούμε με την `np.argsort`. Τυπώνουμε τις κατηγορίες και τις συχνότητες εμφάνισης ταξινομημένα. Χρήσιμες μπορεί να σας φανούν και οι `np.ravel`, `np.nditer`, `np.array2string` και `zip`.

2. Ορίζουμε τη βασική μας <u>συνάρτηση `print_cluster_neurons_movies_report`</u> που δέχεται ως όρισμα τον αριθμό ενός cluster. Με τη χρήση της `np.where` μπορούμε να βρούμε τις συντεταγμένες των bmus που αντιστοιχούν στο cluster και με την `column_stack` να φτιάξουμε έναν πίνακα bmus για το cluster. <font color="red">Προσοχή στη σειρά (στήλη - σειρά) στον πίνακα bmus.</font> Για κάθε bmu αυτού του πίνακα ελέγχουμε αν υπάρχει στον πίνακα μοναδικών bmus που έχουμε υπολογίσει στην αρχή συνολικά και αν ναι προσθέτουμε το αντίστοιχο index του νευρώνα σε μια λίστα. Χρήσιμες μπορεί να είναι και οι `np.rollaxis`, `np.append`, `np.asscalar`. Επίσης πιθανώς να πρέπει να υλοποιήσετε ένα κριτήριο ομοιότητας μεταξύ ενός bmu και ενός μοναδικού bmu από τον αρχικό πίνακα bmus.

3. Υλοποιούμε μια βοηθητική <u>συνάρτηση `neuron_movies_report`</u>. Λαμβάνει ένα σύνολο νευρώνων από την `print_cluster_neurons_movies_report` και μέσω της `indices` φτιάχνει μια λίστα με το σύνολο ταινιών που ανήκουν σε αυτούς τους νευρώνες. Στο τέλος καλεί με αυτή τη λίστα την `print_categories_stats` που τυπώνει τις στατιστικές των κατηγοριών.

Μπορείτε βέβαια να προσθέσετε οποιαδήποτε επιπλέον έξοδο σας βοηθάει. Μια χρήσιμη έξοδος είναι πόσοι νευρώνες ανήκουν στο cluster και σε πόσους και ποιους από αυτούς έχουν ανατεθεί ταινίες.

Θα επιτελούμε τη σημασιολογική ερμηνεία του χάρτη καλώντας την `print_cluster_neurons_movies_report` με τον αριθμός ενός cluster που μας ενδιαφέρει. 

Παράδειγμα εξόδου για ένα cluster (μη βελτιστοποιημένος χάρτης, ωστόσο βλέπετε ότι οι μεγάλες κατηγορίες έχουν σημασιολογική  συνάφεια):

```
Overall Cluster Genres stats:  
[('"Horror"', 86), ('"Science Fiction"', 24), ('"B-movie"', 16), ('"Monster movie"', 10), ('"Creature Film"', 10), ('"Indie"', 9), ('"Zombie Film"', 9), ('"Slasher"', 8), ('"World cinema"', 8), ('"Sci-Fi Horror"', 7), ('"Natural horror films"', 6), ('"Supernatural"', 6), ('"Thriller"', 6), ('"Cult"', 5), ('"Black-and-white"', 5), ('"Japanese Movies"', 4), ('"Short Film"', 3), ('"Drama"', 3), ('"Psychological thriller"', 3), ('"Crime Fiction"', 3), ('"Monster"', 3), ('"Comedy"', 2), ('"Western"', 2), ('"Horror Comedy"', 2), ('"Archaeology"', 2), ('"Alien Film"', 2), ('"Teen"', 2), ('"Mystery"', 2), ('"Adventure"', 2), ('"Comedy film"', 2), ('"Combat Films"', 1), ('"Chinese Movies"', 1), ('"Action/Adventure"', 1), ('"Gothic Film"', 1), ('"Costume drama"', 1), ('"Disaster"', 1), ('"Docudrama"', 1), ('"Film adaptation"', 1), ('"Film noir"', 1), ('"Parody"', 1), ('"Period piece"', 1), ('"Action"', 1)]```
   

In [ ]:
# movie ids -> all categories
def print_categories_stats(movie_ids, show_sum=False):
  list_categories = []

  # Keep the one that you prefer. I just found all of these comprehensions a bit overkill.
  for movie in categories[movie_ids]:
    split_categories = movie[0].split(',')
    filtered_categories = map(lambda c: c.strip()[1:-1], split_categories)
    list_categories += filtered_categories # This extension has the same complexity as the flattening.
  
  ids, counts = np.unique(list_categories, return_counts = True)
  count_order = np.argsort(-counts) # Sort in decreasing order.

  ids_c = ids[count_order].tolist()
  cnt_c = counts[count_order].tolist()
  print("Genres stats for these movies:")
  print(tabulate([ ["Category:"] + ids_c, ["Count:"] + cnt_c]))
  
  if show_sum:
    print(f"Total number of categories {counts.sum()}")


# cluster_id -> SOM neurons (bmus)
def print_cluster_neurons_movies_report(cluster_id, ret = False):  # 0-24
  cl = som.clusters
  i, j = np.where(cl == cluster_id) 
  cl_bmus = np.column_stack((j, i))  # col, row

  sbu = set(tuple(x) for x in bmus_u)
  cl_bmus_existing = np.array([x for x in cl_bmus if tuple(x) in sbu])

  print("Cluster neurons:")
  print(cl_bmus.tolist())   # tolist() for printing in one line
  print()
  print("Existing cluster neurons:")
  print(cl_bmus_existing.tolist())
  print()

  if ret:
    return cl_bmus_existing


def neuron_movies_report(cluster_id):
  cl_bmus = print_cluster_neurons_movies_report(cluster_id, ret=True)

  movie_ids = []
  for clb in cl_bmus:
    for bi in range(len(bmus)):
      if (clb == bmus[bi]).all():
        movie_ids.append(bi)
  
  print("Matching movie IDs:")
  print(movie_ids)
  print()

  print_categories_stats(movie_ids)

In [ ]:
print("~ All movies ~")
print_categories_stats(range(5000), True)

~ All movies ~
Genres stats for these movies:
---------  -----  ------  ------------  --------  ------  ------------  ------  -------------  ---------------  ----------------  -----  ---------  ----------  -----------  --------------  -------  ---------------  ---------  -------  ---------------  -------  -----------  --------  --------------  ------------  -----------  ---------------  ---------------  ------------  -----------  -------  ---------  ----------------------  --------------  -----------------  ----  ------------  ------  -------------  ------  ------------  ----  -----------------  ----------------  ---------  -----------------  ------------  ---------  -------  ---------  ------  ---------------  ---------  ----------  --------  -------------  -------  ----------------  -----------  -------------  --------  -----  ----------------  -------------  ------------------  -------  ------------------  -----------  -------------  ---  ----------  ----------  ------------  ------

```
~ All movies ~
Genres stats for these movies:
---------  -----  ------  ------------  --------  ------  ------------  ------  -------------  ---------------  ----------------  -----  ---------  ----------  -----------  --------------  -------  ---------------  ---------  -------  ---------------  -------  -----------  --------  --------------  ------------  -----------  ---------------  ---------------  ------------  -----------  -------  ---------  ----------------------  --------------  -----------------  ----  ------------  ------  -------------  ------  ------------  ----  -----------------  ----------------  ---------  -----------------  ------------  ---------  -------  ---------  ------  ---------------  ---------  ----------  --------  -------------  -------  ----------------  -----------  -------------  --------  -----  ----------------  -------------  ------------------  -------  ------------------  -----------  -------------  ---  ----------  ----------  ------------  -------------------------  ---------------  ----------------  ---------------  -----  ------------------  --------  --------------  ---------  -----------------  -----------------  ---------  ----------------  ----------------  ---------------  -----------------  ----------  -------------  -------------  --------------  --------------  ---------------  -----  -----------  --------------------  ---------------  ---------  ------------------  -------------  ----  ---------------  --------------  ------------------  -------  -------------  -----------------  --------------  ----------------  -----  ----------  --------  -----------------  -----  ------------  ----------------  --------------  ------------  ----------------  -------------------  -------------  -------------  -------------  --------------  -----------------  ------------  ------------  ----------------  ---------  ------  --------  ----------  -------------  -----------  ---------------  ---------------  -----------  ---------  -----------  ------------------  -----------  ----------------  -----------  --------------  -------------------  ---------------  ---------------  ------  -------  ------------  ---------  -----------  ----------------------  --------------  -----------  ------  ------------  --------------------  -------------------------  -----------------  -------------  -------------------  -----------  --------------  ---------------  ----------------  --------------  --------------  -----------------  --------  ----------------  --------  ---------------  -----------  -----------  ---------------------  --------------  ---------------  --------------  -----  -------------  -----------  -------------------  ------------  -----------  ---------------------------  ----------------------------------------  ---------  ------------  -------------  ---------  ----------------  ------------  --------------  ---------------------------  ----------------  -------------------  ------  ------------  -------------------------  -----------  ----------  --------  ------------  ------------------------------  -------  -----------------------  -----------  ---------  -----------------  ---------------  --------  -----------------------  ---------  -------------  ----------  --------------  ---------  --------  -----------------  -------------  -----------  ------  ---------------  --------  -------------  ----  ------  ---------  ------------  ---------  ------------------------  ---------------------  ---------  -------------  ---------------  ------------------  -------  --------  ---------  ----------  -------------  -----------  ----------------  --------------------  -------------------------------  --------------------------  -----------------  --------------------  ------  --------------  ---------------------  --------  -----------  ----------------  -------------------  -----------  ----------  ------------------------  ------------------------  ------------  --------------  -----------  ------------------------  -----------  --------------  ----------------  --------------  ---------------  ------  ----------------  --------  ------------------
Category:  Drama  Comedy  Romance Film  Thriller  Action  World cinema  Horror  Crime Fiction  Black-and-white  Action/Adventure  Indie  Adventure  Short Film  Family Film  Romantic drama  Musical  Science Fiction  Animation  Fantasy  Romantic comedy  Mystery  Comedy film  War film  Crime Thriller  Comedy-drama  Silent film  Japanese Movies  Film adaptation  Period piece  Documentary  Western  Bollywood  Psychological thriller  Chinese Movies  Children's/Family  Teen  Black comedy  Sports  Coming of age  Parody  Family Drama  Cult  Martial Arts Film  Television movie  Slapstick  Biographical film  Supernatural  Biography  Slasher  Film noir  Satire  Political drama  Melodrama  Children's  Suspense  Costume drama  B-movie  Action Thrillers  Crime Drama  Creature Film  Art film  Music  Biopic [feature]  Ensemble Film  Historical fiction  History  Children's Fantasy  Zombie Film  Gangster Film  Spy  Road movie  Buddy film  Crime Comedy  Family-Oriented Adventure  Superhero movie  Screwball comedy  Filipino Movies  Anime  Political thriller  Disaster  Fantasy Comedy  Docudrama  Comedy of manners  Culture & Society  Detective  Comedy of Errors  Political cinema  Domestic Comedy  Detective fiction  Sex comedy  Horror Comedy  Sci-Fi Horror  Musical comedy  Christian film  Courtroom Drama  Heist  Stop motion  Natural horror films  Childhood Drama  Superhero  Computer Animation  New Hollywood  Epic  Erotic thriller  Marriage Drama  Swashbuckler films  Monster  Action Comedy  Fantasy Adventure  Animal Picture  Historical drama  Wuxia  Alien Film  Pre-Code  Experimental film  Dance  Holiday Film  Adventure Comedy  Blaxploitation  Mockumentary  Romantic fantasy  Slice of life story  Monster movie  Doomsday film  Musical Drama  Comedy Western  Costume Adventure  Media Satire  Tamil cinema  Political satire  Absurdism  Boxing  Dystopia  Fairy tale  Feminist Film  Caper story  Comedy Thriller  Christmas movie  Stoner film  Americana  Avant-garde  Haunted House Film  Time travel  Animated cartoon  Chase Movie  Gross-out film  Revisionist Western  Historical Epic  Propaganda film  Prison  Tragedy  Combat Films  Gross out  Gothic Film  Airplanes and airports  Religious Film  Jungle Film  Remake  Rockumentary  Mythological Fantasy  Juvenile Delinquency Film  Spaghetti Western  Social issues  Social problem film  Auto racing  Hip hop movies  Film u00e0 clef  Animated Musical  Existentialism  Samurai cinema  Sword and sorcery  Whodunit  Workplace Comedy  Neo-noir  Addiction Drama  Educational  Future noir  Master Criminal Films  Hybrid Western  Medical fiction  Costume Horror  Crime  Splatter film  Escape Film  Inspirational Drama  Movie serial  Road-Horror  Roadshow theatrical release  Apocalyptic and post-apocalyptic fiction  Punk rock  Erotic Drama  Comedy horror  B-Western  Sci-Fi Adventure  Concert film  Bengali Cinema  Interpersonal Relationships  Sword and Sandal  Glamorized Spy Film  Travel  Epic Western  Heaven-Can-Wait Fantasies  Tragicomedy  Surrealism  Dogme 95  Cavalry Film  Sci Fi Pictures original films  Z movie  Science fiction Western  Hagiography  Tollywood  Backstage Musical  Heavenly Comedy  Anti-war  Sword and sorcery films  Steampunk  Anti-war film  Beach Film  Extreme Sports  Computers  Cold War  Outlaw biker film  Demonic child  Archaeology  Outlaw  Nuclear warfare  Northern  Fantasy Drama  News  Nature  Anthology  Feature film  Cyberpunk  Inventions & Innovations  Czechoslovak New Wave  Tokusatsu  Media Studies  Ealing Comedies  Early Black Cinema  Animals  Gulf War  Education  Mondo film  Gender Issues  Filmed Play  Supermarionation  Archives and records  Family & Personal Relationships  Parkour in popular culture  Children's Issues  Kitchen sink realism  Latino  Clay animation  Environmental Science  Business  Law & Crime  British New Wave  British Empire Film  Legal drama  Biker Film  Children's Entertainment  Libraries and librarians  Exploitation  Indian Western  Space opera  Illnesses & Disabilities  Live action  Sponsored film  Malayalam Cinema  Foreign legion  Stand-up comedy  Plague  Courtroom Comedy  Baseball  Instrumental Music
Count:     2311   1235    774           764       699     584           492     491            442              429               409    409        394         383          294             290      290              283        235      235              228      227          191       174             165           156          151              144              143           132          129      127        115                     110             105                104   102           90      87             85      82            79    77                 76                74         72                 72            72         71       66         66      63               62         55          53        52             49       47                46           44             44        44     42                41             39                  38       37                  36           36             36   36          34          34            32                         31               31                31               30     30                  29        27              26         25                 25                 25         24                24                23               22                 21          21             20             20              20              18               18     18           17                    17               17         17                  16             15    15               15              15                  15       15             14                 14              14                14     13          13        13                 13     13            12                12              12            12                11                   11             11             11             11              11                 10            10            10                10         10      10        10          10             9            9                9                9            9          9            9                   9            9                 9            8               8                    8                8                8       8        8             8          8            8                       8               8            6       6             6                     6                          6                  6              6                    6            6               6                6                 6               5               5                  5         5                 5         5                5            5            5                      5               4                4               4      4              4            4                    4             4            4                            4                                         3          3             3              3          3                 3             3               3                            3                 3                    3       3             3                          3            3           3         2             2                               2        2                        2            2          2                  2                2         2                        2          2              2           2               2          2         2                  2              1            1       1                1         1              1     1       1          1             1          1                         1                      1          1              1                1                   1        1         1          1           1              1            1                 1                     1                                1                           1                  1                     1       1               1                      1         1            1                 1                    1            1           1                         1                         1             1               1            1                         1            1               1                 1               1                1       1                 1         1
---------  -----  ------  ------------  --------  ------  ------------  ------  -------------  ---------------  ----------------  -----  ---------  ----------  -----------  --------------  -------  ---------------  ---------  -------  ---------------  -------  -----------  --------  --------------  ------------  -----------  ---------------  ---------------  ------------  -----------  -------  ---------  ----------------------  --------------  -----------------  ----  ------------  ------  -------------  ------  ------------  ----  -----------------  ----------------  ---------  -----------------  ------------  ---------  -------  ---------  ------  ---------------  ---------  ----------  --------  -------------  -------  ----------------  -----------  -------------  --------  -----  ----------------  -------------  ------------------  -------  ------------------  -----------  -------------  ---  ----------  ----------  ------------  -------------------------  ---------------  ----------------  ---------------  -----  ------------------  --------  --------------  ---------  -----------------  -----------------  ---------  ----------------  ----------------  ---------------  -----------------  ----------  -------------  -------------  --------------  --------------  ---------------  -----  -----------  --------------------  ---------------  ---------  ------------------  -------------  ----  ---------------  --------------  ------------------  -------  -------------  -----------------  --------------  ----------------  -----  ----------  --------  -----------------  -----  ------------  ----------------  --------------  ------------  ----------------  -------------------  -------------  -------------  -------------  --------------  -----------------  ------------  ------------  ----------------  ---------  ------  --------  ----------  -------------  -----------  ---------------  ---------------  -----------  ---------  -----------  ------------------  -----------  ----------------  -----------  --------------  -------------------  ---------------  ---------------  ------  -------  ------------  ---------  -----------  ----------------------  --------------  -----------  ------  ------------  --------------------  -------------------------  -----------------  -------------  -------------------  -----------  --------------  ---------------  ----------------  --------------  --------------  -----------------  --------  ----------------  --------  ---------------  -----------  -----------  ---------------------  --------------  ---------------  --------------  -----  -------------  -----------  -------------------  ------------  -----------  ---------------------------  ----------------------------------------  ---------  ------------  -------------  ---------  ----------------  ------------  --------------  ---------------------------  ----------------  -------------------  ------  ------------  -------------------------  -----------  ----------  --------  ------------  ------------------------------  -------  -----------------------  -----------  ---------  -----------------  ---------------  --------  -----------------------  ---------  -------------  ----------  --------------  ---------  --------  -----------------  -------------  -----------  ------  ---------------  --------  -------------  ----  ------  ---------  ------------  ---------  ------------------------  ---------------------  ---------  -------------  ---------------  ------------------  -------  --------  ---------  ----------  -------------  -----------  ----------------  --------------------  -------------------------------  --------------------------  -----------------  --------------------  ------  --------------  ---------------------  --------  -----------  ----------------  -------------------  -----------  ----------  ------------------------  ------------------------  ------------  --------------  -----------  ------------------------  -----------  --------------  ----------------  --------------  ---------------  ------  ----------------  --------  ------------------

Total number of categories 17547
```

In [ ]:
# Print info for one cluster
cluster_id = 5
neuron_movies_report(cluster_id)

In [ ]:
print("~~ Info for all clusters, sorted, starting from the cluster with the most neurons ~~")
print()

cluster_id_list = list(reversed(ids[count_order].tolist()))
for cid in cluster_id_list:
  print(f"====================== cid = {cid} ==========================")
  neuron_movies_report(cid)
  print()
  print()

```
~~ Info for all clusters, sorted, starting from the cluster with the most neurons ~~

====================== cid = 3 ==========================
Cluster neurons:
[[22, 0], [6, 1], [5, 3], [6, 3], [7, 3], [8, 3], [28, 3], [29, 3], [4, 4], [5, 4], [6, 4], [7, 4], [8, 4], [27, 4], [28, 4], [29, 4], [2, 5], [3, 5], [4, 5], [5, 5], [6, 5], [7, 5], [8, 5], [9, 5], [27, 5], [28, 5], [29, 5], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [6, 6], [7, 6], [8, 6], [9, 6], [29, 6], [0, 7], [1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [7, 7], [8, 7], [9, 7], [10, 7], [0, 8], [1, 8], [2, 8], [3, 8], [4, 8], [5, 8], [6, 8], [7, 8], [8, 8], [9, 8], [10, 8], [11, 8], [0, 9], [1, 9], [2, 9], [3, 9], [4, 9], [5, 9], [6, 9], [7, 9], [8, 9], [9, 9], [10, 9], [11, 9], [0, 10], [1, 10], [2, 10], [3, 10], [4, 10], [5, 10], [6, 10], [7, 10], [8, 10], [9, 10], [10, 10], [11, 10], [0, 11], [1, 11], [2, 11], [3, 11], [4, 11], [5, 11], [6, 11], [7, 11], [8, 11], [9, 11], [10, 11], [11, 11], [0, 12], [1, 12], [2, 12], [3, 12], [4, 12], [5, 12], [6, 12], [7, 12], [8, 12], [9, 12], [10, 12], [11, 12], [0, 13], [1, 13], [2, 13], [3, 13], [4, 13], [5, 13], [6, 13], [7, 13], [8, 13], [10, 13], [11, 13], [0, 14], [1, 14], [2, 14], [3, 14], [4, 14], [5, 14], [11, 14], [12, 14], [0, 15], [1, 15], [2, 15], [3, 15], [0, 16], [1, 16], [21, 16], [0, 17], [22, 17]]

Existing cluster neurons:
[[22, 0], [6, 1], [5, 3], [6, 3], [7, 3], [8, 3], [28, 3], [29, 3], [4, 4], [5, 4], [6, 4], [7, 4], [8, 4], [27, 4], [28, 4], [29, 4], [2, 5], [3, 5], [4, 5], [5, 5], [6, 5], [7, 5], [8, 5], [9, 5], [28, 5], [29, 5], [1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [6, 6], [7, 6], [8, 6], [9, 6], [29, 6], [0, 7], [1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [7, 7], [8, 7], [9, 7], [10, 7], [0, 8], [1, 8], [2, 8], [3, 8], [4, 8], [5, 8], [6, 8], [7, 8], [8, 8], [9, 8], [10, 8], [11, 8], [0, 9], [1, 9], [2, 9], [3, 9], [4, 9], [5, 9], [6, 9], [7, 9], [8, 9], [9, 9], [10, 9], [11, 9], [0, 10], [1, 10], [2, 10], [3, 10], [4, 10], [5, 10], [6, 10], [7, 10], [8, 10], [10, 10], [11, 10], [0, 11], [1, 11], [2, 11], [3, 11], [5, 11], [6, 11], [7, 11], [8, 11], [9, 11], [10, 11], [11, 11], [0, 12], [1, 12], [2, 12], [3, 12], [4, 12], [5, 12], [6, 12], [8, 12], [9, 12], [10, 12], [11, 12], [0, 13], [1, 13], [2, 13], [3, 13], [4, 13], [5, 13], [6, 13], [7, 13], [8, 13], [10, 13], [11, 13], [0, 14], [1, 14], [2, 14], [3, 14], [4, 14], [5, 14], [11, 14], [12, 14], [0, 15], [1, 15], [2, 15], [3, 15], [0, 16], [1, 16], [21, 16], [0, 17], [22, 17]]

Matching movie IDs:
[335, 351, 407, 409, 585, 1934, 2013, 2146, 2256, 2646, 2935, 3243, 3352, 3650, 4251, 4299, 4557, 4816, 3546, 159, 999, 1214, 1363, 2490, 3071, 3124, 3470, 3589, 3899, 4663, 4692, 1330, 1906, 3136, 3548, 3560, 3893, 1043, 1468, 2774, 3527, 4346, 4889, 1716, 3854, 772, 1313, 1568, 4329, 1174, 1255, 2707, 3444, 3844, 4020, 28, 280, 2265, 2373, 3131, 3760, 4294, 750, 793, 1046, 1230, 1429, 1465, 2484, 2684, 3008, 3097, 190, 321, 488, 950, 1412, 2096, 2716, 3197, 3331, 3415, 3451, 4196, 285, 449, 1347, 1787, 3039, 3130, 4141, 1569, 1687, 3884, 4003, 4673, 131, 261, 276, 745, 764, 928, 941, 1383, 2561, 2950, 3168, 3238, 3397, 1156, 1418, 1462, 2370, 2857, 2887, 2939, 3189, 3275, 4212, 68, 185, 780, 912, 998, 1518, 2503, 2961, 3166, 3252, 3309, 3332, 3622, 3872, 4440, 4611, 4880, 19, 915, 1283, 1397, 2909, 3230, 3554, 4171, 1533, 2288, 2485, 3870, 4579, 4703, 4756, 31, 4269, 2692, 2852, 3810, 4536, 4832, 854, 1139, 3204, 125, 1362, 1554, 751, 923, 1367, 1879, 2702, 3078, 3536, 4479, 2014, 2502, 2734, 3758, 3932, 4754, 4848, 448, 825, 932, 1028, 1119, 1780, 2155, 2177, 2384, 2551, 2759, 2889, 3188, 3583, 3833, 3956, 4248, 4556, 972, 2582, 2629, 3469, 4150, 4306, 4334, 4714, 4727, 1000, 2267, 3156, 3242, 4065, 4700, 1148, 1354, 1557, 2272, 3319, 3550, 4902, 1338, 2399, 2657, 3927, 4596, 413, 1062, 1087, 1128, 1131, 1611, 1910, 2838, 3150, 459, 222, 1595, 1927, 1983, 4733, 4914, 1399, 2815, 3112, 4529, 70, 1069, 4087, 4928, 221, 1141, 1545, 3471, 3762, 4926, 162, 357, 515, 661, 931, 1032, 1614, 4729, 4907, 442, 758, 1376, 2157, 2378, 3958, 1085, 1774, 2144, 3196, 3553, 827, 1836, 3631, 4010, 54, 601, 4439, 1095, 2232, 3257, 3449, 3557, 4283, 4710, 104, 1254, 2193, 2247, 2422, 4015, 4406, 4541, 79, 483, 1063, 1236, 1419, 1681, 2568, 4949, 101, 338, 3073, 4146, 154, 742, 2228, 3158, 3709, 3717, 4747, 78, 3617, 3700, 253, 1166, 1219, 1270, 2794, 3816, 782, 1532, 1576, 2480, 3396, 4069, 4397, 4818, 82, 884, 1789, 2003, 2310, 3725, 3901, 1135, 2254, 2345, 2962, 3031, 4258, 4693, 343, 876, 1325, 1655, 2165, 2189, 4173, 376, 382, 1100, 2525, 3177, 3459, 3934, 3977, 4106, 4399, 4550, 89, 1336, 2662, 2675, 1659, 2478, 2706, 4339, 4423, 4576, 1348, 1449, 2035, 3347, 1803, 2135, 2332, 2645, 3514, 801, 994, 1877, 3753, 4648, 578, 2749, 2915, 1360, 1587, 1894, 1941, 4074, 4186, 4671, 168, 815, 851, 1405, 2654, 3218, 4394, 4402, 4602, 235, 1570, 2312, 3111, 3194, 3652, 4039, 182, 888, 1706, 2116, 3248, 3460, 4142, 315, 1319, 4204, 4308, 4347, 1175, 1446, 1471, 1869, 2664, 3333, 4083, 217, 2787, 3845, 4455, 4922, 1335, 1969, 2306, 2943, 3053, 3907, 4321, 792, 1872, 1986, 2161, 2203, 2631, 2638, 3314, 3562, 4395, 4615, 4628, 4976, 810, 868, 1153, 1259, 3348, 3773, 4031, 4167, 509, 2057, 2048, 2407, 3520, 4227, 1203, 2363, 2483, 2545, 3276, 4244, 4909, 4943, 98, 322, 1736, 2121, 2604, 2850, 4094, 4478, 4655, 3, 152, 858, 1243, 1704, 1973, 2184, 2761, 2880, 3422, 3466, 3921, 4454, 920, 2402, 3191, 4166, 4301, 1713, 1911, 2079, 2386, 2726, 2777, 3155, 4232, 4270, 4994, 2432, 2746, 3217, 4191, 4428, 4559, 4885, 1581, 1762, 1976, 2245, 2420, 2461, 2630, 3604, 4057, 4138, 4230, 4863, 622, 900, 1664, 1771, 2476, 2540, 4577, 4586, 2183, 40, 644, 1984, 3547, 4205, 1388, 1498, 1501, 3026, 4249, 63, 649, 1114, 1129, 1240, 1242, 2129, 2237, 2745, 3694, 3708, 3862, 4032, 224, 1953, 2596, 823, 1265, 1915, 2282, 3157, 3647, 4829, 241, 498, 570, 967, 1597, 1671, 1967, 2061, 2708, 2812, 3955, 4256, 4288, 4572, 4369, 4670, 3755, 4991, 798, 1038, 2440, 3098, 3159, 3556, 3685, 3988, 4189, 4614, 12, 1855, 1947, 2133, 2599, 2648, 3173, 3247, 3949, 4878, 23, 735, 1132, 1371, 2769, 3338, 4414, 2953, 4457, 4834, 1059, 1178, 1925, 2636, 2728, 2947, 3848, 3971, 4619, 4970, 1543, 1551, 1842, 2725, 3089, 3264, 3504, 3929, 3974, 861, 1223, 2163, 2303, 2314, 3251, 4403, 4612, 353, 4331, 1754, 2822, 3192, 3877, 1820, 1982, 4279, 4561, 4613, 870, 1168, 1806, 2642, 3401, 3860, 4237, 4812, 996, 1875, 2041, 2219, 2633, 2996, 3232, 3244, 677, 766, 1076, 1756, 2506, 3487, 3732, 3517, 4033, 4645, 215, 441, 593, 1002, 1368, 4266, 4712, 20, 500, 748, 1407, 2074, 3056, 3516, 3720, 4558, 1392, 2127, 2356, 2677, 3501, 1250, 1539, 1579, 2424, 2750, 3856, 4051, 4996, 2912, 4847, 643, 1636, 1709, 1870, 2304, 2429, 3528, 4975, 473, 844, 3283, 3648, 4238, 1764, 2509, 4366, 4828, 4980, 2918, 3561, 2100, 3119, 3588, 296, 97, 692, 1049, 1057, 4064, 4510, 141, 574, 1508, 1700, 1747, 2025, 2217, 2413, 3797, 1016, 1311, 2650, 2933, 3019, 3979, 832, 1757, 1799, 1937, 1980, 3655, 4156, 4698, 4761, 749, 1208, 1693, 3567, 1229, 1933, 2799, 2825, 2930, 3492, 3543, 4532, 3485, 3458, 4291, 599, 1244, 1922, 4415, 4488, 2453, 3643, 148, 246, 2042, 3015, 3145, 4217, 1102, 2255, 2320, 2027, 377, 863, 1211, 1291, 1491, 1546, 2230, 3645, 3765, 4719, 4884, 390, 1015, 1723, 1755, 1928, 2009, 2455, 2923, 3200, 3696, 4093, 4950, 283, 610, 1433, 1561, 2602, 3967, 4292, 4370, 4746, 684, 1187, 1577, 2342, 4491, 4617, 4763, 4843, 480, 1432, 2400, 3434, 4080]

Genres stats for these movies:
---------  -----  ------------  ---------------  ------------  -----  ------------  -------  --------  -----------  -----------  ------------  -----------  ---------------  -----------------  ---------  ----------------  ---------  -------------  ---------  ----------  --------  ------  ------  ----------------  -------  ---------  ------------  -------  ---------------  ---------  ---------------  ------  ---------  --------------  -------------  --------------  ---------------  ---------------  --------------  -------------  -------------  ----------------  --------  -----------------  -----  ----------  -------------------  --------  ------  ----  ---------------  ------------  -------------  -------  -----------  ------------------  --------  ----------------------  ----------------  ---------------  ----------  -------  -------------------------  ------  ---------  -------------------  --------------  ------------------  ------------  ---------------  -----------------  --------------  ----  -------------  -----------------  ---------------  ---------  ------  ---------  --------  -------------  --------------  -----------  ------------  --------  -----------  -------------------  ---------------  --------------  -----------------  ----  ------------------  ----------------  ---  -----------  ------------  ----------  ---------------------  --------  -------  -----------  -----------  ------------  ---------------  --------------  -----  -------------------------  -----------------  -----------  -------------------  ---------------  --------------  ------------  ------------  -----------------  ---------  --------------  -------------  -------------  ----------------  --------  ----------------  --------------  ------  -----------------  ---------------  ------  -----  ----------------
Category:  Drama  World cinema  Black-and-white  Romance Film  Indie  Family Drama  Musical  Thriller  Comedy film  Family Film  Period piece  Silent film  Film adaptation  Biographical film  Biography  Television movie  Melodrama  Coming of age  Bollywood  Short Film  Art film  Comedy  Sports  Biopic [feature]  Fantasy  Film noir  Comedy-drama  Mystery  Political drama  Adventure  Japanese Movies  Action  Docudrama  Chinese Movies  Costume drama  Christian film  Childhood Drama  Filipino Movies  Romantic drama  Ensemble Film  Crime Fiction  Action/Adventure  War film  Experimental film  Music  Road movie  Social problem film  Pre-Code  Horror  Teen  Science Fiction  Black comedy  New Hollywood  History  Avant-garde  Historical fiction  Disaster  Psychological thriller  Historical drama  Courtroom Drama  Children's  Western  Juvenile Delinquency Film  Boxing  Americana  Slice of life story  Crime Thriller  Swashbuckler films  Tamil cinema  Medical fiction  Martial Arts Film  Marriage Drama  Epic  Musical Drama  Children's/Family  Film u00e0 clef  Punk rock  Satire  Animation  Dystopia  Feminist Film  Existentialism  Documentary  Holiday Film  Dogme 95  Hagiography  Inspirational Drama  Addiction Drama  Samurai cinema  Costume Adventure  Cult  Early Black Cinema  Adventure Comedy  Spy  Stop motion  Supernatural  Surrealism  Czechoslovak New Wave  Suspense  Tragedy  Escape Film  Crime Drama  Erotic Drama  Romantic comedy  Hip hop movies  Dance  Heaven-Can-Wait Fantasies  Comedy of manners  Gothic Film  British Empire Film  Domestic Comedy  Bengali Cinema  Mockumentary  Movie serial  Backstage Musical  Education  Musical comedy  Fantasy Drama  Anti-war film  Political cinema  Anti-war  Political satire  Fantasy Comedy  Prison  Fantasy Adventure  Historical Epic  Remake  Anime  Comedy of Errors
Count:     839    115           96               94            78     44            43       42        41           33           33            32           32               28                 27         25                23         22             22         20          19        19      18      17                17       14         14            12       12               12         11               11      10         9               9              9               9                8                8               8              8              8                 8         7                  7      7           6                    6         6       6     6                6             5              5        5            5                   5         5                       5                 5                4           4        4                          4       4          3                    3               3                   3             3                3                  3               3     3              3                  3                2          2       2          2         2              2               2            2             2         2            2                    1                1               1                  1     1                   1                 1    1            1             1           1                      1         1        1            1            1             1                1               1      1                          1                  1            1                    1                1               1             1             1                  1          1               1              1              1                 1         1                 1               1       1                  1                1       1      1
---------  -----  ------------  ---------------  ------------  -----  ------------  -------  --------  -----------  -----------  ------------  -----------  ---------------  -----------------  ---------  ----------------  ---------  -------------  ---------  ----------  --------  ------  ------  ----------------  -------  ---------  ------------  -------  ---------------  ---------  ---------------  ------  ---------  --------------  -------------  --------------  ---------------  ---------------  --------------  -------------  -------------  ----------------  --------  -----------------  -----  ----------  -------------------  --------  ------  ----  ---------------  ------------  -------------  -------  -----------  ------------------  --------  ----------------------  ----------------  ---------------  ----------  -------  -------------------------  ------  ---------  -------------------  --------------  ------------------  ------------  ---------------  -----------------  --------------  ----  -------------  -----------------  ---------------  ---------  ------  ---------  --------  -------------  --------------  -----------  ------------  --------  -----------  -------------------  ---------------  --------------  -----------------  ----  ------------------  ----------------  ---  -----------  ------------  ----------  ---------------------  --------  -------  -----------  -----------  ------------  ---------------  --------------  -----  -------------------------  -----------------  -----------  -------------------  ---------------  --------------  ------------  ------------  -----------------  ---------  --------------  -------------  -------------  ----------------  --------  ----------------  --------------  ------  -----------------  ---------------  ------  -----  ----------------


====================== cid = 18 ==========================
Cluster neurons:
[[13, 9], [14, 9], [15, 9], [16, 9], [17, 9], [18, 9], [19, 9], [20, 9], [12, 10], [13, 10], [14, 10], [15, 10], [16, 10], [17, 10], [18, 10], [19, 10], [20, 10], [21, 10], [12, 11], [13, 11], [14, 11], [15, 11], [16, 11], [17, 11], [18, 11], [19, 11], [20, 11], [21, 11], [12, 12], [13, 12], [14, 12], [15, 12], [16, 12], [17, 12], [18, 12], [19, 12], [20, 12], [13, 13], [14, 13], [15, 13], [16, 13], [17, 13], [18, 13], [19, 13], [20, 13], [13, 14], [14, 14], [15, 14], [16, 14], [17, 14], [18, 14], [19, 14], [20, 14], [13, 15], [14, 15], [15, 15], [16, 15], [17, 15], [18, 15], [19, 15], [20, 15], [13, 16], [14, 16], [15, 16], [16, 16], [17, 16], [18, 16], [19, 16], [13, 17], [14, 17], [15, 17], [16, 17], [17, 17], [21, 24], [22, 24], [21, 25], [22, 25], [22, 26]]

Existing cluster neurons:
[[13, 9], [14, 9], [15, 9], [16, 9], [17, 9], [18, 9], [19, 9], [20, 9], [12, 10], [13, 10], [14, 10], [15, 10], [16, 10], [17, 10], [18, 10], [19, 10], [20, 10], [21, 10], [12, 11], [13, 11], [14, 11], [15, 11], [16, 11], [17, 11], [18, 11], [19, 11], [21, 11], [12, 12], [13, 12], [14, 12], [15, 12], [16, 12], [17, 12], [18, 12], [20, 12], [13, 13], [14, 13], [15, 13], [17, 13], [18, 13], [19, 13], [20, 13], [13, 14], [14, 14], [15, 14], [18, 14], [19, 14], [20, 14], [13, 15], [14, 15], [16, 15], [17, 15], [18, 15], [19, 15], [20, 15], [13, 16], [14, 16], [15, 16], [16, 16], [17, 16], [18, 16], [19, 16], [13, 17], [14, 17], [15, 17], [16, 17], [17, 17], [21, 24], [22, 24], [21, 25], [22, 25], [22, 26]]

Matching movie IDs:
[1108, 2994, 3378, 603, 1678, 2383, 3904, 4487, 4919, 213, 632, 1349, 1531, 1690, 2738, 4063, 4658, 4800, 2409, 3612, 4082, 1185, 2298, 2643, 2676, 2853, 4469, 4849, 841, 178, 1271, 2673, 2804, 3132, 4298, 4515, 4798, 4865, 2235, 2803, 2960, 3661, 3895, 475, 1978, 3133, 59, 80, 786, 1078, 1258, 1487, 2782, 3365, 3889, 3909, 3928, 4657, 4906, 360, 696, 123, 468, 1209, 2103, 2290, 2820, 2856, 3448, 874, 3102, 3202, 3426, 3467, 3982, 4091, 4968, 334, 637, 768, 1735, 3937, 4652, 1117, 1293, 1413, 2587, 3092, 3286, 3603, 4367, 482, 3724, 3827, 2674, 3326, 140, 979, 1679, 3288, 4528, 312, 714, 1066, 4961, 490, 1357, 2886, 2965, 3792, 4208, 4472, 1722, 2868, 4792, 431, 956, 1474, 2158, 3405, 3410, 3599, 3951, 3981, 4169, 4937, 1268, 1574, 2101, 3042, 3595, 102, 398, 2052, 2743, 4241, 4483, 83, 1136, 1382, 1393, 1827, 2102, 2632, 2802, 2929, 3361, 3838, 652, 1097, 4854, 588, 366, 1586, 2829, 165, 1172, 1860, 3740, 4450, 151, 1430, 2839, 4581, 133, 2845, 2427, 2844, 4789, 1740, 3046, 3968, 4236, 712, 1718, 2573, 2819, 3875, 3975, 3984, 4534, 4682, 4725, 4927, 61, 92, 1103, 1821, 2756, 3014, 4081, 4136, 4508, 4543, 4805, 65, 380, 503, 1489, 1601, 1956, 2709, 3174, 3253, 4509, 4744, 135, 427, 493, 1484, 1759, 1782, 1886, 2679, 4257, 4923, 11, 2142, 3049, 4784, 1739, 2159, 3613, 3912, 4381, 4458, 1566, 1791, 2703, 2955, 3450, 3507, 3539, 4743, 4850, 4954, 728, 2329, 2403, 2833, 3139, 3555, 4549, 1966, 986, 3579, 2335, 2153, 2890, 3770, 4449, 513, 1316, 2723, 3908, 4417, 4736, 142, 984, 1448, 1457, 1904, 2317, 2379, 2491, 3307, 4007, 4045, 592, 1320, 1738, 2575, 3636, 4052, 563, 807, 824, 940, 1292, 1600, 2199, 2466, 2649, 3395, 1758, 2514, 3083, 4753, 4974, 541, 1120, 1901, 1907, 4621, 1054, 4371, 1589, 3310, 3917, 1309, 1352, 1657, 2658, 3141, 4006, 4, 872, 1171, 2488, 2593, 4286, 4342, 4915, 4982, 72, 424, 2494, 3638, 4638, 4786, 690, 2821, 3301, 4382, 4582, 4822, 239, 625, 1384, 1558, 1708, 3357, 3505, 3602, 4987, 3482, 10, 1402, 2841, 3715, 4222, 106, 3057, 3819, 4135, 4143, 2978, 4424, 4441, 4887, 3402, 3961, 4955, 853, 1327, 2323, 4803, 181, 1903, 2338, 4640, 49, 1158, 2394, 2776, 2966, 3060, 3432, 3628, 3713, 3857, 3993, 4265, 4861, 2585]

Genres stats for these movies:
---------  -----------  -----------  ------------  ---------------  -------  ------------  ---------  ---------------  -----------  -----  --------------  ---------------  -------------  ------  -------  -------  ------  -----------------  -------  ---------------  ---------  ----------  ---------------  ------  -----  ------  -------------  -----  -----------------  --------------  -----------  ------------  -----  ---------  ----------------  ------  ----------------------  ---------------------------  --------  ---------  ------------------  ----------------  --------  --------  --------  -----  ----------------  -------------  -------------  -------------------------  ----------  -----------------  ---------------  ----------------  ---------------  ------  ----------  ----------  ----------------  ------------------  -----------------  -----------------  -----------  -----------  -------  --------  --------------  --------  ----------------  ------------  ----  ------------  -------------  --------------  ------------  ------------  -----------------  -------  ---------------  --------  -----------  ----------------  --------------------  -----------------  ------------  ---------  -----  ----------  --------------  -----  -------------  -------------  -----------
Category:  Comedy film  Silent film  Romance Film  Black-and-white  Musical  Comedy-drama  Animation  Japanese Movies  Family Film  Indie  Chinese Movies  Romantic comedy  Crime Fiction  Horror  Mystery  Fantasy  Comedy  Children's/Family  Western  Filipino Movies  Film noir  Short Film  Science Fiction  Action  Music  Satire  Costume drama  Drama  Martial Arts Film  Romantic drama  Stop motion  Black comedy  Anime  Adventure  Political satire  Parody  Psychological thriller  Roadshow theatrical release  Suspense  Melodrama  Swashbuckler films  Television movie  Thriller  War film  Pre-Code  Wuxia  Historical drama  Coming of age  Gangster Film  Family-Oriented Adventure  Children's  Culture & Society  Childhood Drama  Screwball comedy  Christmas movie  Sports  Surrealism  Buddy film  Biopic [feature]  Historical fiction  Comedy of manners  Biographical film  Avant-garde  Time travel  Tragedy  Art film  Animal Picture  Whodunit  Workplace Comedy  Tamil cinema  Teen  Concert film  Gender Issues  Fantasy Comedy  Family Drama  Media Satire  Experimental film  Monster  Domestic Comedy  Dogme 95  Documentary  Romantic fantasy  Mythological Fantasy  Detective fiction  Period piece  Detective  Dance  Road movie  Crime Thriller  Crime  Creature Film  New Hollywood  Zombie Film
Count:     88           74           49            49               32       28            27         26               23           17     13              12               10             10      9        9        7       7                  7        6                6          6           5                5       4      4       4              3      3                  3               3            3             3      3          2                 2       2                       2                            2         2          2                   2                 2         2         2         2      2                 2              2              2                          2           2                  2                1                 1                1       1           1           1                 1                   1                  1                  1            1            1        1         1               1         1                 1             1     1             1              1               1             1             1                  1        1                1         1            1                 1                     1                  1             1          1      1           1               1      1              1              1
---------  -----------  -----------  ------------  ---------------  -------  ------------  ---------  ---------------  -----------  -----  --------------  ---------------  -------------  ------  -------  -------  ------  -----------------  -------  ---------------  ---------  ----------  ---------------  ------  -----  ------  -------------  -----  -----------------  --------------  -----------  ------------  -----  ---------  ----------------  ------  ----------------------  ---------------------------  --------  ---------  ------------------  ----------------  --------  --------  --------  -----  ----------------  -------------  -------------  -------------------------  ----------  -----------------  ---------------  ----------------  ---------------  ------  ----------  ----------  ----------------  ------------------  -----------------  -----------------  -----------  -----------  -------  --------  --------------  --------  ----------------  ------------  ----  ------------  -------------  --------------  ------------  ------------  -----------------  -------  ---------------  --------  -----------  ----------------  --------------------  -----------------  ------------  ---------  -----  ----------  --------------  -----  -------------  -------------  -----------


====================== cid = 9 ==========================
Cluster neurons:
[[25, 10], [24, 11], [25, 11], [21, 12], [22, 12], [23, 12], [24, 12], [25, 12], [21, 13], [22, 13], [23, 13], [24, 13], [25, 13], [26, 13], [21, 14], [22, 14], [23, 14], [24, 14], [25, 14], [26, 14], [27, 14], [28, 14], [29, 14], [21, 15], [22, 15], [23, 15], [24, 15], [25, 15], [26, 15], [27, 15], [28, 15], [29, 15], [22, 16], [23, 16], [24, 16], [25, 16], [26, 16], [27, 16], [28, 16], [29, 16], [23, 17], [24, 17], [25, 17], [26, 17], [27, 17], [28, 17], [29, 17], [24, 18], [25, 18], [26, 18], [27, 18], [28, 18], [29, 18], [24, 19], [25, 19], [26, 19], [27, 19], [28, 19], [29, 19], [26, 20], [27, 20], [28, 20], [29, 20], [27, 21], [28, 21], [29, 21]]

Existing cluster neurons:
[[25, 10], [24, 11], [21, 12], [22, 12], [23, 12], [24, 12], [25, 12], [21, 13], [22, 13], [23, 13], [24, 13], [25, 13], [26, 13], [21, 14], [22, 14], [23, 14], [24, 14], [25, 14], [27, 14], [28, 14], [29, 14], [22, 15], [23, 15], [24, 15], [25, 15], [26, 15], [27, 15], [28, 15], [29, 15], [22, 16], [23, 16], [24, 16], [25, 16], [26, 16], [27, 16], [28, 16], [29, 16], [23, 17], [24, 17], [25, 17], [26, 17], [27, 17], [28, 17], [29, 17], [24, 18], [25, 18], [26, 18], [27, 18], [28, 18], [29, 18], [24, 19], [25, 19], [26, 19], [27, 19], [28, 19], [29, 19], [26, 20], [27, 20], [29, 20], [27, 21], [28, 21], [29, 21]]

Matching movie IDs:
[717, 1273, 1746, 2554, 3236, 3421, 3473, 4705, 4830, 301, 891, 1632, 1841, 957, 4941, 2647, 2412, 2719, 3511, 4497, 233, 512, 527, 877, 1138, 1321, 1431, 1887, 2169, 2851, 3785, 3980, 96, 1714, 2546, 2613, 3983, 4330, 4895, 961, 2401, 3081, 3615, 1410, 1519, 3256, 3727, 3881, 1152, 1213, 4161, 4262, 4272, 73, 197, 700, 1082, 3681, 3902, 3964, 4978, 2152, 3480, 1161, 2221, 3950, 4548, 2024, 2700, 3900, 4461, 1092, 1096, 2044, 2963, 3475, 4735, 1675, 2711, 3409, 3678, 4126, 4385, 4629, 821, 929, 1118, 1499, 2313, 2659, 2771, 2956, 2964, 3600, 3756, 3798, 4318, 4836, 35, 257, 671, 970, 1003, 2381, 2623, 2860, 2977, 3431, 3805, 3941, 4050, 4229, 4500, 4773, 4960, 421, 688, 1996, 3274, 4118, 4314, 4516, 58, 1834, 1896, 2143, 3676, 4475, 4713, 1830, 2194, 3428, 4014, 232, 412, 3064, 770, 865, 2210, 4036, 1060, 2123, 2211, 2555, 2556, 2791, 3737, 4188, 4730, 4872, 903, 2385, 2848, 4247, 4903, 143, 1262, 2037, 4133, 438, 2442, 3167, 505, 551, 2056, 2318, 2584, 3095, 3930, 4104, 520, 716, 726, 741, 951, 1797, 1962, 2498, 3208, 3799, 4361, 4555, 183, 600, 2824, 3359, 4159, 4284, 4328, 30, 2597, 542, 2987, 4332, 4431, 4589, 429, 504, 2504, 3704, 3822, 4666, 93, 108, 981, 1150, 1968, 2778, 2970, 3128, 3549, 4078, 256, 646, 3321, 3416, 4845, 561, 1264, 1365, 1381, 1538, 1948, 2571, 2899, 3161, 3840, 3957, 4122, 4358, 4718, 2768, 3278, 3695, 77, 978, 1426, 1741, 2275, 2637, 3075, 4724, 4846, 4972, 1889, 2319, 2376, 4317, 4860, 4929, 4992, 188, 584, 1391, 2368, 1535, 1848, 2628, 2718, 3736, 4022, 4396, 4817, 739, 158, 494, 1169, 1802, 2059, 2415, 3260, 3782, 4200, 4683, 171, 1253, 4959, 327, 711, 1267, 3292, 3620, 4105, 4609, 627, 1109, 2019, 2589, 3571, 4390, 18, 288, 1777, 2680, 3079, 3436, 3439, 3594, 3933, 4013, 4311, 647, 676, 4419, 4436, 4837, 4916, 329, 2001, 2109, 2430, 2846, 3538, 4243, 4882, 367, 2276, 4908, 4988, 309, 1400, 1421, 1441, 1672, 2238, 2672, 2727, 3667, 558, 565, 691, 1724, 1897, 2500, 2816, 4338, 651, 943, 1871, 2463, 116, 127, 3407, 3842, 1977, 2180, 3327, 4401, 120, 1127, 1506, 2192, 2239, 2609, 4482, 1232, 1527, 2188, 2984, 3206, 3380, 3677, 4246, 4739, 682, 897, 925, 1237, 1334, 1769, 1952, 2493, 3076, 3215, 4072, 4109, 4327, 4447, 4674, 4810, 2337, 3366, 3377, 3769, 548, 2902, 3821, 4336, 4751, 95, 1902, 2065, 2910, 3011, 4418, 4567, 4997]

Genres stats for these movies:
---------  ------  ------  ---------------  ------------  ---------  -----------  -------  ------  -----  -------------  ------  ------------  -------  -----  ----------------  ---------------  ------------  ----------  ----------  -------  ------------  ---------  ---------------  -----------------  --------------  -----------------  ----------------  ----  ----------------  ------  ---------------  ----------  --------  ----------  ---------  ----  ---------------  ----------------  ------------  ---------------  ------------  -------  -------------  ----------  ------------  -----  ----------------  --------  ---------  --------------  --------------  -------------  ---------  ---------  --------------  --------------  ---------------  --------------  ------------  -----------------  ---------------  --------------  -----  ------  -----------  ------  ------------------  -----------  -------------------  ---------------  ---  -----------  ----------------  -----  -------------  --------  ----------------  -----------  ----------  --------  -------------  ---------------  --------------  -------  ----------  --------  -------------  -----------  ----------------  --------------  ----------------  ---------  --------------  -------------  ---------------  -------------  --------  ----------------  -----  -----------  ------  ---------------------  -------------  ----------  ----------------  -------------  ------------  ------  -------------------------  -----------  --------------  ---------  -------
Category:  Comedy  Parody  Black-and-white  Black comedy  Slapstick  Family Film  Musical  Satire  Indie  Crime Fiction  Sports  Crime Comedy  Fantasy  Drama  Screwball comedy  Romantic comedy  Comedy-drama  Children's  Buddy film  Mystery  World cinema  Adventure  Science Fiction  Children's/Family  Fantasy Comedy  Comedy of manners  Comedy of Errors  Cult  Political cinema  Horror  Domestic Comedy  Short Film  Thriller  Sex comedy  Absurdism  Teen  Japanese Movies  Political satire  Mockumentary  Christmas movie  Holiday Film  Western  Ensemble Film  Road movie  Romance Film  Heist  Television movie  War film  Gross out  Gross-out film  Comedy Western  Coming of age  Detective  Animation  Animal Picture  Chinese Movies  Comedy Thriller  Hip hop movies  Media Satire  Detective fiction  Film adaptation  Musical comedy  Music  Action  Caper story  Remake  Children's Fantasy  Comedy film  Slice of life story  Filipino Movies  Spy  Stoner film  Workplace Comedy  Dance  Horror Comedy  Whodunit  Adventure Comedy  Time travel  Beach Film  Baseball  Action Comedy  Stand-up comedy  Bengali Cinema  Slasher  Alien Film  Suspense  Social issues  Silent film  Action/Adventure  Religious Film  Romantic fantasy  Film noir  Foreign legion  Gangster Film  Ealing Comedies  Doomsday film  Disaster  Historical drama  Crime  Jungle Film  Boxing  Master Criminal Films  Creature Film  Mondo film  Courtroom Comedy  New Hollywood  Period piece  Prison  Family-Oriented Adventure  Chase Movie  Romantic drama  Melodrama  Z movie
Count:     395     44      42               39            33         32           30       28      27     26             23      19            18       17     17                16               14            13          12          12       12            12         10               10                 10              9                  9                 9     8                 8       8                8           8         8           7          7     7                7                 6             6                6             6        5              4           4             4      4                 4         4          4               4               4              4          4          4               4               3                3               3             3                  3                3               3      3       3            2       2                   2            2                    2                2    2            2                 2      2              2         2                 2            1           1         1              1                1               1        1           1         1              1            1                 1               1                 1          1               1              1                1              1         1                 1      1            1       1                      1              1           1                 1              1             1       1                          1            1               1          1
---------  ------  ------  ---------------  ------------  ---------  -----------  -------  ------  -----  -------------  ------  ------------  -------  -----  ----------------  ---------------  ------------  ----------  ----------  -------  ------------  ---------  ---------------  -----------------  --------------  -----------------  ----------------  ----  ----------------  ------  ---------------  ----------  --------  ----------  ---------  ----  ---------------  ----------------  ------------  ---------------  ------------  -------  -------------  ----------  ------------  -----  ----------------  --------  ---------  --------------  --------------  -------------  ---------  ---------  --------------  --------------  ---------------  --------------  ------------  -----------------  ---------------  --------------  -----  ------  -----------  ------  ------------------  -----------  -------------------  ---------------  ---  -----------  ----------------  -----  -------------  --------  ----------------  -----------  ----------  --------  -------------  ---------------  --------------  -------  ----------  --------  -------------  -----------  ----------------  --------------  ----------------  ---------  --------------  -------------  ---------------  -------------  --------  ----------------  -----  -----------  ------  ---------------------  -------------  ----------  ----------------  -------------  ------------  ------  -------------------------  -----------  --------------  ---------  -------


====================== cid = 2 ==========================
Cluster neurons:
[[13, 18], [14, 18], [15, 18], [12, 19], [13, 19], [14, 19], [15, 19], [12, 20], [13, 20], [14, 20], [15, 20], [16, 20], [12, 21], [13, 21], [14, 21], [15, 21], [16, 21], [12, 22], [13, 22], [14, 22], [15, 22], [16, 22], [13, 23], [14, 23], [15, 23], [16, 23], [12, 24], [13, 24], [14, 24], [15, 24], [16, 24], [17, 24], [12, 25], [13, 25], [14, 25], [15, 25], [16, 25], [13, 26], [14, 26], [15, 26], [13, 27], [14, 27], [15, 27], [13, 28], [14, 28], [15, 28], [16, 28], [13, 29], [14, 29], [15, 29], [16, 29]]

Existing cluster neurons:
[[13, 18], [14, 18], [15, 18], [12, 19], [13, 19], [14, 19], [15, 19], [12, 20], [13, 20], [14, 20], [15, 20], [16, 20], [12, 21], [13, 21], [14, 21], [15, 21], [16, 21], [12, 22], [13, 22], [14, 22], [16, 22], [13, 23], [14, 23], [15, 23], [16, 23], [12, 24], [13, 24], [14, 24], [15, 24], [16, 24], [17, 24], [12, 25], [13, 25], [15, 25], [16, 25], [13, 26], [14, 26], [15, 26], [13, 27], [14, 27], [15, 27], [13, 28], [14, 28], [15, 28], [16, 28], [13, 29], [14, 29], [15, 29], [16, 29]]

Matching movie IDs:
[2932, 4728, 5, 184, 698, 1467, 1811, 2524, 2704, 525, 1345, 2528, 2897, 4221, 3923, 4148, 4172, 1840, 2224, 2731, 3818, 4777, 4883, 540, 1145, 2938, 3739, 1874, 3245, 3705, 4295, 4446, 4522, 4873, 4999, 672, 945, 1298, 1647, 2208, 3032, 3067, 4699, 234, 899, 1241, 2382, 2544, 3297, 3702, 4467, 145, 506, 1318, 1490, 1898, 2067, 2495, 3544, 436, 451, 590, 1742, 2072, 2473, 2487, 1598, 537, 1510, 2982, 4030, 4393, 3171, 3573, 4274, 2578, 3766, 269, 529, 594, 3287, 3289, 4430, 199, 425, 1370, 1415, 1788, 1954, 2696, 2878, 3106, 445, 707, 1110, 3456, 3640, 281, 2661, 3375, 150, 274, 1199, 1387, 3214, 4526, 803, 2529, 126, 1294, 1398, 2580, 1567, 2062, 2854, 3101, 3997, 4839, 911, 1505, 2808, 3996, 4341, 4967, 0, 1157, 562, 1350, 1621, 1663, 1665, 4481, 363, 613, 849, 1039, 1694, 2012, 2185, 2579, 2784, 3662, 3801, 3815, 3994, 4864, 34, 191, 243, 867, 1149, 2431, 2884, 4158, 88, 342, 598, 1622, 2186, 2842, 3270, 2408, 1390, 577, 704, 1177, 1607, 1888, 1970, 2046, 3296, 3880, 4354, 4606, 4871, 2874, 3462, 86, 4492, 708, 2452, 4121, 1257, 2397, 2619, 4952, 1206, 2043, 2130, 2695, 3534, 4466, 518, 3134, 4111, 522, 767, 826, 1029, 4660, 1261, 1882, 3203, 3531, 1790, 4775, 4868, 1578, 2594, 3830, 1061, 1493, 1931, 2289, 2301, 2879, 3175, 3385, 4506, 3497, 1056, 1246, 1641, 67, 289, 291, 305, 447, 657, 689, 985, 1310, 1593, 1753, 1809, 2322, 2366, 3068, 3823, 3835, 4876, 4921, 90, 99, 829, 1658, 1908, 2006, 2465, 2627, 3750, 3989, 4096, 4125, 4451, 4624, 4905, 325, 715]

Genres stats for these movies:
---------  ------  -------  ----------------  ---------  -----  ---------------  ------  ------------  --------  ---  -----------  ------------  -------------  ----------  ---------------  -------  ------------------  ------------------  -----------------  -----  -----------  -------------------  ------  -----------------  ---------  --------------  ----------------  ----------------  -------------  ------  -----------  -----------------  --------  ---------  -------  -------  ----------------  ------------  ---------------  ---------  -------------  ----------------  ---------  ------------  ------  --------------  -------  ----------  ------------  ---------  -----------  -----------  ---------------  --------------  ---------  ---------  ---------  -----------------  --------  -----------  -----  ------------  ------  ----------  ---------------  ----------------  ---------------  ------------  ----------------------  -----------------  ----------  -------------------------  ----  ------------  -------------  -------------  ---------------  -----------------  ------------  ------------------  --------------  --------------  --------  -----  -----------------  --------------------  -------------  --------  ------------  --------------  -----
Category:  Action  Western  Action/Adventure  Adventure  Drama  Black-and-white  Comedy  Romance Film  War film  Spy  Family Film  World cinema  Crime Fiction  Short Film  Superhero movie  Fantasy  Swashbuckler films  Historical fiction  Costume Adventure  Indie  Comedy film  Revisionist Western  Sports  Spaghetti Western  Bollywood  Chinese Movies  Adventure Comedy  Biopic [feature]  Costume drama  Horror  Jungle Film  Martial Arts Film  Thriller  Animation  Musical  History  Television movie  Period piece  Film adaptation  Film noir  Action Comedy  Action Thrillers  B-Western  Epic Western  Parody  Romantic drama  Mystery  Buddy film  Movie serial  Melodrama  Silent film  Chase Movie  Japanese Movies  Blaxploitation  Biography  Superhero  Detective  Sword and sorcery  Disaster  Auto racing  Anime  Comedy-drama  Boxing  Biker Film  Science Fiction  Romantic fantasy  Romantic comedy  Cavalry Film  Psychological thriller  Outlaw biker film  Children's  Family-Oriented Adventure  Epic  Feature film  Ensemble Film  Gangster Film  Historical Epic  Detective fiction  Crime Comedy  Political thriller  Hybrid Western  Marriage Drama  Cold War  Music  Children's/Family  Natural horror films  New Hollywood  Northern  Family Drama  Comedy Western  Wuxia
Count:     202     86       85                81         74     32               25      24            12        12   11           9             9              7           7                6        5                   5                   5                  5      5            5                    5       5                  5          4               4                 4                 4              4       4            4                  4         4          3        3        3                 3             3                3          3              3                 3          2             2       2               2        2           2             2          2            2            2                2               2          2          2          2                  2         2            2      2             1       1           1                1                 1                1             1                       1                  1           1                          1     1             1              1              1                1                  1             1                   1               1               1         1      1                  1                     1              1         1             1               1
---------  ------  -------  ----------------  ---------  -----  ---------------  ------  ------------  --------  ---  -----------  ------------  -------------  ----------  ---------------  -------  ------------------  ------------------  -----------------  -----  -----------  -------------------  ------  -----------------  ---------  --------------  ----------------  ----------------  -------------  ------  -----------  -----------------  --------  ---------  -------  -------  ----------------  ------------  ---------------  ---------  -------------  ----------------  ---------  ------------  ------  --------------  -------  ----------  ------------  ---------  -----------  -----------  ---------------  --------------  ---------  ---------  ---------  -----------------  --------  -----------  -----  ------------  ------  ----------  ---------------  ----------------  ---------------  ------------  ----------------------  -----------------  ----------  -------------------------  ----  ------------  -------------  -------------  ---------------  -----------------  ------------  ------------------  --------------  --------------  --------  -----  -----------------  --------------------  -------------  --------  ------------  --------------  -----


====================== cid = 16 ==========================
Cluster neurons:
[[12, 3], [13, 3], [21, 3], [9, 4], [10, 4], [11, 4], [12, 4], [13, 4], [14, 4], [15, 4], [16, 4], [20, 4], [21, 4], [10, 5], [11, 5], [12, 5], [13, 5], [14, 5], [15, 5], [16, 5], [17, 5], [20, 5], [10, 6], [11, 6], [12, 6], [13, 6], [14, 6], [15, 6], [16, 6], [17, 6], [18, 6], [19, 6], [11, 7], [12, 7], [13, 7], [14, 7], [15, 7], [16, 7], [17, 7], [18, 7], [19, 7], [12, 8], [13, 8], [14, 8], [15, 8], [16, 8], [17, 8], [18, 8], [19, 8], [12, 9]]

Existing cluster neurons:
[[12, 3], [13, 3], [21, 3], [10, 4], [11, 4], [12, 4], [13, 4], [14, 4], [15, 4], [16, 4], [20, 4], [21, 4], [10, 5], [11, 5], [12, 5], [13, 5], [14, 5], [15, 5], [16, 5], [17, 5], [20, 5], [10, 6], [11, 6], [12, 6], [13, 6], [14, 6], [15, 6], [16, 6], [17, 6], [18, 6], [19, 6], [11, 7], [12, 7], [13, 7], [14, 7], [15, 7], [16, 7], [17, 7], [18, 7], [19, 7], [12, 8], [13, 8], [14, 8], [15, 8], [16, 8], [17, 8], [19, 8], [12, 9]]

Matching movie IDs:
[231, 2489, 2715, 75, 736, 939, 968, 4890, 670, 776, 1083, 2085, 9, 1478, 2023, 3618, 402, 524, 645, 292, 1070, 2390, 845, 1606, 1866, 3121, 3794, 1772, 2607, 2798, 4174, 4422, 4517, 812, 1523, 2028, 2714, 3522, 153, 259, 552, 719, 2713, 3351, 4067, 4261, 547, 1140, 1627, 1667, 2076, 3261, 4794, 1481, 1571, 1588, 2227, 2742, 2873, 3502, 3656, 4110, 4115, 113, 2162, 3358, 2425, 2936, 3336, 3545, 4981, 1024, 2550, 2724, 4438, 48, 1975, 2281, 3433, 3761, 3858, 4377, 3400, 4090, 124, 1837, 2212, 3660, 971, 3598, 1649, 2983, 45, 156, 469, 2482, 4313, 249, 2911, 3474, 4124, 4716, 294, 2496, 3183, 3788, 4654, 2324, 3491, 3654, 50, 694, 579, 1279, 1372, 2827, 3671, 4512, 2284, 2722, 3149, 1074, 3644, 4049, 4413, 514, 2606, 3832, 4114, 4273, 4717, 3679, 4463, 14, 568, 2084, 2099, 3255, 3789, 4687, 1146, 2448, 4373, 94, 240, 1472, 4282, 608, 2393, 3764, 4519, 262, 2167, 3234, 4224, 612, 1629, 2214, 226, 753, 1452, 2688, 734, 804, 2369, 2858, 1661, 2729, 2906, 2419, 4507, 55, 196, 287, 1051, 4866, 2418, 3390, 3686, 4599, 1133, 1375, 1620, 1695, 2032, 2105, 3440, 4132, 284, 2333, 4587, 1280, 2299, 2895, 3066, 3558, 2828]

Genres stats for these movies:
---------  -----  ------  ------------  --------  ---------------  ------  -------------  ------  ------------  ---------  ---------------  -------  -----------  -----------  -------  -------  ----------------  ----------------------  -----------  --------  -------------  --------------  ---------  -------  ------------  -----------------  --------------  ---------------  ---------  ------  -----  ----  ---------------  ---------------  ----------  ------------  -----------------  ----  ------------------  -----------------  --------------  ---------------  ---------  -----  ------------------  ------------  ---  ---------  ----------------  ----------  ------  ------------------  --------------  --------------  -----  -----------  -----  --------------  --------------  ------  -------------  ------------  ----------------  ---------  -----------  -------------  -------  -------------  -------------  ---------  --------------  --------  ------------  --------  ----  ----------------  ----------------  ------  --------------  ----------  ------  ------------  -------------  ----------------  ------------  ---------------  ---------  -------  -------------  -----  ---------------  -------------  --------------  ---------------  --------------  ----------  ---------------  ------------------  --------  ---------------  ---------------  --------------  ------------  -----------
Category:  Drama  Comedy  Romance Film  Thriller  Japanese Movies  Action  Crime Fiction  Horror  World cinema  Adventure  Black-and-white  Mystery  Comedy film  Family Film  Fantasy  Western  Action/Adventure  Psychological thriller  Silent film  War film  Coming of age  Crime Thriller  Film noir  Musical  Period piece  Children's/Family  Romantic drama  Science Fiction  Biography  Sports  Indie  Epic  Romantic comedy  Film adaptation  Short Film  Tamil cinema  Martial Arts Film  Teen  Historical fiction  Biographical film  Chinese Movies  Political drama  Animation  Anime  Swashbuckler films  Supernatural  Spy  Slapstick  Biopic [feature]  Sex comedy  Parody  Children's Fantasy  Marriage Drama  Comedy Western  Wuxia  Crime Drama  Crime  Indian Western  Samurai cinema  Satire  Sci-Fi Horror  Black comedy  Screwball comedy  Gross out  Future noir  Creature Film  Slasher  Anti-war film  Feminist Film  Steampunk  Fantasy Comedy  Suspense  Crime Comedy  Anti-war  Cult  Adventure Comedy  Action Thrillers  Boxing  Hybrid Western  Buddy film  Remake  Comedy-drama  Costume drama  Malayalam Cinema  Holiday Film  Christmas movie  Melodrama  Monster  Monster movie  Music  Courtroom Drama  Musical Drama  Musical comedy  Historical Epic  Gross-out film  Children's  Domestic Comedy  Political thriller  Pre-Code  Propaganda film  Childhood Drama  Religious Film  Cavalry Film  Documentary
Count:     98     36      34            21        19               19      14             14      13            12         12               11       8            8            8        8        7                 5                       5            5         4              4               4          4        4             4                  4               4                4          4       3      3     3                3                3           3             3                  3     3                   3                  3               2                2          2      2                   2             2    2          2                 2           2       2                   2               2               2      2            2      1               1               1       1              1             1                 1          1            1              1        1              1              1          1               1         1             1         1     1                 1                 1       1               1           1       1             1              1                 1             1                1          1        1              1      1                1              1               1                1               1           1                1                   1         1                1                1               1             1
---------  -----  ------  ------------  --------  ---------------  ------  -------------  ------  ------------  ---------  ---------------  -------  -----------  -----------  -------  -------  ----------------  ----------------------  -----------  --------  -------------  --------------  ---------  -------  ------------  -----------------  --------------  ---------------  ---------  ------  -----  ----  ---------------  ---------------  ----------  ------------  -----------------  ----  ------------------  -----------------  --------------  ---------------  ---------  -----  ------------------  ------------  ---  ---------  ----------------  ----------  ------  ------------------  --------------  --------------  -----  -----------  -----  --------------  --------------  ------  -------------  ------------  ----------------  ---------  -----------  -------------  -------  -------------  -------------  ---------  --------------  --------  ------------  --------  ----  ----------------  ----------------  ------  --------------  ----------  ------  ------------  -------------  ----------------  ------------  ---------------  ---------  -------  -------------  -----  ---------------  -------------  --------------  ---------------  --------------  ----------  ---------------  ------------------  --------  ---------------  ---------------  --------------  ------------  -----------


====================== cid = 14 ==========================
Cluster neurons:
[[9, 20], [8, 21], [9, 21], [10, 21], [7, 22], [8, 22], [9, 22], [10, 22], [7, 23], [8, 23], [9, 23], [10, 23], [11, 23], [12, 23], [7, 24], [8, 24], [9, 24], [10, 24], [11, 24], [7, 25], [8, 25], [9, 25], [10, 25], [11, 25], [7, 26], [8, 26], [9, 26], [10, 26], [11, 26], [12, 26], [7, 27], [8, 27], [9, 27], [10, 27], [11, 27], [12, 27], [7, 28], [8, 28], [9, 28], [10, 28], [11, 28], [12, 28], [8, 29], [9, 29], [10, 29], [11, 29]]

Existing cluster neurons:
[[9, 20], [8, 21], [9, 21], [10, 21], [8, 22], [9, 22], [10, 22], [7, 23], [8, 23], [9, 23], [10, 23], [11, 23], [12, 23], [7, 24], [8, 24], [9, 24], [10, 24], [11, 24], [7, 25], [8, 25], [9, 25], [10, 25], [11, 25], [7, 26], [8, 26], [9, 26], [10, 26], [11, 26], [12, 26], [7, 27], [8, 27], [9, 27], [10, 27], [11, 27], [12, 27], [7, 28], [8, 28], [9, 28], [10, 28], [11, 28], [12, 28], [8, 29], [9, 29], [10, 29], [11, 29]]

Matching movie IDs:
[2904, 3024, 33, 236, 2598, 4322, 4776, 484, 754, 1064, 1748, 2773, 3138, 3298, 3935, 4253, 4622, 550, 2752, 3178, 3593, 3659, 4477, 4722, 307, 2147, 2660, 2811, 3250, 4144, 4948, 3254, 3457, 3624, 4043, 4918, 2343, 2346, 4398, 384, 1337, 1862, 2266, 2864, 2971, 3759, 411, 778, 922, 1950, 3201, 3883, 347, 3425, 4562, 416, 1041, 1844, 2151, 2331, 3777, 4400, 2016, 2469, 2807, 3525, 3664, 4021, 4471, 4706, 4827, 2083, 2202, 2533, 3148, 3861, 1560, 1697, 2092, 2954, 3873, 4533, 1773, 4216, 2229, 3322, 3625, 4168, 4525, 4787, 66, 3388, 1332, 1455, 2941, 3109, 3172, 3535, 4474, 205, 252, 275, 781, 818, 1080, 1732, 2309, 3952, 4665, 4676, 1761, 2259, 521, 530, 556, 557, 1098, 2477, 2639, 4456, 4584, 3055, 3486, 3747, 4771, 187, 458, 3515, 76, 1944, 1958, 4726, 930, 2047, 3374, 3606, 110, 546, 1075, 1116, 1625, 1813, 4178, 460, 1689, 2030, 2066, 2744, 2872, 3114, 713, 2321, 3213, 3419, 3802, 4938, 2462, 4307, 214, 948, 1165, 1234, 1528, 1673, 2459, 3423, 423, 2625, 2951, 4659, 4709, 621, 2260, 3117, 4793, 1373, 4376, 39, 673, 1726, 3304, 175, 245, 1562, 1861, 3152, 4605, 4738, 4934, 642, 3344, 1488, 3105, 3320, 3707, 4151, 1328, 2421, 2456, 313, 467, 1930, 2534, 4966, 3744, 2564, 3329, 238, 784, 910, 2069, 2942, 3592, 3876, 4038, 755, 1182, 1193, 2358, 2683, 3146, 3465, 3973, 4181, 26, 211, 882, 1683, 2843, 3000, 4025, 4352, 1288, 1981, 3476, 4071]

Genres stats for these movies:
---------  ------  ----------------  --------  -------------  -----  ------  --------------  ---------------  ----------------  -----------------  ------------  ---------  ------  -----  ---------------  -------  ---------------  ---------  -------  --------  -------------  -------------  -----  ------------  ------  ---  ---------------  ------------  --------  ------------------  --------------------  --------  --------------  ----------  ----  ----------------------  ----------------------  ------  ------------  ---------  ---------  --------------  ---------  -----------  -----------  ------------  -----  -----------  ----------------  ------  -----------  -------  ---------------------  -------  ------------  ----------------  -----------  -------------  -------------------  ----  ---------------  ---------------  ----------------  ------------  -------------  -----------  ---------  -----------------  ----------  -----------------------  ----------  --------  ---------------  -------  ---------------  --------------  ---------------  --------------  --------------  -----------  -----------  ---------  -------------  -------------------------  -----------  ---------  ------------  ---------------  ---------  ---------  --------  --------------------  -------------  ----------------  -------------  --------------  --------------  ------------------  ------  -----------  -----------------  -------------------  --------------------------  -------------  ----  -------
Category:  Action  Action/Adventure  Thriller  Crime Fiction  Drama  Comedy  Crime Thriller  Science Fiction  Action Thrillers  Martial Arts Film  World cinema  Adventure  Horror  Indie  Superhero movie  Fantasy  Japanese Movies  Superhero  Mystery  War film  Gangster Film  Action Comedy  Heist  Black comedy  Sports  Spy  Film adaptation  Crime Comedy  Disaster  Political thriller  Natural horror films  Suspense  Chinese Movies  Buddy film  Teen  Airplanes and airports  Psychological thriller  Parody  Supernatural  Slapstick  Animation  Blaxploitation  Bollywood  Family Film  Zombie Film  Romance Film  Anime  Crime Drama  Television movie  Satire  Auto racing  Musical  Master Criminal Films  Western  Period piece  Screwball comedy  Chase Movie  Creature Film  Glamorized Spy Film  Cult  Romantic comedy  Comedy Thriller  Comedy of Errors  Comedy-drama  Coming of age  Caper story  Detective  Detective fiction  Short Film  Science fiction Western  Road movie  Dystopia  Political drama  B-movie  Black-and-white  Hybrid Western  Filipino Movies  Existentialism  Comedy Western  Comedy film  Gothic Film  Biography  Costume drama  Juvenile Delinquency Film  Future noir  Steampunk  Mockumentary  Film u00e0 clef  Film noir  Docudrama  Neo-noir  Archives and records  Sci-Fi Horror  Sci-Fi Adventure  Doomsday film  Samurai cinema  Romantic drama  Historical fiction  Boxing  Road-Horror  Outlaw biker film  Revisionist Western  Parkour in popular culture  Ensemble Film  Epic  History
Count:     234     193               188       88             57     51      41              37               35                31                 26            24         23      19     18               16       14               13         10       9         9              9              8      8             8       8    8                8             7         7                   7                     7         7               6           6     6                       6                       6       5             5          5          4               4          4            4            4             4      4            4                 3       3            3        3                      3        3             3                 3            3              3                    3     2                2                2                 2             2              2            2          2                  2           2                        2           2         2                2        2                2               2                1               1               1            1            1          1              1                          1            1          1             1                1          1          1         1                     1              1                 1              1               1               1                   1       1            1                  1                    1                           1              1     1
---------  ------  ----------------  --------  -------------  -----  ------  --------------  ---------------  ----------------  -----------------  ------------  ---------  ------  -----  ---------------  -------  ---------------  ---------  -------  --------  -------------  -------------  -----  ------------  ------  ---  ---------------  ------------  --------  ------------------  --------------------  --------  --------------  ----------  ----  ----------------------  ----------------------  ------  ------------  ---------  ---------  --------------  ---------  -----------  -----------  ------------  -----  -----------  ----------------  ------  -----------  -------  ---------------------  -------  ------------  ----------------  -----------  -------------  -------------------  ----  ---------------  ---------------  ----------------  ------------  -------------  -----------  ---------  -----------------  ----------  -----------------------  ----------  --------  ---------------  -------  ---------------  --------------  ---------------  --------------  --------------  -----------  -----------  ---------  -------------  -------------------------  -----------  ---------  ------------  ---------------  ---------  ---------  --------  --------------------  -------------  ----------------  -------------  --------------  --------------  ------------------  ------  -----------  -----------------  -------------------  --------------------------  -------------  ----  -------


====================== cid = 5 ==========================
Cluster neurons:
[[9, 13], [6, 14], [7, 14], [8, 14], [9, 14], [10, 14], [5, 15], [6, 15], [7, 15], [8, 15], [9, 15], [10, 15], [11, 15], [12, 15], [6, 16], [7, 16], [8, 16], [9, 16], [10, 16], [11, 16], [12, 16], [7, 17], [8, 17], [9, 17], [10, 17], [11, 17], [12, 17], [7, 18], [8, 18], [9, 18], [10, 18], [11, 18], [12, 18], [9, 19], [10, 19], [11, 19], [10, 20], [11, 20], [11, 21], [11, 22]]

Existing cluster neurons:
[[9, 13], [6, 14], [7, 14], [8, 14], [9, 14], [10, 14], [6, 15], [7, 15], [8, 15], [9, 15], [10, 15], [11, 15], [6, 16], [7, 16], [8, 16], [9, 16], [10, 16], [11, 16], [12, 16], [7, 17], [9, 17], [10, 17], [11, 17], [12, 17], [7, 18], [8, 18], [9, 18], [10, 18], [11, 18], [12, 18], [9, 19], [10, 19], [11, 19], [10, 20], [11, 20], [11, 21], [11, 22]]

Matching movie IDs:
[84, 1302, 918, 2251, 2653, 3239, 4326, 4344, 4182, 230, 958, 4625, 401, 462, 927, 1027, 1034, 1594, 2583, 3099, 4319, 4637, 4672, 112, 4086, 297, 1435, 1553, 1666, 2439, 2464, 3670, 3693, 4198, 4894, 4486, 4953, 760, 2206, 2867, 2921, 2981, 4304, 4350, 4840, 2655, 3568, 4465, 3781, 3891, 3954, 4685, 1125, 4353, 100, 268, 361, 611, 871, 1582, 1965, 2106, 71, 1014, 1089, 2136, 3199, 3506, 3969, 4303, 917, 1899, 2543, 2617, 2917, 4113, 4116, 2126, 2969, 3829, 254, 1437, 2021, 2089, 2173, 2698, 3063, 3619, 4103, 4434, 4737, 4851, 373, 732, 1004, 1130, 1648, 1719, 2244, 2699, 4000, 4197, 4480, 796, 1107, 1307, 2352, 2411, 4433, 4944, 2840, 3446, 4374, 4421, 4768, 4984, 569, 839, 1366, 1893, 2641, 2766, 3221, 3233, 3284, 4484, 924, 1340, 2622, 3228, 3392, 3752, 4667, 1277, 1873, 2071, 2257, 2517, 3054, 3120, 85, 820, 1798, 2859, 947, 3574, 4855, 167, 2922, 3279, 566, 892, 2187, 2515, 3085, 3090, 1380, 2470, 3371, 4998, 115, 203, 641, 860, 1612, 2590, 3596, 3795, 3938, 4623, 1143, 3110, 4101, 3454, 397, 479, 2608, 850, 980]

Genres stats for these movies:
---------  -------------  -----  --------  ------  -----------  ---------------  -----  ------------  ---------  -------  -------------  ---------  ---------  ----------------  -----------------  ------  -------------  ---------------  ------------  ------------  ---------------  ------  --------------  ----------  ------  -----------  ----  ---------  -------  ----------------  ---------  --------------  ------------  ---------------  ---------------  ------  --------  ---------------  -------------  -------  ----------  -----  -----------  ----------------  --------------  ----------------  -----------  -------------------  --------------  --------------  --------  ------------  ---------  ---------------  -------------------  -------  -----------  ------  -----------  -----------------  -------  -------------  -----------  -----------  ---------------  ------  ----------------------  ----------  -------  --------------  -------------------------  ------------  ------------  --------  ------------------  ------
Category:  Crime Fiction  Drama  Thriller  Action  Crime Drama  Black-and-white  Indie  Romance Film  Film noir  Mystery  Gangster Film  Melodrama  Adventure  Action/Adventure  Biographical film  Comedy  Coming of age  Film adaptation  World cinema  Period piece  Courtroom Drama  Sports  Crime Thriller  Road movie  Horror  Comedy film  Teen  Biography  Western  Biopic [feature]  Docudrama  Blaxploitation  Family Drama  Science Fiction  Japanese Movies  Prison  Pre-Code  Political drama  New Hollywood  Musical  Short Film  Heist  Silent film  Historical drama  Christian film  Television movie  Documentary  Slice of life story  Chinese Movies  Romantic drama  Art film  Supernatural  Detective  Addiction Drama  Revisionist Western  Tragedy  Tragicomedy  Remake  Family Film  Detective fiction  Fantasy  Feminist Film  Chase Movie  Caper story  Film u00e0 clef  Boxing  Psychological thriller  Children's  History  Hybrid Western  Juvenile Delinquency Film  Media Satire  Comedy-drama  Neo-noir  Historical fiction  Outlaw
Count:     179            163    34        32      26           24               22     18            17         15       11             9          9          9                 9                  8       8              7                7             6             5                5       5               5           5       4            4     4          4        4                 4          3               3             3                3                3       2         2                2              2        2           2      2            2                 2               2                 2            1                    1               1               1         1             1          1                1                    1        1            1       1            1                  1        1              1            1            1                1       1                       1           1        1               1                          1             1             1         1                   1
---------  -------------  -----  --------  ------  -----------  ---------------  -----  ------------  ---------  -------  -------------  ---------  ---------  ----------------  -----------------  ------  -------------  ---------------  ------------  ------------  ---------------  ------  --------------  ----------  ------  -----------  ----  ---------  -------  ----------------  ---------  --------------  ------------  ---------------  ---------------  ------  --------  ---------------  -------------  -------  ----------  -----  -----------  ----------------  --------------  ----------------  -----------  -------------------  --------------  --------------  --------  ------------  ---------  ---------------  -------------------  -------  -----------  ------  -----------  -----------------  -------  -------------  -----------  -----------  ---------------  ------  ----------------------  ----------  -------  --------------  -------------------------  ------------  ------------  --------  ------------------  ------


====================== cid = 4 ==========================
Cluster neurons:
[[13, 0], [14, 0], [15, 0], [16, 0], [17, 0], [18, 0], [19, 0], [20, 0], [21, 0], [13, 1], [14, 1], [15, 1], [16, 1], [17, 1], [18, 1], [19, 1], [20, 1], [21, 1], [13, 2], [14, 2], [15, 2], [16, 2], [17, 2], [18, 2], [19, 2], [20, 2], [21, 2], [14, 3], [15, 3], [16, 3], [17, 3], [18, 3], [19, 3], [20, 3], [17, 4], [18, 4], [19, 4], [18, 5], [19, 5]]

Existing cluster neurons:
[[13, 0], [14, 0], [15, 0], [16, 0], [17, 0], [18, 0], [19, 0], [20, 0], [21, 0], [13, 1], [14, 1], [15, 1], [16, 1], [17, 1], [18, 1], [19, 1], [20, 1], [21, 1], [13, 2], [14, 2], [15, 2], [16, 2], [17, 2], [18, 2], [19, 2], [20, 2], [21, 2], [14, 3], [15, 3], [16, 3], [17, 3], [18, 3], [19, 3], [20, 3], [17, 4], [18, 4], [19, 4], [18, 5], [19, 5]]

Matching movie IDs:
[1026, 1088, 1093, 1189, 1516, 1638, 3387, 4945, 420, 1839, 2396, 3518, 3787, 3940, 4432, 4858, 4875, 2, 227, 1522, 2980, 3091, 3303, 4088, 279, 346, 457, 1121, 1207, 2405, 3809, 1463, 2007, 2132, 3461, 4011, 4365, 1055, 2302, 3682, 3803, 4309, 4600, 575, 1091, 1276, 1751, 1849, 2022, 2985, 3710, 3898, 4407, 4760, 942, 1929, 2098, 2949, 2990, 3478, 3771, 4720, 4769, 374, 890, 898, 1154, 1202, 1355, 1396, 1669, 3041, 3135, 3509, 3519, 3826, 4801, 4901, 847, 852, 1436, 1995, 3629, 3646, 4552, 146, 326, 607, 843, 1401, 2195, 352, 656, 959, 2717, 3484, 1895, 2616, 3013, 3849, 2311, 3037, 3291, 4820, 1443, 1725, 2005, 3430, 3729, 2927, 4610, 1850, 2327, 3345, 3372, 3489, 3748, 4044, 4170, 4758, 179, 1012, 2975, 3778, 4264, 1838, 2712, 3281, 4040, 4289, 4759, 42, 2516, 3077, 3742, 121, 426, 433, 1181, 1604, 2417, 2732, 3353, 332, 549, 1831, 3265, 4616, 2049, 2991, 4356, 4564, 4627, 4731, 1781, 1864, 2036, 3316, 3672, 4904, 130, 1540, 2261, 4412, 1050, 1112, 3144, 3559, 4531, 157, 2293, 2536, 3894, 3987, 4877, 1204, 1833, 2347, 3293, 4364, 147, 901, 2034, 4476, 545, 639, 1951, 2559, 3633, 4977, 119, 1286, 1826, 2045, 3036, 4504, 3007, 4302, 4951, 22, 450, 1221, 1361, 2665, 2800, 3634, 1256, 2220, 1880, 2410, 2612, 2689, 2720, 3179, 3723, 3012, 1974, 3688, 4254, 4312, 4468, 4750, 1196, 2154, 2835, 2998, 3162, 3343, 4626]

Genres stats for these movies:
---------  -----  ------------  ------------  ---------  -------  ------  --------  --------------  ------  -------------  ----------------  -----------  ---------------  -----------  -------  ------  ---------  --------------  --------  ------------  ----------------  -----------------  ----------  -----  ---------------  ------------  -----------  ----  ---------  --------------  --------  ------------  ---------------  ------  ---------  ---------------  -----------------  ------------------  -------------  -------------  ---------  -------  ---------------  ----------------  -----------------
Category:  Drama  World cinema  Romance Film  Bollywood  Musical  Action  Thriller  Romantic drama  Comedy  Crime Fiction  Action/Adventure  Family Film  Romantic comedy  Comedy film  Mystery  Horror  Adventure  Crime Thriller  Art film  Tamil cinema  Action Thrillers  Biographical film  Buddy film  Indie  Black-and-white  Family Drama  Stoner film  Cult  Slapstick  Animal Picture  Suspense  Supernatural  Superhero movie  Sports  Tollywood  Film adaptation  Comedy of manners  Political thriller  Ensemble Film  Musical Drama  Melodrama  Fantasy  Japanese Movies  Screwball comedy  Martial Arts Film
Count:     139    110           79            75         72       44      37        36              33      18             14                11           9                8            8        7       5          4               4         3             3                 3                  2           2      2                2             2            2     2          1               1         1             1                1       1          1                1                  1                   1              1              1          1        1                1                 1
---------  -----  ------------  ------------  ---------  -------  ------  --------  --------------  ------  -------------  ----------------  -----------  ---------------  -----------  -------  ------  ---------  --------------  --------  ------------  ----------------  -----------------  ----------  -----  ---------------  ------------  -----------  ----  ---------  --------------  --------  ------------  ---------------  ------  ---------  ---------------  -----------------  ------------------  -------------  -------------  ---------  -------  ---------------  ----------------  -----------------


====================== cid = 6 ==========================
Cluster neurons:
[[25, 4], [21, 5], [22, 5], [23, 5], [24, 5], [25, 5], [26, 5], [20, 6], [21, 6], [22, 6], [23, 6], [24, 6], [25, 6], [26, 6], [20, 7], [21, 7], [22, 7], [23, 7], [24, 7], [25, 7], [26, 7], [20, 8], [21, 8], [22, 8], [23, 8], [24, 8], [25, 8], [26, 8], [21, 9], [22, 9], [23, 9], [24, 9], [25, 9], [26, 9], [22, 10], [23, 10], [24, 10], [22, 11], [23, 11]]

Existing cluster neurons:
[[25, 4], [21, 5], [22, 5], [23, 5], [24, 5], [25, 5], [26, 5], [20, 6], [21, 6], [22, 6], [23, 6], [24, 6], [25, 6], [26, 6], [20, 7], [21, 7], [22, 7], [23, 7], [24, 7], [25, 7], [26, 7], [20, 8], [21, 8], [22, 8], [23, 8], [25, 8], [26, 8], [21, 9], [22, 9], [23, 9], [24, 9], [25, 9], [26, 9], [22, 10], [23, 10], [22, 11], [23, 11]]

Matching movie IDs:
[323, 634, 1007, 1812, 2492, 2809, 2905, 3116, 3591, 3991, 4796, 4808, 43, 2435, 3960, 4780, 472, 1552, 538, 1137, 4300, 4416, 474, 3963, 4164, 4225, 4684, 4862, 666, 1275, 1856, 2225, 3118, 3793, 3943, 1949, 3429, 1924, 2914, 3533, 1356, 1692, 3273, 3825, 4732, 822, 1626, 3187, 4324, 955, 4026, 364, 976, 1885, 3532, 4202, 4498, 1920, 2174, 2547, 3104, 3866, 761, 962, 993, 1378, 3905, 4530, 4633, 396, 1192, 1245, 1793, 25, 2519, 2730, 3985, 703, 789, 1195, 1284, 1512, 1525, 1605, 1737, 3386, 3424, 4995, 1353, 2499, 3009, 3537, 4797, 264, 1251, 2600, 3806, 539, 567, 1009, 2115, 2527, 3946, 4340, 4437, 4495, 1094, 2537, 2581, 2739, 2849, 1749, 3780, 4066, 4195, 4411, 4554, 466, 476, 1876, 3498, 3790, 4176, 1386, 3605, 3837, 333, 496, 508, 2691, 3034, 4073, 4215, 4325, 4650, 87, 471, 819, 1011, 2008, 2164, 2205, 2795, 2945, 3381, 3776, 4271, 4410, 1081, 4019, 4711, 481, 2721, 1222, 345, 419, 669, 838, 3163, 3317, 3493, 3637, 3738, 3757, 4117, 7, 596, 1001, 1483, 1497, 2110, 2592, 3442, 3581, 4016, 4804, 2091, 3962, 4900]

Genres stats for these movies:
---------  ------  ------------  ---------------  -----  -----  -------  --------------  ------------  ---------------  ----  --------------  -------------  ---------  ----------  -----------  -------  -------------  ----------------  -------------  ----------------  ------  ----------------  ---------------  -----------  ---------------  ------  ----  -----------------  ---------  -----------------  ------------  --------------  ---------  --------  ---------------  -----------  ------  ----------------  ----------------  -------  --------------  -----  --------------  ----------  ----------  -----  ------------  --------------  ----------------  -------------------  ----------  ------  -----------  ----------  ---  -----------  --------------  -------------  ----------  --------------  ---------  -----------------  ----------  ---------  ------  -----------  --------------  ------------  ---------------  -----------  ------------  ------  -------------  ---------------  ---------  ------  -----------
Category:  Comedy  Romance Film  Romantic comedy  Drama  Indie  Musical  Romantic drama  Comedy-drama  Black-and-white  Teen  Musical comedy  Coming of age  Slapstick  Sex comedy  Family Film  Fantasy  Ensemble Film  Comedy of Errors  Crime Fiction  Screwball comedy  Satire  Television movie  Science Fiction  Silent film  Domestic Comedy  Action  Cult  Comedy of manners  Adventure  Children's/Family  Black comedy  Gross-out film  Gross out  Thriller  Film adaptation  Comedy film  Parody  Action/Adventure  Romantic fantasy  Mystery  Religious Film  Music  Marriage Drama  Children's  Buddy film  Dance  World cinema  Fantasy Comedy  Workplace Comedy  Slice of life story  Road movie  Sports  Stoner film  Short Film  Spy  Time travel  Hip hop movies  Musical Drama  Alien Film  Animal Picture  Animation  Backstage Musical  Beach Film  Bollywood  Boxing  Chase Movie  Crime Thriller  Family Drama  Filipino Movies  Filmed Play  Holiday Film  Horror  Horror Comedy  Japanese Movies  Melodrama  Remake  Zombie Film
Count:     175     173           144              52     37     34       25              24            22               15    10              8              7          7           7            7        6              6                 5              5                 5       5                 4                4            4                4       4     4                  4          3                  3             3               3          3         3                3            3       3                 3                 2        2               2      2               2           2           2      2             2               2                 2                    2           1       1            1           1    1            1               1              1           1               1          1                  1           1          1       1            1               1             1                1            1             1       1              1                1          1       1
---------  ------  ------------  ---------------  -----  -----  -------  --------------  ------------  ---------------  ----  --------------  -------------  ---------  ----------  -----------  -------  -------------  ----------------  -------------  ----------------  ------  ----------------  ---------------  -----------  ---------------  ------  ----  -----------------  ---------  -----------------  ------------  --------------  ---------  --------  ---------------  -----------  ------  ----------------  ----------------  -------  --------------  -----  --------------  ----------  ----------  -----  ------------  --------------  ----------------  -------------------  ----------  ------  -----------  ----------  ---  -----------  --------------  -------------  ----------  --------------  ---------  -----------------  ----------  ---------  ------  -----------  --------------  ------------  ---------------  -----------  ------------  ------  -------------  ---------------  ---------  ------  -----------


====================== cid = 7 ==========================
Cluster neurons:
[[25, 23], [26, 23], [24, 24], [25, 24], [26, 24], [23, 25], [24, 25], [25, 25], [26, 25], [27, 25], [23, 26], [24, 26], [25, 26], [26, 26], [27, 26], [28, 26], [29, 26], [23, 27], [24, 27], [25, 27], [26, 27], [27, 27], [28, 27], [29, 27], [23, 28], [24, 28], [25, 28], [26, 28], [27, 28], [28, 28], [29, 28], [24, 29], [25, 29], [26, 29], [27, 29], [28, 29], [29, 29]]

Existing cluster neurons:
[[25, 23], [26, 23], [24, 24], [25, 24], [26, 24], [23, 25], [25, 25], [26, 25], [27, 25], [23, 26], [24, 26], [25, 26], [26, 26], [27, 26], [28, 26], [29, 26], [23, 27], [24, 27], [25, 27], [26, 27], [27, 27], [28, 27], [29, 27], [23, 28], [24, 28], [25, 28], [26, 28], [27, 28], [28, 28], [29, 28], [24, 29], [25, 29], [26, 29], [27, 29], [29, 29]]

Matching movie IDs:
[2137, 2670, 681, 813, 3526, 499, 787, 1477, 2249, 2901, 3855, 4925, 4946, 161, 404, 869, 973, 1955, 2566, 3542, 3814, 3942, 4442, 4553, 555, 1105, 2326, 2574, 2618, 4233, 138, 247, 405, 553, 1235, 4160, 4917, 4448, 4571, 4741, 887, 1549, 3642, 3824, 582, 800, 895, 1333, 2082, 4464, 4765, 4809, 169, 679, 4445, 4779, 4957, 228, 3871, 4838, 773, 2095, 2701, 2894, 3337, 3356, 3726, 3885, 3896, 4814, 4857, 163, 340, 687, 1494, 1651, 1988, 2068, 3383, 4310, 4721, 650, 705, 723, 1289, 2018, 2051, 2357, 4813, 225, 2567, 3481, 806, 919, 1215, 1495, 1760, 2348, 2686, 3093, 3931, 375, 1524, 2893, 4149, 328, 797, 2635, 2748, 3209, 4027, 339, 3355, 4715, 286, 921, 1122, 1556, 1731, 1932, 3335, 3404, 4811, 848, 989, 1225, 1456, 2398, 2426, 3339, 3496, 4157, 1542, 2375, 4218, 4386, 1239, 1521, 2262, 2823, 2989, 3767, 4323, 1341, 2215, 4677, 4881, 2191, 3464, 3578, 4009, 4077, 2371, 2508, 3420, 3868, 721, 1269, 2093, 1099, 1278, 1323, 1624, 2775, 3939, 1013, 4348, 1249, 3699, 107, 990, 1458, 2549, 3530, 3582, 4140, 4963, 747, 1752, 2446, 2640, 3349, 4790, 4852, 4893, 2937, 3282, 3379, 3575, 3813, 4002, 4267, 331, 456, 1006, 1408, 2148, 2360, 2563, 2605, 2764, 3363, 4443, 4704, 302, 1008, 1017, 1957, 2454, 2558, 2685, 2740, 3463, 3959, 4203, 4345, 4444]

Genres stats for these movies:
---------  ---------  -----------  ----------  ------  -----------  -----------------  ---------  -------  -------  ----------  ------------------  ------------------  -----  -----  ---------------  --------------  ---------------  -------------------------  --------  ------  ------------  -------------  ----------------  ----------------  -------  -----------  ------  ----------------  -------  ------------  ----------  ----------  -------------  --------  -------------  ------  ----  -----------  -----------------------  ---------------  ------------  --------------------  ---------------  -----------------  ---------  ---------------  ------------  -----------  ------------------------  ---------------  --------------  ------------  ------------  -----------  ---------------  -----------  -------------  -----------------  ---------------  -------------  ---------------  ------------------  ------  -----------  ------------
Category:  Animation  Family Film  Short Film  Comedy  Comedy film  Children's/Family  Adventure  Fantasy  Musical  Children's  Children's Fantasy  Computer Animation  Drama  Music  Science Fiction  Fantasy Comedy  Black-and-white  Family-Oriented Adventure  Thriller  Parody  Romance Film  Crime Fiction  Animated Musical  Action/Adventure  Mystery  Stop motion  Action  Animated cartoon  Western  Holiday Film  Buddy film  Fairy tale  Costume drama  War film  Coming of age  Satire  Teen  Silent film  Sword and sorcery films  Propaganda film  Period piece  Mythological Fantasy  Superhero movie  Sword and sorcery  Absurdism  Japanese Movies  Black comedy  Caper story  Children's Entertainment  Christmas movie  Comedy Western  Comedy-drama  Crime Comedy  Documentary  Domestic Comedy  Educational  Ensemble Film  Fantasy Adventure  Film adaptation  Gangster Film  Historical Epic  Historical fiction  Horror  Live action  World cinema
Count:     181        137          135         77      40           31                 30         22       21       15          14                  10                  9      8      6                6               6                6                          5         5       5             5              5                 5                 4        4            4       4                 4        3             3           3           2              2         2              1       1     1            1                        1                1             1                     1                1                  1          1                1             1            1                         1                1               1             1             1            1                1            1              1                  1                1              1                1                   1       1            1
---------  ---------  -----------  ----------  ------  -----------  -----------------  ---------  -------  -------  ----------  ------------------  ------------------  -----  -----  ---------------  --------------  ---------------  -------------------------  --------  ------  ------------  -------------  ----------------  ----------------  -------  -----------  ------  ----------------  -------  ------------  ----------  ----------  -------------  --------  -------------  ------  ----  -----------  -----------------------  ---------------  ------------  --------------------  ---------------  -----------------  ---------  ---------------  ------------  -----------  ------------------------  ---------------  --------------  ------------  ------------  -----------  ---------------  -----------  -------------  -----------------  ---------------  -------------  ---------------  ------------------  ------  -----------  ------------


====================== cid = 1 ==========================
Cluster neurons:
[[20, 16], [18, 17], [19, 17], [20, 17], [21, 17], [16, 18], [17, 18], [18, 18], [19, 18], [20, 18], [21, 18], [22, 18], [16, 19], [17, 19], [18, 19], [19, 19], [20, 19], [21, 19], [17, 20], [18, 20], [19, 20], [20, 20], [21, 20], [17, 21], [18, 21], [19, 21], [20, 21], [21, 21], [17, 22], [18, 22], [19, 22], [20, 22], [17, 23], [18, 23], [19, 23], [20, 23]]

Existing cluster neurons:
[[20, 16], [18, 17], [19, 17], [20, 17], [21, 17], [16, 18], [17, 18], [18, 18], [19, 18], [20, 18], [21, 18], [22, 18], [17, 19], [18, 19], [19, 19], [20, 19], [21, 19], [17, 20], [18, 20], [19, 20], [20, 20], [21, 20], [17, 21], [18, 21], [19, 21], [20, 21], [21, 21], [17, 22], [18, 22], [19, 22], [20, 22], [17, 23], [18, 23], [19, 23], [20, 23]]

Matching movie IDs:
[1106, 3569, 4537, 4574, 295, 464, 1676, 2521, 272, 2231, 581, 1282, 3342, 3841, 91, 630, 3455, 1504, 1892, 2681, 3160, 1190, 1828, 2467, 3038, 4076, 4521, 799, 2234, 3003, 3207, 4643, 303, 791, 1351, 2054, 2080, 2548, 2565, 2610, 2885, 3623, 3990, 620, 983, 1339, 1485, 2916, 386, 795, 1617, 2300, 3488, 616, 697, 1940, 3916, 4099, 4425, 933, 4355, 4527, 658, 1677, 1994, 3447, 4833, 1923, 2361, 2532, 3587, 4245, 4913, 44, 2113, 2406, 393, 478, 1052, 3010, 4835, 29, 1464, 1550, 1938, 3231, 3328, 4058, 4060, 1071, 2270, 2447, 4131, 4618, 779, 1721, 1935, 4601, 250, 290, 324, 387, 1792, 3185, 3669, 4190, 4538, 4664, 4681, 4252, 4387, 4588, 271, 709, 1374, 3483, 733, 2414, 3585, 1031, 2497, 399, 701, 934, 2086, 2875, 3925, 266, 379, 1843, 2271, 4042, 4825, 2834, 2931, 4595, 638, 2671, 278, 311, 763, 982, 1183, 3712, 4184, 4335, 1005, 1445, 2145, 2572, 2687, 3641, 4137, 4391, 1717, 1729, 1814, 2535, 4632, 4694, 4924]

Genres stats for these movies:
---------  -----------  ---------------  -----------------  ---------  ----------  -----  -----  ------------  ----------------  ------  -------  -----  ---------------  ------------  -------------  ----------------  --------  ------  -----  ---------  ------  ---------------------------  --------  ---------------  ----------------------------------------  ---------------  -----------------  ------------  ------------  --------  --------  ---------  -----  ------------  -------  -------------  -------------  ---------------  ---------  --------------  ----  ------  --------  ----------------  -------------  ------  -----------------  --------------  -------------  -------------  -----------  ---------------------  --------------  -------------------------------  -----------  -------  -----------  --------  ------  ------------------------  -----------------  ------------------  ------------------------  -----------------  -------------  -----------  -------------  -----------
Category:  Documentary  Science Fiction  Culture & Society  Biography  Short Film  Indie  Music  World cinema  Political cinema  Sports  History  Drama  Japanese Movies  Rockumentary  Social issues  Television movie  War film  Action  Anime  Adventure  Travel  Interpersonal Relationships  Disaster  Political drama  Apocalyptic and post-apocalyptic fiction  Black-and-white  Biographical film  Romance Film  Mockumentary  Dystopia  Thriller  Computers  Dance  Concert film  Animals  Monster movie  Sci-Fi Horror  Romantic comedy  Tokusatsu  Religious Film  News  Nature  Business  Action Thrillers  Creature Film  Latino  Martial Arts Film  Crime Thriller  Doomsday film  Comedy horror  Educational  Environmental Science  Extreme Sports  Family & Personal Relationships  Family Film  Fantasy  Comedy film  Gulf War  Comedy  Illnesses & Disabilities  Children's/Family  Instrumental Music  Inventions & Innovations  Children's Issues  Costume drama  Law & Crime  Media Studies  Future noir
Count:     113          35               21                 19         18          14     11     9             8                 7       7        7      6                6             5              5                 5         5       4      4          3       3                            3         3                2                                         2                2                  2             2             2         2         2          2      2             1        1              1              1                1          1               1     1       1         1                 1              1       1                  1               1              1              1            1                      1               1                                1            1        1            1         1       1                         1                  1                   1                         1                  1              1            1              1
---------  -----------  ---------------  -----------------  ---------  ----------  -----  -----  ------------  ----------------  ------  -------  -----  ---------------  ------------  -------------  ----------------  --------  ------  -----  ---------  ------  ---------------------------  --------  ---------------  ----------------------------------------  ---------------  -----------------  ------------  ------------  --------  --------  ---------  -----  ------------  -------  -------------  -------------  ---------------  ---------  --------------  ----  ------  --------  ----------------  -------------  ------  -----------------  --------------  -------------  -------------  -----------  ---------------------  --------------  -------------------------------  -----------  -------  -----------  --------  ------  ------------------------  -----------------  ------------------  ------------------------  -----------------  -------------  -----------  -------------  -----------


====================== cid = 19 ==========================
Cluster neurons:
[[1, 17], [0, 18], [1, 18], [0, 19], [1, 19], [0, 20], [1, 20], [2, 20], [0, 21], [1, 21], [2, 21], [3, 21], [0, 22], [1, 22], [2, 22], [3, 22], [4, 22], [5, 22], [1, 23], [2, 23], [3, 23], [4, 23], [5, 23], [6, 23], [3, 24], [4, 24], [5, 24], [6, 24], [4, 25], [5, 25], [6, 25], [5, 26], [6, 26], [6, 27], [6, 28], [7, 29]]

Existing cluster neurons:
[[1, 17], [0, 18], [1, 18], [0, 19], [1, 19], [0, 20], [2, 20], [0, 21], [1, 21], [2, 21], [3, 21], [0, 22], [1, 22], [2, 22], [3, 22], [4, 22], [1, 23], [2, 23], [3, 23], [4, 23], [5, 23], [6, 23], [3, 24], [4, 24], [5, 24], [6, 24], [4, 25], [5, 25], [6, 25], [6, 26], [6, 27], [6, 28], [7, 29]]

Matching movie IDs:
[1048, 2391, 2621, 4518, 4979, 2569, 2928, 3048, 4201, 4213, 1176, 2736, 3892, 3998, 62, 626, 1414, 3180, 3609, 3728, 4084, 4153, 4280, 105, 1765, 2460, 3246, 37, 909, 1344, 3391, 4502, 4932, 752, 1720, 4004, 4910, 330, 1263, 1440, 2196, 2611, 2624, 3016, 3572, 3882, 219, 597, 1480, 1854, 3584, 4089, 4594, 52, 535, 3059, 3280, 3472, 3711, 400, 1290, 1395, 1989, 2341, 3508, 3817, 4012, 4127, 4590, 740, 1019, 1072, 2377, 2445, 2539, 2797, 3052, 3140, 3340, 3919, 4192, 4734, 47, 905, 265, 674, 856, 1674, 1707, 4635, 4856, 1417, 1805, 4867, 2995, 3065, 3680, 4973, 371, 1461, 3564, 3754, 4690, 805, 893, 1022, 1197, 1691, 1863, 2796, 3267, 3836, 4644, 2765, 4102, 4806, 559, 640, 1308, 2334, 3153, 3719, 4592, 4723, 4821, 406, 1883, 2017, 2087, 2107, 2832, 3061, 3212, 4285, 2900, 937, 2458, 2925, 2974, 3169, 4524, 580, 605, 811, 1515, 1580, 2305, 4473, 1486, 1728, 1778, 3566, 3808, 4255, 4578, 4826, 706, 1643, 2924, 1312, 2486, 3897, 4194, 492, 1173, 1573, 2292, 3127, 4634, 4678, 4697, 4767, 4990, 3334, 477, 3198, 1990, 2957, 3427, 4199, 1065, 1451, 2050, 2078, 3211, 4799, 395, 953, 1218, 1905, 2223, 4037]

Genres stats for these movies:
---------  --------  ------  ----------------------  -----  ------------  ---------------  -------  ------------  --------------  -----  -------  ------  --------  -------------  ---------------  ----  ---------------  ----  -------  ---------------  ------------------  -------------  ----------  ------------------  -----------  --------  ---------  ----------------  -------  ------  ------------  ---------------  -------------  ---------------  ------------  -------------  -----  ---  -------------  ----------------  --------------  ----------------  -----------  -----------  ----------  ---------  ----------  ---------------  ----------------  ------  ----------  ---------  ------------  -----------  --------------  -----------  ------------------  ---------------  -----------------  ---------  -------------  -------------  -----------  -----------------  ----------  ------------  -----------  --------  -----------  -----------  ---------  ------------  -------------  -----
Category:  Thriller  Horror  Psychological thriller  Drama  Supernatural  Science Fiction  Mystery  World cinema  Crime Thriller  Indie  Slasher  Action  Suspense  Crime Fiction  Japanese Movies  Cult  Erotic thriller  Teen  Fantasy  Black-and-white  Political thriller  Creature Film  Alien Film  Haunted House Film  Gothic Film  Disaster  Adventure  Television movie  B-movie  Comedy  Romance Film  Film adaptation  Monster movie  Filipino Movies  Period piece  Sci-Fi Horror  Anime  Spy  Demonic child  Action/Adventure  Chinese Movies  Romantic fantasy  Road-Horror  Stop motion  Road movie  Animation  Surrealism  Political drama  Action Thrillers  Satire  Short Film  Bollywood  Mockumentary  Chase Movie  Christian film  Comedy film  Computer Animation  Courtroom Drama  Biographical film  Cyberpunk  Doomsday film  Ensemble Film  Escape Film  Experimental film  Fairy tale  Family Drama  Family Film  Whodunit  Future noir  Avant-garde  Anthology  Media Satire  New Hollywood  Heist
Count:     190       109     57                      35     34            30               28       22            19              18     16       14      13        9              9                8     7                6     5        5                5                   4              4           4                   4            4         4          3                 3        3       3             2                2              2                2             2              2      2    2              2                 2               1                 1            1            1           1          1           1                1                 1       1           1          1             1            1               1            1                   1                1                  1          1              1              1            1                  1           1             1            1         1            1            1          1             1              1
---------  --------  ------  ----------------------  -----  ------------  ---------------  -------  ------------  --------------  -----  -------  ------  --------  -------------  ---------------  ----  ---------------  ----  -------  ---------------  ------------------  -------------  ----------  ------------------  -----------  --------  ---------  ----------------  -------  ------  ------------  ---------------  -------------  ---------------  ------------  -------------  -----  ---  -------------  ----------------  --------------  ----------------  -----------  -----------  ----------  ---------  ----------  ---------------  ----------------  ------  ----------  ---------  ------------  -----------  --------------  -----------  ------------------  ---------------  -----------------  ---------  -------------  -------------  -----------  -----------------  ----------  ------------  -----------  --------  -----------  -----------  ---------  ------------  -------------  -----


====================== cid = 11 ==========================
Cluster neurons:
[[23, 18], [22, 19], [23, 19], [22, 20], [23, 20], [24, 20], [25, 20], [22, 21], [23, 21], [24, 21], [25, 21], [26, 21], [21, 22], [22, 22], [23, 22], [24, 22], [25, 22], [26, 22], [27, 22], [28, 22], [29, 22], [21, 23], [22, 23], [23, 23], [24, 23], [27, 23], [28, 23], [29, 23], [23, 24], [27, 24], [28, 24], [29, 24], [28, 25], [29, 25]]

Existing cluster neurons:
[[23, 18], [22, 19], [23, 19], [22, 20], [23, 20], [24, 20], [25, 20], [22, 21], [23, 21], [24, 21], [25, 21], [26, 21], [21, 22], [22, 22], [23, 22], [24, 22], [25, 22], [26, 22], [27, 22], [28, 22], [29, 22], [21, 23], [22, 23], [23, 23], [24, 23], [27, 23], [28, 23], [29, 23], [23, 24], [27, 24], [28, 24], [29, 24], [28, 25], [29, 25]]

Matching movie IDs:
[38, 1248, 2861, 1835, 3749, 1297, 1444, 3772, 4409, 2813, 3096, 4503, 170, 308, 316, 583, 944, 964, 1730, 2033, 2877, 3154, 3784, 4575, 4791, 1917, 4764, 1233, 1985, 3692, 4162, 4305, 1609, 3294, 344, 517, 949, 2166, 2531, 2693, 2793, 2973, 465, 831, 974, 3210, 3290, 3691, 4029, 410, 788, 1997, 3403, 388, 702, 1767, 2112, 3735, 370, 1423, 1476, 1509, 2892, 4669, 176, 237, 543, 1224, 3018, 560, 737, 1685, 2433, 2760, 3657, 3741, 4187, 1, 840, 1053, 1285, 1590, 1991, 2907, 4513, 730, 1329, 1502, 1881, 1912, 2678, 3047, 3312, 3734, 4604, 4631, 1705, 695, 729, 1572, 2070, 56, 606, 655, 828, 1682, 1698, 2903, 3051, 3716, 4389, 300, 1800, 2124, 3394, 4740, 1317, 3043, 3080, 1073, 1164, 1252, 1326, 2513, 2733, 4920, 502, 1686, 4785, 2979, 4956, 536, 3632, 3847, 4770, 4899, 1044, 3341, 3376, 3494, 4501, 1170, 2029, 3684, 4459, 166, 1817, 4226, 248, 267, 392, 759, 790, 1331, 1918, 2997, 3268, 3315, 3580, 4047, 21, 129, 785, 1324, 2297, 2553, 2805, 2946, 3122, 3565, 4580, 64, 4112, 348, 437, 446, 862, 1652, 1825, 3002, 3123, 4691]

Genres stats for these movies:
---------  ----------  ------  ---------------  -----  -----------  ---------  ---------  -----------  ---------------  -----------  ----------------  ------  -------  ---------------  ------------------  ------------  -----  --------  ------  -----------  -----------  -----------  ---------  ------  ----------------  --------------  -------  ------  ---------------  --------------  ---------  ------------  --------------  --------  -------------  ------------  ----------------  --------------  --------------  ----------  -----------------  -----------  -----  ----------------  -------  --------
Category:  Short Film  Comedy  Black-and-white  Indie  Silent film  Animation  Slapstick  Family Film  Science Fiction  Documentary  Animated cartoon  Parody  Fantasy  Japanese Movies  Computer Animation  Romance Film  Drama  Thriller  Horror  Educational  Comedy film  Stop motion  Adventure  Action  Screwball comedy  Sponsored film  Mystery  Sports  Political drama  Musical comedy  Absurdism  Mockumentary  Extreme Sports  Dystopia  Crime Fiction  Crime Comedy  Comedy of Errors  Clay animation  Christian film  Children's  Biographical film  Auto racing  Anime  Adventure Comedy  Musical  War film
Count:     177         71      61               27     24           21         17         15           5                5            3                 3       3        3                3                   3             3      2         2       2            2            2            2          2       1                 1               1        1       1                1               1          1             1               1         1              1             1                 1               1               1           1                  1            1      1                 1        1
---------  ----------  ------  ---------------  -----  -----------  ---------  ---------  -----------  ---------------  -----------  ----------------  ------  -------  ---------------  ------------------  ------------  -----  --------  ------  -----------  -----------  -----------  ---------  ------  ----------------  --------------  -------  ------  ---------------  --------------  ---------  ------------  --------------  --------  -------------  ------------  ----------------  --------------  --------------  ----------  -----------------  -----------  -----  ----------------  -------  --------


====================== cid = 0 ==========================
Cluster neurons:
[[18, 24], [19, 24], [20, 24], [17, 25], [18, 25], [19, 25], [20, 25], [16, 26], [17, 26], [18, 26], [19, 26], [20, 26], [21, 26], [16, 27], [17, 27], [18, 27], [19, 27], [20, 27], [21, 27], [22, 27], [17, 28], [18, 28], [19, 28], [20, 28], [21, 28], [22, 28], [12, 29], [17, 29], [18, 29], [19, 29], [20, 29], [21, 29], [22, 29], [23, 29]]

Existing cluster neurons:
[[18, 24], [19, 24], [20, 24], [17, 25], [18, 25], [19, 25], [20, 25], [16, 26], [17, 26], [18, 26], [19, 26], [20, 26], [21, 26], [16, 27], [17, 27], [18, 27], [19, 27], [20, 27], [21, 27], [22, 27], [17, 28], [18, 28], [19, 28], [20, 28], [21, 28], [22, 28], [12, 29], [17, 29], [18, 29], [19, 29], [20, 29], [21, 29], [22, 29], [23, 29]]

Matching movie IDs:
[1162, 2097, 3874, 631, 727, 896, 988, 1021, 2055, 2438, 3285, 4608, 4696, 4971, 2577, 2758, 3995, 4426, 589, 1422, 2190, 3115, 722, 864, 866, 2004, 3004, 3350, 3541, 4964, 4120, 1745, 4316, 4360, 60, 609, 855, 1416, 1646, 1807, 1845, 2785, 4070, 362, 4320, 2416, 3295, 3441, 3477, 3590, 4239, 4359, 4420, 4942, 533, 1394, 1442, 1660, 1822, 2353, 2362, 2757, 4134, 2216, 2883, 4707, 1473, 1583, 1642, 2285, 3869, 4085, 4935, 122, 635, 1045, 1304, 1711, 394, 693, 3453, 3687, 218, 4815, 440, 3240, 3323, 3701, 4207, 4869, 111, 223, 282, 385, 1247, 1628, 3418, 4591, 4993, 1301, 1610, 1878, 1909, 2241, 2246, 3668, 4985, 2204, 2526, 4098, 4620, 4879, 306, 320, 1037, 1514, 1914, 2039, 2182, 2198, 2350, 3086, 3318, 3675, 4343, 194, 885, 1507, 2359, 501, 341, 554, 1438, 408, 3021, 519, 817, 1090, 1144, 1859, 1959, 2817, 2913, 3165, 4774, 718, 1281, 1619, 2737, 2862, 2940, 3193, 3411, 3630, 3863, 3976, 4429, 4493, 4781, 415, 432, 1358, 2767, 3413, 3529, 3663, 4154, 204, 349, 452, 1216, 2226, 2315, 3414, 3614, 3674, 3745, 3999, 4263, 4435, 4745, 619, 2131, 2141, 3070, 4059, 4260, 132, 486, 913, 1067, 1191, 2474, 3834, 3846, 4023, 4380, 4603, 1303, 1466, 2449, 3113, 4193, 4593, 4782, 1314, 1424, 2170, 2576, 2898, 3259, 3586, 3768]

Genres stats for these movies:
---------  ---------  -------  -----------  ---------------  -----  -----------------  ------  ----------------  ------  ---------  -------------------------  ------------  ------------------  ---------------  ------------  -------------  --------  -----------------  ----------  ----------------  -----  -------  ---------------  ---------------  -----  ------------  -----------  -----------  ------  ----  -------------  --------------  ----------  ---------  ---------------  ------------------  -------  ------------  -----------  ------  --------------------  ----------------  --------  ---------  ----  -------------  ----  --------------  -----------------  ----------------  --------  ------  -----------  ----------------  -------  -----------  ---------------  ---------------  ----------  ----------------  ----------------------------------------  --------  ------------  -----------------  ------------------  -------------  ----------  ---------  ---------  --------------  ------  ------------------  ----------  -----------  ----------------  -----------------------  ----------------  ---------------------------  ---------------  ------------------------------  ---------  ------  ---------  ----------------  --------------  -----------------  ----------------  ----------  --------------  -------------  ---------  -----------------  --------  -----------  --------------  --------------  ----------------  -------  -------------  -----------------  -------------  ------------  -----  -------------  --------------  --------------------  -------------  --------  -------------------------  -----------
Category:  Adventure  Fantasy  Family Film  Science Fiction  Drama  Children's/Family  Comedy  Action/Adventure  Action  Animation  Family-Oriented Adventure  World cinema  Children's Fantasy  Japanese Movies  Romance Film  Costume drama  Thriller  Fantasy Adventure  Children's  Television movie  Indie  Musical  Black-and-white  Film adaptation  Anime  Family Drama  Comedy film  Stop motion  Horror  Cult  Coming of age  Animal Picture  Fairy tale  Slapstick  Superhero movie  Historical fiction  Mystery  Period piece  Time travel  Parody  Mythological Fantasy  Adventure Comedy  War film  Biography  Teen  Doomsday film  Epic  Christian film  Costume Adventure  Sword and Sandal  Disaster  Satire  Silent film  Sci-Fi Adventure  Monster  Jungle Film  Heavenly Comedy  Historical Epic  Alien Film  Animated cartoon  Apocalyptic and post-apocalyptic fiction  Art film  Black comedy  Sword and sorcery  Computer Animation  Crime Fiction  Road movie  Docudrama  Superhero  Fantasy Comedy  Sports  Swashbuckler films  Short Film  Space opera  Supermarionation  Sword and sorcery films  Romantic fantasy  Roadshow theatrical release  Romantic comedy  Sci Fi Pictures original films  Absurdism  Prison  Americana  Animated Musical  Bengali Cinema  Biographical film  Biopic [feature]  Buddy film  Chinese Movies  Creature Film  Detective  Detective fiction  Dystopia  Escape Film  Religious Film  Existentialism  Historical drama  History  Horror Comedy  Martial Arts Film  Monster movie  Movie serial  Music  Musical Drama  Musical comedy  Natural horror films  New Hollywood  Pre-Code  Heaven-Can-Wait Fantasies  Zombie Film
Count:     168        87       75           69               63     41                 35      32                27      25         22                         20            19                  15               14            13             12        11                 11          10                10     9        9                9                8      8             7            7            7       7     6              6               4           4          4                4                   4        4             4            4       4                     3                 3         3          3     3              3     3               3                  3                 3         2       2            2                 2        2            2                2                2           2                 2                                         2         2             2                  2                   2              2           2          2          2               2       2                   1           1            1                 1                        1                 1                            1                1                               1          1       1          1                 1               1                  1                 1           1               1              1          1                  1         1            1               1               1                 1        1              1                  1              1             1      1              1               1                     1              1         1                          1
---------  ---------  -------  -----------  ---------------  -----  -----------------  ------  ----------------  ------  ---------  -------------------------  ------------  ------------------  ---------------  ------------  -------------  --------  -----------------  ----------  ----------------  -----  -------  ---------------  ---------------  -----  ------------  -----------  -----------  ------  ----  -------------  --------------  ----------  ---------  ---------------  ------------------  -------  ------------  -----------  ------  --------------------  ----------------  --------  ---------  ----  -------------  ----  --------------  -----------------  ----------------  --------  ------  -----------  ----------------  -------  -----------  ---------------  ---------------  ----------  ----------------  ----------------------------------------  --------  ------------  -----------------  ------------------  -------------  ----------  ---------  ---------  --------------  ------  ------------------  ----------  -----------  ----------------  -----------------------  ----------------  ---------------------------  ---------------  ------------------------------  ---------  ------  ---------  ----------------  --------------  -----------------  ----------------  ----------  --------------  -------------  ---------  -----------------  --------  -----------  --------------  --------------  ----------------  -------  -------------  -----------------  -------------  ------------  -----  -------------  --------------  --------------------  -------------  --------  -------------------------  -----------


====================== cid = 8 ==========================
Cluster neurons:
[[23, 0], [24, 0], [25, 0], [26, 0], [27, 0], [28, 0], [29, 0], [22, 1], [23, 1], [24, 1], [25, 1], [26, 1], [27, 1], [28, 1], [29, 1], [22, 2], [23, 2], [24, 2], [25, 2], [26, 2], [27, 2], [28, 2], [29, 2], [22, 3], [23, 3], [24, 3], [25, 3], [26, 3], [27, 3], [22, 4], [23, 4], [24, 4], [26, 4]]

Existing cluster neurons:
[[23, 0], [24, 0], [25, 0], [26, 0], [27, 0], [28, 0], [29, 0], [22, 1], [23, 1], [24, 1], [25, 1], [26, 1], [27, 1], [28, 1], [29, 1], [22, 2], [23, 2], [24, 2], [25, 2], [26, 2], [27, 2], [28, 2], [29, 2], [22, 3], [23, 3], [24, 3], [25, 3], [26, 3], [27, 3], [22, 4], [23, 4], [24, 4], [26, 4]]

Matching movie IDs:
[2780, 207, 372, 873, 1142, 1147, 1160, 1274, 1696, 3417, 4100, 946, 1151, 2988, 3272, 3445, 3500, 3918, 3936, 4874, 244, 879, 954, 1808, 1853, 2064, 2457, 2644, 3033, 3774, 3859, 4514, 4939, 270, 1541, 4772, 1715, 2669, 3087, 3399, 3978, 4046, 4165, 4220, 4235, 4547, 136, 201, 277, 356, 414, 444, 624, 1596, 1796, 1857, 2111, 2240, 2541, 3184, 3222, 3237, 3524, 4408, 4542, 660, 32, 260, 1434, 1818, 2472, 2562, 2754, 2790, 1618, 1829, 4405, 210, 830, 1786, 2197, 2330, 2437, 3791, 3850, 4068, 4545, 809, 2053, 8, 69, 319, 668, 1104, 1379, 1409, 2038, 2264, 2999, 4911, 908, 2268, 3513, 3914, 4379, 1536, 3608, 3698, 4372, 1637, 1851, 1520, 2011, 2119, 2479, 2948, 273, 1868, 2836, 526, 724, 2934, 4214, 683, 1364, 1734, 2855, 403, 455, 564, 623, 731, 846, 991, 1180, 2328, 2481, 2510, 2656, 2697, 2882, 3170, 3639, 4008, 192, 1010, 1537, 1645, 2291, 3479, 4055, 293, 665, 743, 1420, 1454, 2002, 3635, 3779, 4177, 3690, 17, 2952, 2993, 3324, 3811, 4240, 4297, 463, 470, 604, 883, 2108, 3142, 3435, 3926, 4378, 4912, 173, 3384, 602, 2881, 4651, 2222, 4228, 4499, 2471, 3965, 1047, 453, 1511, 2751, 3867, 4931, 653, 3906]

Genres stats for these movies:
---------  ------------  -----  --------------  ------------  ------------  ------  ---------------  ---------------  -------  -----  ---------------  ---------  -------------  ---------  -------------  ----------------  --------  -------  ------------  --------------  -------------  ---------  ------------  ---------------  ----  --------  -----------------  --------  ----------------  --------------  -------  ---------------  ------  ----------------  ---------  ---------------  ----------  --------------  -------------  -----  -------------  -------------------  -----------------  -----------------  -------  ----------------  -------------------  ------  -----------  ------  -------------  ------------------  -------  -------  ---------------  ---------------  ----------  ------  ----------------  ----------  ----  -----------  ------------  ---  --------------  ----------------  ------------  ---------  -------------  -----------------  -----------  --------------  -------------  ---------------  --------------------  -----------  ---------------  ---------  -----  ----  --------------  --------------  ---------  ----------------------  ----------------  --------  ---------  -----------  ---------------------------
Category:  Romance Film  Drama  Romantic drama  World cinema  Period piece  Comedy  Romantic comedy  Film adaptation  Musical  Indie  Black-and-white  Bollywood  Crime Fiction  Melodrama  Costume drama  Action/Adventure  Thriller  Fantasy  Comedy-drama  Chinese Movies  Coming of age  Adventure  Family Drama  Filipino Movies  Teen  Art film  Biographical film  War film  Television movie  Marriage Drama  Western  Political drama  Action  Biopic [feature]  Biography  Japanese Movies  Road movie  Crime Thriller  Feminist Film  Dance  Ensemble Film  Inspirational Drama  Costume Adventure  Comedy of manners  Mystery  Romantic fantasy  Slice of life story  Sports  Comedy film  Horror  Musical Drama  Historical fiction  History  Tragedy  Addiction Drama  Domestic Comedy  Fairy tale  Satire  Screwball comedy  Sex comedy  Epic  Silent film  Erotic Drama  Spy  Christian film  British New Wave  Epic Western  Tollywood  Comedy horror  Experimental film  Avant-garde  Religious Film  Gangster Film  Film u00e0 clef  Kitchen sink realism  Crime Drama  Medical fiction  Film noir  Music  Cult  Animal Picture  Musical comedy  Docudrama  Airplanes and airports  Comedy of Errors  Pre-Code  Punk rock  Family Film  Roadshow theatrical release
Count:     196           195    192             94            41            39      35               22               21       19     15               14         11             11         11             11                10        10       9             8               8              8          7             7                7     7         6                  6         5                 5               4        4                4       4                 3          3                3           3               3              3      2              2                    2                  2                  2        2                 2                    2       2            2       2              2                   2        1        1                1                1           1       1                 1           1     1            1             1    1               1                 1             1          1              1                  1            1               1              1                1                     1            1                1          1      1     1               1               1          1                       1                 1         1          1            1
---------  ------------  -----  --------------  ------------  ------------  ------  ---------------  ---------------  -------  -----  ---------------  ---------  -------------  ---------  -------------  ----------------  --------  -------  ------------  --------------  -------------  ---------  ------------  ---------------  ----  --------  -----------------  --------  ----------------  --------------  -------  ---------------  ------  ----------------  ---------  ---------------  ----------  --------------  -------------  -----  -------------  -------------------  -----------------  -----------------  -------  ----------------  -------------------  ------  -----------  ------  -------------  ------------------  -------  -------  ---------------  ---------------  ----------  ------  ----------------  ----------  ----  -----------  ------------  ---  --------------  ----------------  ------------  ---------  -------------  -----------------  -----------  --------------  -------------  ---------------  --------------------  -----------  ---------------  ---------  -----  ----  --------------  --------------  ---------  ----------------------  ----------------  --------  ---------  -----------  ---------------------------


====================== cid = 17 ==========================
Cluster neurons:
[[4, 15], [2, 16], [3, 16], [4, 16], [5, 16], [2, 17], [3, 17], [4, 17], [5, 17], [6, 17], [2, 18], [3, 18], [4, 18], [5, 18], [6, 18], [2, 19], [3, 19], [4, 19], [5, 19], [6, 19], [7, 19], [8, 19], [3, 20], [4, 20], [5, 20], [6, 20], [7, 20], [8, 20], [4, 21], [5, 21], [6, 21], [7, 21], [6, 22]]

Existing cluster neurons:
[[4, 15], [2, 16], [3, 16], [4, 16], [5, 16], [2, 17], [3, 17], [4, 17], [5, 17], [6, 17], [2, 18], [3, 18], [4, 18], [5, 18], [6, 18], [2, 19], [3, 19], [4, 19], [5, 19], [6, 19], [7, 19], [8, 19], [3, 20], [4, 20], [5, 20], [7, 20], [8, 20], [4, 21], [5, 21], [6, 21], [7, 21], [6, 22]]

Matching movie IDs:
[117, 434, 1727, 1884, 3393, 3398, 532, 1226, 4695, 1404, 3828, 4778, 534, 3503, 4933, 857, 1346, 2138, 2436, 3452, 3697, 3924, 177, 664, 935, 960, 1134, 1616, 1900, 3225, 3235, 3627, 4075, 4540, 4560, 516, 1227, 2010, 2278, 2871, 992, 2294, 2395, 2747, 3597, 3948, 802, 1656, 1662, 1919, 1343, 1710, 4783, 2307, 2423, 1500, 1971, 3616, 4219, 355, 1023, 1296, 3786, 3886, 4155, 835, 1526, 4470, 200, 1186, 3360, 3389, 3986, 4108, 4234, 137, 1615, 1763, 1916, 1979, 2614, 3249, 461, 3408, 4675, 4819, 4983, 206, 310, 1492, 1205, 1530, 1946, 3006, 3305, 4281, 251, 391, 1266, 2160, 2591, 4568, 4597, 4649, 198, 762, 3796, 4646, 2443, 4139, 4175, 4569, 1783, 2505, 3666, 4453, 775, 1794, 2392, 2557, 3831, 4333, 336, 963, 1712, 4930, 1945, 2283, 3005, 531, 1369, 1585, 3673, 4097, 298, 614, 3028, 3920, 4061, 27, 2976, 4392, 675, 1058, 1891, 2349, 2570, 3512, 3540, 4268]

Genres stats for these movies:
---------  --------  -----  -------------  -------  --------------  ----------------------  -----  --------  ---------------  ------------  ---------  ------  ---------  ---------------  -----------------  ------  -----------  -------------  ---------------  ------------------  ---------------  ------------  ------------  ---------------  ---------------  ---------------  ------------  ---------  ------------  -------  ------------  -----------  ---------------  ---  ------  ------------  -------------  --------  ----------------  -----  --------------  ----------------  -------------  ------  -----------  ----  ---------  -------------  ------  -------  -----------  --------------  ----------------  ----  -------------  --------------  -----------  ----------------  -----------  ----------------  --------  ---------------  ----------  ----------  --------------  ------------  -----------  --------  ---------  --------  -----  ---------  ------------------------  -----------  -----------  -----------------  ----------  -------
Category:  Thriller  Drama  Crime Fiction  Mystery  Crime Thriller  Psychological thriller  Indie  Suspense  Black-and-white  World cinema  Film noir  Action  Detective  Film adaptation  Detective fiction  Comedy  Crime Drama  Gangster Film  Science Fiction  Political thriller  Erotic thriller  Black comedy  Romance Film  Japanese Movies  Political drama  Courtroom Drama  Supernatural  Adventure  Period piece  Fantasy  Comedy-drama  Comedy film  Comedy Thriller  Spy  Satire  Crime Comedy  Coming of age  Neo-noir  Action/Adventure  Heist  Romantic drama  Television movie  New Hollywood  Sports  Caper story  Cult  Docudrama  Ensemble Film  Horror  Tragedy  Time travel  Christian film  Comedy of Errors  Teen  Costume drama  Blaxploitation  Silent film  Screwball comedy  Documentary  Biopic [feature]  Dystopia  Romantic comedy  Road movie  Alien Film  Existentialism  Family Drama  Family Film  Whodunit  Biography  Art film  Music  Melodrama  Libraries and librarians  Legal drama  Future noir  Biographical film  Buddy film  History
Count:     149       115    88             87       86              31                      28     23        21               20            17         15      13         13               12                 12      10           9              9                8                   8                7             6             5                5                5                5             5          4             4        4             4            4                3    3       3             3              3         3                 3      2               2                 2              2       2            2     2          2              1       1        1            1               1                 1     1              1               1            1                 1            1                 1         1                1           1           1               1             1            1         1          1         1      1          1                         1            1            1                  1           1
---------  --------  -----  -------------  -------  --------------  ----------------------  -----  --------  ---------------  ------------  ---------  ------  ---------  ---------------  -----------------  ------  -----------  -------------  ---------------  ------------------  ---------------  ------------  ------------  ---------------  ---------------  ---------------  ------------  ---------  ------------  -------  ------------  -----------  ---------------  ---  ------  ------------  -------------  --------  ----------------  -----  --------------  ----------------  -------------  ------  -----------  ----  ---------  -------------  ------  -------  -----------  --------------  ----------------  ----  -------------  --------------  -----------  ----------------  -----------  ----------------  --------  ---------------  ----------  ----------  --------------  ------------  -----------  --------  ---------  --------  -----  ---------  ------------------------  -----------  -----------  -----------------  ----------  -------


====================== cid = 13 ==========================
Cluster neurons:
[[0, 23], [0, 24], [1, 24], [2, 24], [0, 25], [1, 25], [2, 25], [3, 25], [0, 26], [1, 26], [2, 26], [3, 26], [4, 26], [0, 27], [1, 27], [2, 27], [3, 27], [4, 27], [5, 27], [0, 28], [1, 28], [2, 28], [3, 28], [4, 28], [5, 28], [0, 29], [1, 29], [2, 29], [3, 29], [4, 29], [5, 29], [6, 29]]

Existing cluster neurons:
[[0, 23], [0, 24], [1, 24], [2, 24], [0, 25], [1, 25], [2, 25], [3, 25], [0, 26], [1, 26], [2, 26], [3, 26], [4, 26], [0, 27], [1, 27], [2, 27], [3, 27], [4, 27], [5, 27], [0, 28], [1, 28], [2, 28], [3, 28], [4, 28], [5, 28], [0, 29], [1, 29], [2, 29], [3, 29], [4, 29], [5, 29], [6, 29]]

Matching movie IDs:
[1548, 1766, 1913, 3851, 4278, 4647, 4898, 680, 1287, 1503, 1613, 2308, 2351, 2468, 2690, 3610, 4485, 4891, 4940, 383, 523, 757, 4958, 365, 685, 2542, 2762, 3001, 3044, 3721, 571, 765, 916, 1403, 1943, 2252, 2336, 2652, 3040, 3746, 3972, 4028, 4062, 4802, 4947, 4986, 794, 1425, 2986, 3364, 3812, 4636, 4896, 354, 389, 507, 659, 738, 1113, 1217, 1300, 1469, 1470, 1592, 2339, 2344, 2735, 2847, 3910, 3853, 46, 74, 510, 1228, 1644, 2209, 2896, 3577, 4107, 3151, 4762, 81, 174, 699, 783, 1703, 1784, 1963, 2770, 3775, 4523, 4688, 13, 51, 1450, 1702, 2128, 2200, 2512, 3271, 3722, 4034, 4223, 4231, 299, 1035, 1305, 4823, 4824, 654, 902, 1591, 1776, 1819, 2374, 2666, 2968, 3143, 3147, 149, 1459, 1654, 1684, 2634, 3084, 3137, 3438, 3576, 3839, 4092, 4404, 4427, 4494, 155, 595, 2213, 3306, 3621, 3665, 3800, 4041, 4275, 4661, 109, 144, 926, 2122, 2139, 2179, 2908, 3226, 3523, 172, 242, 618, 889, 2277, 3551, 4119, 966, 1179, 2325, 3074, 6, 417, 1210, 1534, 1743, 2218, 2668, 4337, 4639, 4766, 4841, 2171, 2253, 3125, 3570, 4708, 4936, 894, 1547, 2243, 3852, 3922, 4452, 662, 2663, 938, 1167, 1220, 1926, 2031, 2269, 2441, 2755, 2772, 3205, 3406, 4152, 4315, 4363, 255, 1040, 1961, 1964, 2117, 4005, 4630, 4689, 358, 906, 1042, 2081, 2263, 2530, 3045, 3069, 3277, 3308, 3346, 3751, 4056, 4185, 368, 833, 2511, 3216, 3653, 180, 1036, 1231, 2015, 2134, 2651, 2694, 2919, 2992, 3030, 3626, 4535, 4969, 1385, 2367, 2926, 3190, 3521, 4351, 4460, 4962, 103, 139, 1460, 1650, 2063, 2741, 3094, 3269, 3611, 3947, 4024, 1439, 1775, 2026, 2250, 2295, 2920, 3126, 4145, 4965, 587, 1159, 1815, 1999, 2279, 3027, 3050, 3088]

Genres stats for these movies:
---------  ------  ---------------  -------  -------  ----  ----  -------------  -----------  ------  -----  ------------  --------  ------------  -------------  -------------  -----  ---------------  -------  ---------------  --------------------  -------  -------  ---------------  ------  ----------------------  -------------  ------  ------------------  ------------  -------------  ---------  --------------  ----------  ----------  -------------  ------------  -----------  ----------------  ----------------  --------------  -----------  -------------  ------  --------  -----------  --------  --------------  ---------------  -----------  ------------------  ------------------------------  ---------  --------------  -----------  -------------  ------  ------------  ------------------  ---------  -------  --------------  ---------  -----------  -----------------  ---------  -------  -----------------  -------------  --------------  ----------------  -----------
Category:  Horror  Science Fiction  Slasher  B-movie  Cult  Teen  Creature Film  Zombie Film  Comedy  Indie  Supernatural  Thriller  World cinema  Horror Comedy  Sci-Fi Horror  Drama  Black-and-white  Monster  Film adaptation  Natural horror films  Mystery  Fantasy  Japanese Movies  Parody  Psychological thriller  Monster movie  Action  Haunted House Film  Black comedy  Splatter film  Adventure  Costume Horror  Alien Film  Short Film  Crime Fiction  Romance Film  Comedy film  Television movie  Action/Adventure  Chinese Movies  Road-Horror  Doomsday film  Satire  Suspense  Gothic Film  Disaster  Romantic drama  Addiction Drama  Stoner film  Swashbuckler films  Sci Fi Pictures original films  Bollywood  Blaxploitation  Documentary  Comedy horror  Plague  Period piece  Computer Animation  Animation  Musical  Crime Thriller  Melodrama  Archaeology  Culture & Society  Detective  Z movie  Detective fiction  Feminist Film  Fantasy Comedy  Romantic fantasy  Jungle Film
Count:     278     61               52       44       34    32    31             28           27      23     21            17        16            16             15             14     11               11       8                8                     8        7        6                6       6                       6              6       5                   5             4              4          4               4           4           4              3             3            3                 3                 3               2            2              2       2         2            2         2               1                1            1                   1                               1          1               1            1              1       1             1                   1          1        1               1          1            1                  1          1        1                  1              1               1                 1
---------  ------  ---------------  -------  -------  ----  ----  -------------  -----------  ------  -----  ------------  --------  ------------  -------------  -------------  -----  ---------------  -------  ---------------  --------------------  -------  -------  ---------------  ------  ----------------------  -------------  ------  ------------------  ------------  -------------  ---------  --------------  ----------  ----------  -------------  ------------  -----------  ----------------  ----------------  --------------  -----------  -------------  ------  --------  -----------  --------  --------------  ---------------  -----------  ------------------  ------------------------------  ---------  --------------  -----------  -------------  ------  ------------  ------------------  ---------  -------  --------------  ---------  -----------  -----------------  ---------  -------  -----------------  -------------  --------------  ----------------  -----------


====================== cid = 12 ==========================
Cluster neurons:
[[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [0, 1], [1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [0, 2], [1, 2], [2, 2], [3, 2], [4, 2], [5, 2], [0, 3], [1, 3], [2, 3], [3, 3], [4, 3], [0, 4], [1, 4], [2, 4], [3, 4], [0, 5], [1, 5], [0, 6], [12, 13]]

Existing cluster neurons:
[[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [0, 1], [1, 1], [2, 1], [4, 1], [5, 1], [0, 2], [1, 2], [2, 2], [3, 2], [4, 2], [5, 2], [0, 3], [1, 3], [2, 3], [3, 3], [4, 3], [0, 4], [1, 4], [2, 4], [3, 4], [0, 5], [1, 5], [0, 6], [12, 13]]

Matching movie IDs:
[164, 744, 746, 1030, 1427, 1479, 1517, 1640, 1816, 1972, 2380, 2588, 2615, 2818, 3020, 3302, 3807, 4496, 4749, 4788, 304, 544, 987, 1025, 1744, 1810, 2077, 2475, 2753, 3072, 3195, 3601, 4054, 4384, 4892, 212, 350, 2365, 3107, 3263, 4383, 41, 258, 572, 1295, 1306, 1377, 3029, 3129, 4001, 4656, 195, 1653, 1890, 2248, 2258, 2286, 2538, 3992, 4048, 4130, 4368, 875, 1184, 1453, 3945, 4870, 648, 678, 1555, 1634, 2000, 2620, 2891, 1406, 2207, 3022, 3651, 4017, 4888, 586, 1111, 1688, 2389, 3763, 4296, 4511, 4546, 4795, 777, 1359, 2372, 1942, 3223, 710, 771, 1123, 2020, 2404, 3706, 3970, 4701, 4844, 430, 435, 633, 969, 1299, 1575, 1608, 1633, 2114, 4180, 24, 128, 439, 443, 663, 842, 1475, 1513, 1823, 2586, 2783, 3176, 3683, 4123, 4668, 4752, 4755, 1200, 4842, 229, 629, 816, 1921, 2094, 2552, 2705, 2837, 4035, 4375, 1238, 1801, 2501, 2786, 220, 881, 1260, 1699, 2522, 2601, 3108, 3258, 3510, 4206, 4209, 189, 1428, 3186, 3443, 3164, 118, 193, 1194, 1623, 2340, 57, 418, 936, 1101, 2118, 2172, 2273, 2316, 2603, 3731, 3879, 4211, 216, 2451, 2781, 3220, 3325, 3330, 36, 573, 628, 1201, 1852, 2434, 2814, 3311, 3888, 4757, 2869, 904, 1163, 1322, 1389, 1631, 1770, 1824, 2944, 3023, 3362, 4642, 4989, 317, 1804, 2355, 2831, 16, 1733, 1832, 2156, 2236, 3181, 3864, 1630, 3915]

Genres stats for these movies:
---------  -----  --------  ------------  ------  ---------------  ----------------  ------------  ---------------  --------  ------  ------------  ---------------  ------------------  -------  ---------  -----------------  ---------------  ------------  -----  ------------------  ---------  ---------  ---------------  --------  --------------  ----------------  ---------  ---  ----------  --------------  -------------  -------  -------  ------  -------  -----------------  -------  ----------------  -------------  ------------  -----------  ---------------  -----  -----------  ---------  --------------  --------  ----------------  ----------------  ----------------  ------------  ---------  -------------  -------------  -----------  --------  ----  --------------  ---------  ------------  -----------  ------------  ---------------  ------  -----------------  --------  -------  ---------------  -----------  ----------  -------------  --------------  -------------------  ---------------  ----------  -----------------  --------------  -----------  -----  ----------  -----------  ---------------  -------------  -----------------  ------  -----------  ---------------  --------  ----------------------
Category:  Drama  War film  World cinema  Action  Political drama  Action/Adventure  Period piece  Black-and-white  Thriller  Comedy  Romance Film  Film adaptation  Historical fiction  History  Adventure  Biographical film  Japanese Movies  Combat Films  Indie  Political thriller  Docudrama  Animation  Propaganda film  Art film  Romantic drama  Biopic [feature]  Biography  Spy  Short Film  Chinese Movies  Ensemble Film  Western  Musical  Horror  Mystery  Martial Arts Film  Fantasy  Political cinema  Costume drama  Comedy-drama  Silent film  Childhood Drama  Anime  Documentary  Melodrama  Samurai cinema  Suspense  Television movie  Action Thrillers  Historical drama  Family Drama  Film noir  Coming of age  Crime Fiction  Comedy film  Disaster  Epic  Christian film  Bollywood  Black comedy  Family Film  Supernatural  Courtroom Drama  Sports  Spaghetti Western  Cold War  Tragedy  Science Fiction  Avant-garde  Children's  Creature Film  Religious Film  Revisionist Western  Filipino Movies  Fairy tale  Experimental film  Existentialism  Escape Film  Music  Road movie  Educational  Nuclear warfare  Doomsday film  Culture & Society  Prison  Crime Drama  Historical Epic  Dystopia  Airplanes and airports
Count:     204    126       50            41      28               25                25            22               18        18      17            17               15                  15       12         12                 9                8             8      7                   6          6          6                5         5               5                 4          4    4           4               3              3        3        3       3        3                  3        3                 3              3             3            3                3      3            2          2               2         2                 2                 2                 2             2          2              2              2            2         2     2               2          2             2            1             1                1       1                  1         1        1                1            1           1              1               1                    1                1           1                  1               1            1      1           1            1                1              1                  1       1            1                1         1
---------  -----  --------  ------------  ------  ---------------  ----------------  ------------  ---------------  --------  ------  ------------  ---------------  ------------------  -------  ---------  -----------------  ---------------  ------------  -----  ------------------  ---------  ---------  ---------------  --------  --------------  ----------------  ---------  ---  ----------  --------------  -------------  -------  -------  ------  -------  -----------------  -------  ----------------  -------------  ------------  -----------  ---------------  -----  -----------  ---------  --------------  --------  ----------------  ----------------  ----------------  ------------  ---------  -------------  -------------  -----------  --------  ----  --------------  ---------  ------------  -----------  ------------  ---------------  ------  -----------------  --------  -------  ---------------  -----------  ----------  -------------  --------------  -------------------  ---------------  ----------  -----------------  --------------  -----------  -----  ----------  -----------  ---------------  -------------  -----------------  ------  -----------  ---------------  --------  ----------------------


====================== cid = 10 ==========================
Cluster neurons:
[[6, 2], [7, 2], [27, 6], [28, 6], [27, 7], [28, 7], [29, 7], [27, 8], [28, 8], [29, 8], [27, 9], [28, 9], [29, 9], [26, 10], [27, 10], [28, 10], [29, 10], [26, 11], [27, 11], [28, 11], [29, 11], [26, 12], [27, 12], [28, 12], [29, 12], [27, 13], [28, 13], [29, 13]]

Existing cluster neurons:
[[6, 2], [7, 2], [27, 6], [28, 6], [27, 7], [28, 7], [29, 7], [27, 8], [28, 8], [29, 8], [27, 9], [28, 9], [29, 9], [26, 10], [27, 10], [28, 10], [29, 10], [26, 11], [27, 11], [28, 11], [29, 11], [26, 12], [27, 12], [29, 12], [27, 13], [28, 13], [29, 13]]

Matching movie IDs:
[314, 337, 636, 808, 1599, 1858, 2595, 3966, 489, 1564, 4653, 4831, 209, 491, 686, 1559, 1768, 2789, 3025, 3241, 3412, 4183, 4388, 4539, 4570, 725, 3313, 4293, 2125, 528, 617, 2287, 3368, 4489, 454, 2866, 3820, 134, 1563, 2058, 2444, 53, 774, 1079, 2958, 3082, 3227, 3843, 3890, 4598, 381, 485, 497, 836, 1020, 1068, 1584, 1779, 2104, 2296, 2779, 3058, 3490, 3730, 4505, 4748, 378, 1272, 4250, 2388, 3718, 859, 907, 1544, 2233, 2810, 3299, 4242, 4544, 1315, 2088, 263, 886, 1033, 1198, 1680, 1785, 2274, 2710, 4290, 1992, 2040, 3035, 2364, 2972, 3224, 4490, 4551, 4679, 495, 1960, 4128, 15, 1496, 1639, 1847, 1939, 2150, 2523, 3017, 3382, 3552, 3607, 208, 1865, 1987, 2178, 2763, 2830, 3062, 3219, 4807, 667, 2792, 3499, 4662, 4859, 1565, 1846, 2168, 2181, 3703, 186, 834, 837, 1795, 2520, 2876, 3370, 3495, 3714, 511, 814, 914, 1212, 1482, 1867, 1936, 2863, 2865, 3468, 3649, 3865, 3903, 3953, 4129, 4163, 4277, 4520, 4583, 4641, 487, 1126, 1668, 369, 2075, 2120, 2175, 2387, 2806, 4565, 4686, 422, 1115, 1342, 2518, 2667, 3100]

Genres stats for these movies:
---------  ------  -----  ------------  -----  ------------  ------------  ------------  -------------  ------  -----------  ----  ---------------  ------------  -------  -------------  ------  ------------  ----------  ---------------  -----------------  ---------  ----  ---------------  ---------------  -------  -----------------  ------  --------  ---------  ----------------  ---------  ---------------  ---------  ----------  --------------  -----------------  ---------------  -------  ----------  ------------  --------------  --------------  ---------  ----------------  --------  --------  -----------  ----------  --------------  ---------------  ----------  -------------  -------------  ----------------  -----  -------------  -----  --------------  -----------  -----------------  -----------  -----------  -------------  ----------------  -------------------  ------  ------  --------------  ------  --------------  -------------------------  ------------  ------------  -------------  -----------  -----------  ---------------  ---------  ---------------  ----------------  -------------
Category:  Comedy  Drama  Comedy-drama  Indie  Romance Film  Black comedy  World cinema  Coming of age  Satire  Family Film  Teen  Film adaptation  Period piece  Musical  Ensemble Film  Parody  Family Drama  Road movie  Domestic Comedy  Comedy of manners  Melodrama  Cult  Black-and-white  Japanese Movies  Fantasy  Children's/Family  Sports  Art film  Americana  Political cinema  Adventure  Romantic comedy  Biography  Short Film  Chinese Movies  Biographical film  Science Fiction  Mystery  Sex comedy  Media Satire  Fantasy Comedy  Romantic drama  Slapstick  Comedy of Errors  Thriller  War film  Tragicomedy  Children's  Comedy Western  Political drama  Buddy film  New Hollywood  Costume drama  Biopic [feature]  Music  Crime Fiction  Dance  Marriage Drama  Documentary  Experimental film  Silent film  Stoner film  Musical Drama  Television movie  Slice of life story  Action  Prison  Musical comedy  Horror  Hip hop movies  Heaven-Can-Wait Fantasies  Exploitation  Erotic Drama  Doomsday film  Crime Drama  Comedy film  Childhood Drama  Bollywood  Addiction Drama  Romantic fantasy  Feminist Film
Count:     177     175    58            46     26            23            20            14             13      13           12    11               10            9        9              8       8             7           6                6                  6          6     5                5                5        4                  4       4         4          4                 4          4                3          3           3               3                  3                3        3           3             3               3               3          3                 3         3         2            2           2               2                2           2              2              2                 2      2              2      2               2            2                  2            2            2              1                 1                    1       1       1               1       1               1                          1             1             1              1            1            1                1          1                1                 1
---------  ------  -----  ------------  -----  ------------  ------------  ------------  -------------  ------  -----------  ----  ---------------  ------------  -------  -------------  ------  ------------  ----------  ---------------  -----------------  ---------  ----  ---------------  ---------------  -------  -----------------  ------  --------  ---------  ----------------  ---------  ---------------  ---------  ----------  --------------  -----------------  ---------------  -------  ----------  ------------  --------------  --------------  ---------  ----------------  --------  --------  -----------  ----------  --------------  ---------------  ----------  -------------  -------------  ----------------  -----  -------------  -----  --------------  -----------  -----------------  -----------  -----------  -------------  ----------------  -------------------  ------  ------  --------------  ------  --------------  -------------------------  ------------  ------------  -------------  -----------  -----------  ---------------  ---------  ---------------  ----------------  -------------


====================== cid = 15 ==========================
Cluster neurons:
[[6, 0], [7, 0], [8, 0], [9, 0], [10, 0], [11, 0], [12, 0], [7, 1], [8, 1], [9, 1], [10, 1], [11, 1], [12, 1], [8, 2], [9, 2], [10, 2], [11, 2], [12, 2], [9, 3], [10, 3], [11, 3]]

Existing cluster neurons:
[[6, 0], [7, 0], [8, 0], [9, 0], [10, 0], [11, 0], [12, 0], [7, 1], [8, 1], [9, 1], [10, 1], [11, 1], [12, 1], [8, 2], [9, 2], [10, 2], [11, 2], [12, 2], [9, 3], [10, 3], [11, 3]]

Matching movie IDs:
[756, 1188, 1411, 2060, 2626, 2788, 4053, 4210, 4853, 1077, 2090, 2201, 3689, 4357, 4585, 4886, 160, 1602, 1750, 1998, 2682, 2967, 4349, 4607, 4742, 880, 1670, 2242, 2428, 2959, 3103, 3266, 3369, 3437, 3229, 4147, 318, 359, 965, 1124, 2140, 3300, 3367, 3804, 2280, 3733, 114, 2073, 2560, 3182, 3887, 576, 1529, 2507, 997, 2870, 2888, 3262, 3913, 4018, 977, 1084, 2450, 2801, 4095, 952, 1701, 1993, 3354, 3783, 4462, 4680, 769, 995, 4276, 1155, 1603, 3563, 3878, 4079, 4566, 4573, 3944, 4362, 4702, 615, 1018, 1086, 2354, 3658, 3743, 4259, 4563, 591, 2176, 4287, 4897, 1447, 3911, 4179, 720, 878, 1635, 2149, 2826, 202, 428, 975, 3373]

Genres stats for these movies:
---------  -----  --------------  ------------  ----------------  -----------------  ------  ------  ------------  --------  -------------  ------  -----  --------------  -----------  -------  -----  ---------  ---------------  -------  ---------------  --------------  --------  ------  -----------  ------------  ---------------  ------------  ---------  ------------  -------------  -------------  -------  -------  ------------  ---------------  ----------  -------  -----------  ------------  ---------------  -----------  -------  ---------  ---------------  --------------  ----------------------  -----  -------------  ------------  ----------------  -----------------  ---------  ----------------  --------------  ---------  ----------  ---------------  ----------  -----------------  ----  -------------  ----  -----------------  -------------  ---------------  ---------------  ----------------  ------------------  -------------  ---------------------  ---------  -----------
Category:  Drama  Chinese Movies  World cinema  Action/Adventure  Martial Arts Film  Action  Comedy  Romance Film  Thriller  Crime Fiction  Horror  Wuxia  Romantic drama  Family Film  Mystery  Indie  Adventure  Romantic comedy  Fantasy  Japanese Movies  Crime Thriller  War film  Sports  Comedy film  Period piece  Black-and-white  Comedy-drama  Animation  Family Drama  Coming of age  Action Comedy  Tragedy  History  Black comedy  Filipino Movies  Short Film  Musical  Silent film  Supernatural  Political drama  Stoner film  Slasher  Slapstick  Science Fiction  Musical comedy  Psychological thriller  Music  Gangster Film  Media Satire  Action Thrillers  Biographical film  Biography  Biopic [feature]  Blaxploitation  Bollywood  Buddy film  Childhood Drama  Children's  Comedy of manners  Cult  Ensemble Film  Epic  Fantasy Adventure  Feminist Film  Film adaptation  Historical Epic  Historical drama  Historical fiction  Horror Comedy  Master Criminal Films  Melodrama  Zombie Film
Count:     49     48              38            28                27                 26      26      20            17        10             10      9      9               7            7        6      6          5                5        5                5               5         5       4            4             4                4             3          3             2              2              2        2        2             2                2           2        2            1             1                1            1        1          1                1               1                       1      1              1             1                 1                  1          1                 1               1          1           1                1           1                  1     1              1     1                  1              1                1                1                 1                   1              1                      1          1
---------  -----  --------------  ------------  ----------------  -----------------  ------  ------  ------------  --------  -------------  ------  -----  --------------  -----------  -------  -----  ---------  ---------------  -------  ---------------  --------------  --------  ------  -----------  ------------  ---------------  ------------  ---------  ------------  -------------  -------------  -------  -------  ------------  ---------------  ----------  -------  -----------  ------------  ---------------  -----------  -------  ---------  ---------------  --------------  ----------------------  -----  -------------  ------------  ----------------  -----------------  ---------  ----------------  --------------  ---------  ----------  ---------------  ----------  -----------------  ----  -------------  ----  -----------------  -------------  ---------------  ---------------  ----------------  ------------------  -------------  ---------------------  ---------  -----------


```

In [ ]:
# Test: See suggested movies
ts = []
for t in ts:
  print(titles[t], categories[t], corpus[t], sep="\n")
  print()


## Tips για το SOM και το clustering

- Για την ομαδοποίηση ένα U-matrix καλό είναι να εμφανίζει και μπλε-πράσινες περιοχές (clusters) και κόκκινες περιοχές (ορίων). Παρατηρήστε ποια σχέση υπάρχει μεταξύ αριθμού ταινιών στο final set, μεγέθους grid και ποιότητας U-matrix.
- Για το k του k-Means προσπαθήστε να προσεγγίζει σχετικά τα clusters του U-matrix (όπως είπαμε είναι διαφορετικοί μέθοδοι clustering). Μικρός αριθμός k δεν θα σέβεται τα όρια. Μεγάλος αριθμός θα δημιουργεί υπο-clusters εντός των clusters που φαίνονται στο U-matrix. Το τελευταίο δεν είναι απαραίτητα κακό, αλλά μεγαλώνει τον αριθμό clusters που πρέπει να αναλυθούν σημασιολογικά.
- Σε μικρούς χάρτες και με μικρά final sets δοκιμάστε διαφορετικές παραμέτρους για την εκπαίδευση του SOM. Σημειώστε τυχόν παραμέτρους που επηρεάζουν την ποιότητα του clustering για το dataset σας ώστε να τις εφαρμόσετε στους μεγάλους χάρτες.
- Κάποια τοπολογικά χαρακτηριστικά εμφανίζονται ήδη σε μικρούς χάρτες. Κάποια άλλα χρειάζονται μεγαλύτερους χάρτες. Δοκιμάστε μεγέθη 20x20, 25x25 ή και 30x30 και αντίστοιχη προσαρμογή των k. Όσο μεγαλώνουν οι χάρτες, μεγαλώνει η ανάλυση του χάρτη αλλά μεγαλώνει και ο αριθμός clusters που πρέπει να αναλυθούν.




## Ανάλυση τοπολογικών ιδιοτήτων χάρτη SOM

Μετά το πέρας της εκπαίδευσης και του clustering θα έχετε ένα χάρτη με τοπολογικές ιδιότητες ως προς τα είδη των ταίνιών της συλλογής σας, κάτι αντίστοιχο με την εικόνα στην αρχή της Εφαρμογής 2 αυτού του notebook. Η συγκεκριμένη εικόνα είναι μόνο για εικονογράφιση, δεν είναι χάρτης SOM καιδεν έχει καμία σχέση με τη συλλογή δεδομένων και τις κατηγορίες μας.

Για τον τελικό χάρτη SOM που θα παράξετε για τη συλλογή σας, αναλύστε σε markdown με συγκεκριμένη <u>αναφορά σε αριθμούς clusters</u> και τη <u>σημασιολογική ερμηνεία τους</u> τις εξής τρεις τοπολογικές ιδιότητες του SOM: 

1. Δεδομένα που έχουν **μεγαλύτερη πυκνότητα πιθανότητας** στο χώρο **εισόδου** τείνουν να **απεικονίζονται με περισσότερους νευρώνες** στο χώρο μειωμένης διαστατικότητας. Δώστε παραδείγματα από συχνές και λιγότερο συχνές κατηγορίες ταινιών. Χρησιμοποιήστε τις στατιστικές των κατηγοριών στη συλλογή σας και τον αριθμό κόμβων που χαρακτηρίζουν.
2. **Μακρινά πρότυπα εισόδου** τείνουν να απεικονίζονται απομακρυσμένα στο χάρτη. Υπάρχουν χαρακτηριστικές κατηγορίες ταινιών που ήδη από μικρούς χάρτες τείνουν να τοποθετούνται σε διαφορετικά ή **απομονωμένα σημεία του χάρτη**.
3. **Κοντινά πρότυπα εισόδου** τείνουν να απεικονίζονται **κοντά** στο χάρτη. Σε μεγάλους χάρτες εντοπίστε είδη ταινιών και κοντινά τους υποείδη.

Προφανώς τοποθέτηση σε 2 διαστάσεις που να σέβεται μια απόλυτη τοπολογία δεν είναι εφικτή, αφενός γιατί δεν υπάρχει κάποια απόλυτη εξ ορισμού για τα κινηματογραφικά είδη ακόμα και σε πολλές διαστάσεις, αφετέρου γιατί πραγματοποιούμε μείωση διαστατικότητας.

Εντοπίστε μεγάλα clusters και μικρά <u>clusters που δεν έχουν σαφή χαρακτηριστικά</u>. Εντοπίστε clusters συγκεκριμένων ειδών που μοιάζουν να <u>μην έχουν τοπολογική συνάφεια</u> με γύρω περιοχές. Προτείνετε πιθανές ερμηνείες.


Τέλος, εντοπίστε clusters που έχουν κατά την άποψή σας <u>ιδιαίτερο ενδιαφέρον</u> στη συλλογή της ομάδας σας (data exploration / discovery value) και σχολιάστε.


***Σχολιασμός αποτελεσμάτων***

Ύστερα από πειράματα, καταλήγουμε στο ότι το καλύτερο πλέγμα SOM έχει 30 νευρώνες, είναι επίπεδο και ορθογωνικό, με τυχαία αρχικοποίηση βαρών και γκαουσσιανή γειτονιά. Γίνεται χρήση 20 clusters, καθώς θεωρούμε ότι τα 25 clusters είναι πολλά αν λάβουμε υπόψη τα κυρίαρχα είδη ταινίας (δηλαδή με 25 θα εμφανίζεται πολλές φορές το ίδιο κύριο είδος, σε πολλά clusters, με μικρές διαφορές στα λιγότερο κυρίρχα).

Το πρώτο που επιβεβαιώνουμε είναι ότι τα πιο συχνά είδη ταινίας στην είσοδο απασχολούν πολλούς νευρώνες. Το είδος Drama που είναι το πιο συχνό με 13% εμφάνισης στις ταινίες περιέχεται σχεδόν σε όλα τα clusters. Για παράδειγμα το `cid = 3` είναι το κύριο cluster δράματος απασχολώντας 136 νευρώνες, που είναι σχεδόν το διπλάσιο από το επόμενο μεγαλύτερο cluster. Στα `cid = 16, 4, 12, 15` είναι το πιο συχνό είδος, ενώ στα `cid = 5, 8, 17, 10` το δεύτερο πιο συχνό. Παρόμοια είναι η εικόνα για το Comedy (7%). Αντιθέτως, στο cluster `15` όπου έχουν μαζευτεί όλες οι Chinese Movies, 110 εμφανίσεις με 0.63%, είναι το cluster με τους λιγότερους νευρώνες (21).

Έχουμε να παρατηρήσουμε ότι τα μακρινά πρότυπα πράγματι απέχουν στον χάρτη. Για παράδειγμα, τα Documentary ανήκουν στο cluster με `cid = 1` και τα περισσότερα Zombie films ανήκουν στο cluster `13`, τα πρώτα βρίσκονται κοντά στη μέση τη U-matrix ενά τα δεύτερα στην κάτω αριστερή γωνία απομονωμένα. Πιο πολύ από τα Zombie films απέχει το cluster `8` με τις ρομαντικές ταινίες (Romance Films) που βρίσκεται στην πάνω δεξιά γωνία, πράγμα αναμενόμενο.

Αντίστοιχα, τα κοντινά πρότυπα είναι γενικά μαζί στον χάρτη: Πηγαίνοντας από την κάτω αριστερή γωνία προς το κέντρο του U-matrix, μετά τις ταινίες με zombie βρίσκεται το cluster `19` που αφορά ταινίες τρόμου (Thriller, Horror). Σαν δεύτερο παράδειγμα, το `cid = 12` όπου το κύριο είδος είναι το δράμα με υποείδη τις ταινίες πολέμου και πολιτικής βρίσκεται κοντά στο κύριο cluster του δράματος (`cid = 3`). Σαν τρίτο παράδειγμα, το cluster `10` με υποείδος Comedy-drama έχει κύρια είδη Comedy και Drama.


Ένα μεγάλο cluster χωρίς σαφή χαρακτηριστικά είναι το `3` που, προφανώς, αναπαριστά γενικά τις ταινίες δράματος σαν είδος. Ένα μικρό παρόμοιο cluster είναι το `16`: περίεχει κυρίως Drama, Comedy και Romance Films, αλλά και Japanese Movies, Action, Horror. Ενδεχομένως, αυτό το cluster αναφέρται σε ταινίες που είναι κατά βάση ρομαντικές αλλά περιέχουν και σκηνές δράσης.

Η ασάφεια παρατηρείται και μεταξύ των clusters. Για παράδειγμα, το cluster `1` που αφορά Ντοκιμαντέρ βρίσκεται δίπλα στο cluster με τις ταινίες Western, `2`. Ερμηνεία: Εφόσον οι Western ταινίες διαδραματίζονται σε παλαιά εποχή, έχει ιστορική αξία ο τρόπος ζωής των ανθρώπων τότε, παρόλο που οι σκηνές επικεντρώνονται στη δράση.

Ενδιαφέρον σύνδεσης παρουσιάζει αυτό που αναφέρθηκε στην προηγούμενη παράγραφο για τα ντοκιμαντέρ και τις ταινίες western. Ακόμη, το ότι κάποια Comedy films βρίσκονται στο ίδιο cluster (`18`) με τα Silent films: Η έλλειψη διαλόγου προωθεί την κωμωδία.

Τέλος, αξίζει να αναφέρουμε τις παρατηρήσεις μας από τις διαφορετικές παραμέτρους στο SOM. Παρατηρήσαμε ότι η τοροειδής τοπολογία δεν παράγει αισθητή βελτίωση, το ανάποδο μάλιστα, καθώς στις δοκιμές μας ομαδοποιεί το κύριο cluster από το είδος ταινίας Documentary με πολλά Thriller. Αντιθέτως, ο εξαγωνικός χάρτης διαχωρίζει πολύ καλά τα Documentary, αλλά ο τρόπος που το Somoclu εμφανίζει το U-matrix αποτρέπει την ερμηνεία (βλ. παράδειγμα στην επόμενη εικόνα), για αυτόν τον λόγο δεν έγινε εκτενής μελέτη του. Ομοίως, η γειτονιά Bubble εμφανίζει περισσότερα κόκκινα σημεία στον χάρτη και είδαμε ότι το Documentary ομαδοποιείται με το Comedy, πράγμα ανεπιθύμητο. Η αρχικοποίηση PCA δε βλετιώνει τη γειτονιά Bubble διότι εμφανίζει U-matrix με πολλές κόκκινες περιοχές. Γενικά, η PCA βελτίωσε το τοροειδές πλέγμα, αλλά φαίνεται πως δυσκόλεψε τον διαχωρισμό των ειδών μετάξυ τους.

![κατάλογος.jpg](https://i.postimg.cc/hGTJ2tj2/image.jpg)

*Το εξαγωνικό SOM grid παράγει U-matrix όπως αυτό της εικόνας, για οποιονδήποτε αριθμό dcouments και πλήθος νευρώνων. Από αυτό δεν είναι σαφής η ποιότητα του kmeans clustering, για αυτό είναι αδύνατη η σύγκριση με το τετραγωνικό πλέγμα. Έτσι, δεν αξίζει να ασχοληθούμε με αυτό, παρόλο που διαχώρισε πλήρως την κατηγορία των Ντοκιμαντέρ στις εκτελέσεις μας (ελέγχθηκε με τη `neuron_movies_report()`).*


# Τελική παράδοση άσκησης

- Θα παραδώσετε στο helios το παρόν notebook επεξεργασμένο ή ένα ή δύο νέα zipαρισμένα με τις απαντήσεις σας για τα ζητούμενα και των δύο εφαρμογών. 
- Θυμηθείτε ότι η ανάλυση του χάρτη στο markdown <u>με αναφορά σε αριθμούς clusters</u> πρέπει να αναφέρεται στον τελικό χάρτη με τα κελιά ορατά που θα παραδώσετε αλλιώς ο χάρτης που θα προκύψει θα είναι διαφορετικός και τα labels των clusters δεν θα αντιστοιχούν στην ανάλυσή σας. 
- Μην ξεχάσετε στην αρχή ένα κελί markdown με **τα στοιχεία της ομάδας σας**.

<table>
  <tr><td align="center">
    <font size="4">Παρακαλούμε διατρέξτε βήμα-βήμα το notebook για να μην ξεχάσετε παραδοτέα</font>
</td>
  </tr>
</table>